1 subject. Feature extraction. Classification joining all frequency bands.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv

#def select_file(file):
    #Lectura mediante pandas
f = "data_preprocessed_python\\s01.dat"
subject_id = 1
dictRaw = pd.read_pickle(f)

labels = dictRaw.get('labels')

dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

data = dictRaw.get('data')

df_videos = {}
    #Recorrido de cada una de las columnas de la tabla 3d, y guardado en una una lista de dataframes para cada uno de los videos.
for i in range(40):
    df_videos[i] = pd.DataFrame(data[i])
    df_videos[i] = df_videos[i].transpose()
    df_videos[i].columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature']
    df_videos[i].drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature'],axis = 'columns', inplace=True)

In [2]:
    # Frecuencia de muestreo
    fs = 128
    # Window
    window = "hann"
    # Length of each segment
    # nperseg = 256 por defecto
    # noverlap
    # Por defecto a None, if None : noverlap = nperseg / 2
    # [...]

    # Definicion de bandas
    eeg_bands = {'Delta': (1, 4),
                'Theta': (4, 8),
                'Alpha': (8, 14),
                'Beta': (14, 30),
                'Gamma': (30, 50)}

In [3]:
# FOURIER
import scipy
from scipy import signal

# SE COGE EL PRIMER VÍDEOS, Df_VIDEOS[0]
def do_stft(video, channel):
    # Array 
    x = df_videos[video][channel]

    # Array of the sample frequency, Array of the segment times, STFT of x
    f, t , Zxx = scipy.signal.stft(x, fs, window)
    return Zxx


In [4]:
#SEPARACION EN LAS DISTINTAS BANDAS DE FRECUENCIA

def make_bands(Zxx):
    #Obtener valores reales de STFT, solo positivos.
    values = np.absolute(Zxx)

    # Get frequencies for amplitudes in Hz
    fft_freq = np.fft.rfftfreq(len(data), 1.0/fs)
    #print(fft_freq)

    # Obtención de cada una de las bandas de frecuencia
    eeg_band_fft = dict()
    for band in eeg_bands:  
        freq_ix = np.where((fft_freq >= eeg_bands[band][0]) & (fft_freq <= eeg_bands[band][1]))[0]
        eeg_band_fft[band] = values[freq_ix]
    return eeg_band_fft


In [5]:
# Ahora hay que hacer las medias sobre cada banda de frecuencia. La media entre cada una de las posiciones de los bloques. Hacer la media entre todas las posiciones 0, 1, 2... 63 de cada uno de los bloques
import numpy as np

def channel_freq(eeg_band_fft):
        
        freq = np.zeros((5, 64))        # 5 bandas, 64 posiciones de los datos

        # Se va a recorrer {Delta, Theta, Alpha, Beta, Gamma}
        for band in eeg_bands:

                # Para cada una de las 64 posiciones del array
                for j in range (0, 64):
                        
                        val = []
                        # Se va a recorrer cada uno de los arrays que hay en cada una de las bandas
                        for i in range (0, len(eeg_band_fft[band])):
                                val.append(eeg_band_fft[band][i][j])
                                # Ejemplo : eeg_band_fft['Delta'][0][0*0 + 0]
                        if (band == "Delta"):
                                freq[0][j] = np.mean(val)
                        elif (band == "Theta"):
                                freq[1][j] = np.mean(val)
                        elif (band == "Alpha"):
                                freq[2][j] = np.mean(val)
                        elif (band == "Beta"):
                                freq[3][j] = np.mean(val)
                        elif (band == "Gamma"):
                                freq[4][j] = np.mean(val)
        return(freq)     # En freq tenemos las frecuencias medias de cada una de las bandas

# Recordamos : Estamos tratando los datos relativos al video 0, un canal concreto de un sujeto concreto

In [6]:
#df_file_videos = select_file('d\\s01.dat')
# Obtener la división por bandas de frecuencia de cada uno de los canales del video 0
for video in range(0, 40):
    delta_bands = []
    theta_bands = []
    beta_bands = []
    alpha_bands = []
    gamma_bands = []

    for channel in ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']:
        Zxx = do_stft(video, channel)
        eeg_band_fft = make_bands(Zxx)
     #   print(video)
      #  print("\n")
       # print(channel)
        freq = channel_freq(eeg_band_fft)

        delta_bands.append(freq[0])
        theta_bands.append(freq[1])
        alpha_bands.append(freq[2])
        beta_bands.append(freq[3])
        gamma_bands.append(freq[4])
        
#        print(freq[0])
 #       print("\n\n")
    # En freq tenemos la media de todas las bandas de frecuencia de cada uno de los canales del vídeo 0

In [7]:
# Banda delta
df = pd.DataFrame(delta_bands)
delta_df = df.transpose()
delta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
delta_df

rows = list()
for _,row in delta_df.iterrows():
    rows += [row]*40
aux_delta = pd.DataFrame(rows).reset_index(drop=True)

In [8]:
# Banda theta
df = pd.DataFrame(theta_bands)
theta_df = df.transpose()
theta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
theta_df

rows = list()
for _,row in theta_df.iterrows():
    rows += [row]*40
aux_theta = pd.DataFrame(rows).reset_index(drop=True)


In [9]:
#Bnada alpha
df = pd.DataFrame(alpha_bands)
alpha_df = df.transpose()
alpha_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
alpha_df

rows = list()
for _,row in alpha_df.iterrows():
    rows += [row]*40
aux_alpha = pd.DataFrame(rows).reset_index(drop=True)

In [10]:
# Banda beta
df = pd.DataFrame(beta_bands)
beta_df = df.transpose()
beta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
beta_df

rows = list()
for _,row in beta_df.iterrows():
    rows += [row]*40
aux_beta = pd.DataFrame(rows).reset_index(drop=True)

In [11]:
# Banda gamma
df = pd.DataFrame(gamma_bands)
gamma_df = df.transpose()
gamma_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
gamma_df

rows = list()
for _,row in gamma_df.iterrows():
    rows += [row]*40
aux_gamma = pd.DataFrame(rows).reset_index(drop=True)

In [12]:
df_labels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])
rows = list()
for _,row in dfLabels.iterrows():
    rows += [row]*64
aux_labels = pd.DataFrame(rows).reset_index(drop=True)

In [13]:
data_delta = aux_delta.merge(aux_labels, left_index=True, right_index=True)
data_theta = aux_theta.merge(aux_labels, left_index=True, right_index=True)
data_alpha = aux_alpha.merge(aux_labels, left_index=True, right_index=True)
data_beta = aux_beta.merge(aux_labels, left_index=True, right_index=True)
data_gamma = aux_gamma.merge(aux_labels, left_index=True, right_index=True)

# Todas las bandas una al lado de otra

In [14]:
frames = [aux_delta, aux_theta, aux_alpha, aux_beta, aux_gamma]
aux_all_bands = pd.concat(frames, axis=1)
all_bands_df = aux_all_bands.merge(aux_labels, left_index=True, right_index=True)

In [15]:
data = all_bands_df.drop(['dominance', 'liking'], axis=1)

conditions = [
    ((data['valence'] <4.5) & (data['arousal']<4.5)),
    ((data['valence'] <4.5) & (data['arousal'] > 4.5)),
    ((data['valence'] >4.5) & (data['arousal']<4.5)),
    ((data['valence'] >4.5) & (data['arousal'] > 4.5))]
choices = ['sadness', 'pleasure', 'anger', 'joy']

data['valence'] = np.select(conditions, choices, default='black')
data = data.rename(columns={'valence' : 'label'})
data = data.drop(columns='arousal')

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['label']]

x = copia.drop(columns=['label'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

param_distribs_rf = {
    'max_depth': [3, 5, 10],
    'min_samples_split': [2, 5, 10]
}

param_distribs_svm = {
    'kernel': ('linear', 'rbf'),
    'C': [0.1, 1.0, 10.0, 100.0]
}

param_distribs_knn = {
    'n_neighbors': [3,5,6,7,8,9,10],
    'weights': ['uniform', 'distance']
}

from sklearn.model_selection import GridSearchCV

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment6_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writeheader()
    w.writerow(params_dict)

rf_acc = accuracy_score(yTest, predict)
svm_acc = accuracy_score(yTest, svm_predict)
knn_acc = accuracy_score(yTest, knn_predict)

<ipython-input-15-b42ddd358f6d>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['valence'] = np.select(conditions, choices, default='black')
[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  56 tasks      | elapsed:    2.3s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    2.5s finished
[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  80 tasks      | elapsed:    0.2s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:   

In [16]:
from sklearn.metrics import precision_score

rf_prec_sadness = precision_score(yTest, predict, average="micro", labels=["sadness"])
rf_prec_pleasure = precision_score(yTest, predict, average="micro", labels=["pleasure"])
rf_prec_anger = precision_score(yTest, predict, average="micro", labels=["anger"])
rf_prec_joy = precision_score(yTest, predict, average="micro", labels=["joy"])


svm_prec_sadness = precision_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_prec_pleasure = precision_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_prec_anger = precision_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_prec_joy = precision_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_prec_sadness = precision_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_prec_pleasure = precision_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_prec_anger = precision_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_prec_joy = precision_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import recall_score

rf_rec_sadness = recall_score(yTest, predict, average="micro", labels=["sadness"])
rf_rec_pleasure = recall_score(yTest, predict, average="micro", labels=["pleasure"])
rf_rec_anger = recall_score(yTest, predict, average="micro", labels=["anger"])
rf_rec_joy = recall_score(yTest, predict, average="micro", labels=["joy"])


svm_rec_sadness = recall_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_rec_pleasure = recall_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_rec_anger = recall_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_rec_joy = recall_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_rec_sadness = recall_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_rec_pleasure = recall_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_rec_anger = recall_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_rec_joy = recall_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import f1_score

rf_f1_sadness = f1_score(yTest, predict, average="micro", labels=["sadness"])
rf_f1_pleasure = f1_score(yTest, predict, average="micro", labels=["pleasure"])
rf_f1_anger = f1_score(yTest, predict, average="micro", labels=["anger"])
rf_f1_joy = f1_score(yTest, predict, average="micro", labels=["joy"])


svm_f1_sadness = f1_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_f1_pleasure = f1_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_f1_anger = f1_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_f1_joy = f1_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_f1_sadness = f1_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_f1_pleasure = f1_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_f1_anger = f1_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_f1_joy = f1_score(yTest, knn_predict, average="micro", labels=["joy"])

In [17]:
col_names = ['Subject',
             'Experiment ID',
             'Feature extraction',
             'Band selection',
             'Channel selection',
             'Classification algorithm',
             'Accuracy',
             'Sadness precision',
             'Pleasure precision',
             'Anger precision',
             'Joy precision',
             'Sadness recall',
             'Pleasure recall',
             'Anger recall',
             'Joy recall',
             'Sadness f1-score',
             'Pleasure f1-score',
             'Anger f1-score',
             'Joy f1-score']

data_CSV = [subject_id, 6, "STFT", "All bands", "-", "Random Forest", rf_acc, rf_prec_sadness, rf_prec_pleasure, rf_prec_anger, rf_prec_joy, rf_rec_sadness, rf_rec_pleasure, rf_rec_anger, rf_rec_joy, rf_f1_sadness, rf_f1_pleasure, rf_f1_anger, rf_f1_joy], [subject_id, 6, "STFT", "All bands", "-", "SVM", svm_acc, svm_prec_sadness, svm_prec_pleasure, svm_prec_anger, svm_prec_joy, svm_rec_sadness, svm_rec_pleasure, svm_rec_anger, svm_rec_joy, svm_f1_sadness, svm_f1_pleasure, svm_f1_anger, svm_f1_joy], [subject_id, 6, "STFT", "All bands", "-", "kNN", knn_acc, knn_prec_sadness, knn_prec_pleasure, knn_prec_anger, knn_prec_joy, knn_rec_sadness, knn_rec_pleasure, knn_rec_anger, knn_rec_joy, knn_f1_sadness, knn_f1_pleasure, knn_f1_anger, knn_f1_joy]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment6_1.csv", mode="a", header=col_names, index=False)

In [18]:
f = "data_preprocessed_python\\s02.dat"
subject_id = 2
dictRaw = pd.read_pickle(f)

labels = dictRaw.get('labels')

dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

data = dictRaw.get('data')

df_videos = {}
    #Recorrido de cada una de las columnas de la tabla 3d, y guardado en una una lista de dataframes para cada uno de los videos.
for i in range(40):
    df_videos[i] = pd.DataFrame(data[i])
    df_videos[i] = df_videos[i].transpose()
    df_videos[i].columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature']
    df_videos[i].drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature'],axis = 'columns', inplace=True)

for video in range(0, 40):
    delta_bands = []
    theta_bands = []
    beta_bands = []
    alpha_bands = []
    gamma_bands = []

    for channel in ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']:
        Zxx = do_stft(video, channel)
        eeg_band_fft = make_bands(Zxx)
     #   print(video)
      #  print("\n")
       # print(channel)
        freq = channel_freq(eeg_band_fft)

        delta_bands.append(freq[0])
        theta_bands.append(freq[1])
        alpha_bands.append(freq[2])
        beta_bands.append(freq[3])
        gamma_bands.append(freq[4])

# Banda delta
df = pd.DataFrame(delta_bands)
delta_df = df.transpose()
delta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
delta_df

rows = list()
for _,row in delta_df.iterrows():
    rows += [row]*40
aux_delta = pd.DataFrame(rows).reset_index(drop=True)

# Banda theta
df = pd.DataFrame(theta_bands)
theta_df = df.transpose()
theta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
theta_df

rows = list()
for _,row in theta_df.iterrows():
    rows += [row]*40
aux_theta = pd.DataFrame(rows).reset_index(drop=True)

#Bnada alpha
df = pd.DataFrame(alpha_bands)
alpha_df = df.transpose()
alpha_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
alpha_df

rows = list()
for _,row in alpha_df.iterrows():
    rows += [row]*40
aux_alpha = pd.DataFrame(rows).reset_index(drop=True)

# Banda beta
df = pd.DataFrame(beta_bands)
beta_df = df.transpose()
beta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
beta_df

rows = list()
for _,row in beta_df.iterrows():
    rows += [row]*40
aux_beta = pd.DataFrame(rows).reset_index(drop=True)

# Banda gamma
df = pd.DataFrame(gamma_bands)
gamma_df = df.transpose()
gamma_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
gamma_df

rows = list()
for _,row in gamma_df.iterrows():
    rows += [row]*40
aux_gamma = pd.DataFrame(rows).reset_index(drop=True)

df_labels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])
rows = list()
for _,row in dfLabels.iterrows():
    rows += [row]*64
aux_labels = pd.DataFrame(rows).reset_index(drop=True)

data_delta = aux_delta.merge(aux_labels, left_index=True, right_index=True)
data_theta = aux_theta.merge(aux_labels, left_index=True, right_index=True)
data_alpha = aux_alpha.merge(aux_labels, left_index=True, right_index=True)
data_beta = aux_beta.merge(aux_labels, left_index=True, right_index=True)
data_gamma = aux_gamma.merge(aux_labels, left_index=True, right_index=True)

frames = [aux_delta, aux_theta, aux_alpha, aux_beta, aux_gamma]
aux_all_bands = pd.concat(frames, axis=1)
all_bands_df = aux_all_bands.merge(aux_labels, left_index=True, right_index=True)

data = all_bands_df.drop(['dominance', 'liking'], axis=1)

conditions = [
    ((data['valence'] <4.5) & (data['arousal']<4.5)),
    ((data['valence'] <4.5) & (data['arousal'] > 4.5)),
    ((data['valence'] >4.5) & (data['arousal']<4.5)),
    ((data['valence'] >4.5) & (data['arousal'] > 4.5))]
choices = ['sadness', 'pleasure', 'anger', 'joy']

data['valence'] = np.select(conditions, choices, default='black')
data = data.rename(columns={'valence' : 'label'})
data = data.drop(columns='arousal')

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['label']]

x = copia.drop(columns=['label'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment6_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)

rf_acc = accuracy_score(yTest, predict)
svm_acc = accuracy_score(yTest, svm_predict)
knn_acc = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_sadness = precision_score(yTest, predict, average="micro", labels=["sadness"])
rf_prec_pleasure = precision_score(yTest, predict, average="micro", labels=["pleasure"])
rf_prec_anger = precision_score(yTest, predict, average="micro", labels=["anger"])
rf_prec_joy = precision_score(yTest, predict, average="micro", labels=["joy"])


svm_prec_sadness = precision_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_prec_pleasure = precision_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_prec_anger = precision_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_prec_joy = precision_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_prec_sadness = precision_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_prec_pleasure = precision_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_prec_anger = precision_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_prec_joy = precision_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import recall_score

rf_rec_sadness = recall_score(yTest, predict, average="micro", labels=["sadness"])
rf_rec_pleasure = recall_score(yTest, predict, average="micro", labels=["pleasure"])
rf_rec_anger = recall_score(yTest, predict, average="micro", labels=["anger"])
rf_rec_joy = recall_score(yTest, predict, average="micro", labels=["joy"])


svm_rec_sadness = recall_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_rec_pleasure = recall_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_rec_anger = recall_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_rec_joy = recall_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_rec_sadness = recall_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_rec_pleasure = recall_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_rec_anger = recall_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_rec_joy = recall_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import f1_score

rf_f1_sadness = f1_score(yTest, predict, average="micro", labels=["sadness"])
rf_f1_pleasure = f1_score(yTest, predict, average="micro", labels=["pleasure"])
rf_f1_anger = f1_score(yTest, predict, average="micro", labels=["anger"])
rf_f1_joy = f1_score(yTest, predict, average="micro", labels=["joy"])


svm_f1_sadness = f1_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_f1_pleasure = f1_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_f1_anger = f1_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_f1_joy = f1_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_f1_sadness = f1_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_f1_pleasure = f1_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_f1_anger = f1_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_f1_joy = f1_score(yTest, knn_predict, average="micro", labels=["joy"])

data_CSV = [subject_id, 6, "STFT", "All bands", "-", "Random Forest", rf_acc, rf_prec_sadness, rf_prec_pleasure, rf_prec_anger, rf_prec_joy, rf_rec_sadness, rf_rec_pleasure, rf_rec_anger, rf_rec_joy, rf_f1_sadness, rf_f1_pleasure, rf_f1_anger, rf_f1_joy], [subject_id, 6, "STFT", "All bands", "-", "SVM", svm_acc, svm_prec_sadness, svm_prec_pleasure, svm_prec_anger, svm_prec_joy, svm_rec_sadness, svm_rec_pleasure, svm_rec_anger, svm_rec_joy, svm_f1_sadness, svm_f1_pleasure, svm_f1_anger, svm_f1_joy], [subject_id, 6, "STFT", "All bands", "-", "kNN", knn_acc, knn_prec_sadness, knn_prec_pleasure, knn_prec_anger, knn_prec_joy, knn_rec_sadness, knn_rec_pleasure, knn_rec_anger, knn_rec_joy, knn_f1_sadness, knn_f1_pleasure, knn_f1_anger, knn_f1_joy]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment6_1.csv", mode="a", header=False, index=False)

f = "data_preprocessed_python\\s03.dat"
subject_id = 3
dictRaw = pd.read_pickle(f)

labels = dictRaw.get('labels')

dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

data = dictRaw.get('data')

df_videos = {}
    #Recorrido de cada una de las columnas de la tabla 3d, y guardado en una una lista de dataframes para cada uno de los videos.
for i in range(40):
    df_videos[i] = pd.DataFrame(data[i])
    df_videos[i] = df_videos[i].transpose()
    df_videos[i].columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature']
    df_videos[i].drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature'],axis = 'columns', inplace=True)

for video in range(0, 40):
    delta_bands = []
    theta_bands = []
    beta_bands = []
    alpha_bands = []
    gamma_bands = []

    for channel in ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']:
        Zxx = do_stft(video, channel)
        eeg_band_fft = make_bands(Zxx)
     #   print(video)
      #  print("\n")
       # print(channel)
        freq = channel_freq(eeg_band_fft)

        delta_bands.append(freq[0])
        theta_bands.append(freq[1])
        alpha_bands.append(freq[2])
        beta_bands.append(freq[3])
        gamma_bands.append(freq[4])

# Banda delta
df = pd.DataFrame(delta_bands)
delta_df = df.transpose()
delta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
delta_df

rows = list()
for _,row in delta_df.iterrows():
    rows += [row]*40
aux_delta = pd.DataFrame(rows).reset_index(drop=True)

# Banda theta
df = pd.DataFrame(theta_bands)
theta_df = df.transpose()
theta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
theta_df

rows = list()
for _,row in theta_df.iterrows():
    rows += [row]*40
aux_theta = pd.DataFrame(rows).reset_index(drop=True)

#Bnada alpha
df = pd.DataFrame(alpha_bands)
alpha_df = df.transpose()
alpha_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
alpha_df

rows = list()
for _,row in alpha_df.iterrows():
    rows += [row]*40
aux_alpha = pd.DataFrame(rows).reset_index(drop=True)

# Banda beta
df = pd.DataFrame(beta_bands)
beta_df = df.transpose()
beta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
beta_df

rows = list()
for _,row in beta_df.iterrows():
    rows += [row]*40
aux_beta = pd.DataFrame(rows).reset_index(drop=True)

# Banda gamma
df = pd.DataFrame(gamma_bands)
gamma_df = df.transpose()
gamma_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
gamma_df

rows = list()
for _,row in gamma_df.iterrows():
    rows += [row]*40
aux_gamma = pd.DataFrame(rows).reset_index(drop=True)

df_labels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])
rows = list()
for _,row in dfLabels.iterrows():
    rows += [row]*64
aux_labels = pd.DataFrame(rows).reset_index(drop=True)

data_delta = aux_delta.merge(aux_labels, left_index=True, right_index=True)
data_theta = aux_theta.merge(aux_labels, left_index=True, right_index=True)
data_alpha = aux_alpha.merge(aux_labels, left_index=True, right_index=True)
data_beta = aux_beta.merge(aux_labels, left_index=True, right_index=True)
data_gamma = aux_gamma.merge(aux_labels, left_index=True, right_index=True)

frames = [aux_delta, aux_theta, aux_alpha, aux_beta, aux_gamma]
aux_all_bands = pd.concat(frames, axis=1)
all_bands_df = aux_all_bands.merge(aux_labels, left_index=True, right_index=True)

data = all_bands_df.drop(['dominance', 'liking'], axis=1)

conditions = [
    ((data['valence'] <4.5) & (data['arousal']<4.5)),
    ((data['valence'] <4.5) & (data['arousal'] > 4.5)),
    ((data['valence'] >4.5) & (data['arousal']<4.5)),
    ((data['valence'] >4.5) & (data['arousal'] > 4.5))]
choices = ['sadness', 'pleasure', 'anger', 'joy']

data['valence'] = np.select(conditions, choices, default='black')
data = data.rename(columns={'valence' : 'label'})
data = data.drop(columns='arousal')

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['label']]

x = copia.drop(columns=['label'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment6_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)
    
rf_acc = accuracy_score(yTest, predict)
svm_acc = accuracy_score(yTest, svm_predict)
knn_acc = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_sadness = precision_score(yTest, predict, average="micro", labels=["sadness"])
rf_prec_pleasure = precision_score(yTest, predict, average="micro", labels=["pleasure"])
rf_prec_anger = precision_score(yTest, predict, average="micro", labels=["anger"])
rf_prec_joy = precision_score(yTest, predict, average="micro", labels=["joy"])


svm_prec_sadness = precision_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_prec_pleasure = precision_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_prec_anger = precision_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_prec_joy = precision_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_prec_sadness = precision_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_prec_pleasure = precision_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_prec_anger = precision_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_prec_joy = precision_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import recall_score

rf_rec_sadness = recall_score(yTest, predict, average="micro", labels=["sadness"])
rf_rec_pleasure = recall_score(yTest, predict, average="micro", labels=["pleasure"])
rf_rec_anger = recall_score(yTest, predict, average="micro", labels=["anger"])
rf_rec_joy = recall_score(yTest, predict, average="micro", labels=["joy"])


svm_rec_sadness = recall_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_rec_pleasure = recall_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_rec_anger = recall_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_rec_joy = recall_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_rec_sadness = recall_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_rec_pleasure = recall_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_rec_anger = recall_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_rec_joy = recall_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import f1_score

rf_f1_sadness = f1_score(yTest, predict, average="micro", labels=["sadness"])
rf_f1_pleasure = f1_score(yTest, predict, average="micro", labels=["pleasure"])
rf_f1_anger = f1_score(yTest, predict, average="micro", labels=["anger"])
rf_f1_joy = f1_score(yTest, predict, average="micro", labels=["joy"])


svm_f1_sadness = f1_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_f1_pleasure = f1_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_f1_anger = f1_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_f1_joy = f1_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_f1_sadness = f1_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_f1_pleasure = f1_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_f1_anger = f1_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_f1_joy = f1_score(yTest, knn_predict, average="micro", labels=["joy"])

data_CSV = [subject_id, 6, "STFT", "All bands", "-", "Random Forest", rf_acc, rf_prec_sadness, rf_prec_pleasure, rf_prec_anger, rf_prec_joy, rf_rec_sadness, rf_rec_pleasure, rf_rec_anger, rf_rec_joy, rf_f1_sadness, rf_f1_pleasure, rf_f1_anger, rf_f1_joy], [subject_id, 6, "STFT", "All bands", "-", "SVM", svm_acc, svm_prec_sadness, svm_prec_pleasure, svm_prec_anger, svm_prec_joy, svm_rec_sadness, svm_rec_pleasure, svm_rec_anger, svm_rec_joy, svm_f1_sadness, svm_f1_pleasure, svm_f1_anger, svm_f1_joy], [subject_id, 6, "STFT", "All bands", "-", "kNN", knn_acc, knn_prec_sadness, knn_prec_pleasure, knn_prec_anger, knn_prec_joy, knn_rec_sadness, knn_rec_pleasure, knn_rec_anger, knn_rec_joy, knn_f1_sadness, knn_f1_pleasure, knn_f1_anger, knn_f1_joy]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment6_1.csv", mode="a", header=False, index=False)

f = "data_preprocessed_python\\s04.dat"
subject_id = 4
dictRaw = pd.read_pickle(f)

labels = dictRaw.get('labels')

dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

data = dictRaw.get('data')

df_videos = {}
    #Recorrido de cada una de las columnas de la tabla 3d, y guardado en una una lista de dataframes para cada uno de los videos.
for i in range(40):
    df_videos[i] = pd.DataFrame(data[i])
    df_videos[i] = df_videos[i].transpose()
    df_videos[i].columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature']
    df_videos[i].drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature'],axis = 'columns', inplace=True)

for video in range(0, 40):
    delta_bands = []
    theta_bands = []
    beta_bands = []
    alpha_bands = []
    gamma_bands = []

    for channel in ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']:
        Zxx = do_stft(video, channel)
        eeg_band_fft = make_bands(Zxx)
     #   print(video)
      #  print("\n")
       # print(channel)
        freq = channel_freq(eeg_band_fft)

        delta_bands.append(freq[0])
        theta_bands.append(freq[1])
        alpha_bands.append(freq[2])
        beta_bands.append(freq[3])
        gamma_bands.append(freq[4])

# Banda delta
df = pd.DataFrame(delta_bands)
delta_df = df.transpose()
delta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
delta_df

rows = list()
for _,row in delta_df.iterrows():
    rows += [row]*40
aux_delta = pd.DataFrame(rows).reset_index(drop=True)

# Banda theta
df = pd.DataFrame(theta_bands)
theta_df = df.transpose()
theta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
theta_df

rows = list()
for _,row in theta_df.iterrows():
    rows += [row]*40
aux_theta = pd.DataFrame(rows).reset_index(drop=True)

#Bnada alpha
df = pd.DataFrame(alpha_bands)
alpha_df = df.transpose()
alpha_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
alpha_df

rows = list()
for _,row in alpha_df.iterrows():
    rows += [row]*40
aux_alpha = pd.DataFrame(rows).reset_index(drop=True)

# Banda beta
df = pd.DataFrame(beta_bands)
beta_df = df.transpose()
beta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
beta_df

rows = list()
for _,row in beta_df.iterrows():
    rows += [row]*40
aux_beta = pd.DataFrame(rows).reset_index(drop=True)

# Banda gamma
df = pd.DataFrame(gamma_bands)
gamma_df = df.transpose()
gamma_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
gamma_df

rows = list()
for _,row in gamma_df.iterrows():
    rows += [row]*40
aux_gamma = pd.DataFrame(rows).reset_index(drop=True)

df_labels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])
rows = list()
for _,row in dfLabels.iterrows():
    rows += [row]*64
aux_labels = pd.DataFrame(rows).reset_index(drop=True)

data_delta = aux_delta.merge(aux_labels, left_index=True, right_index=True)
data_theta = aux_theta.merge(aux_labels, left_index=True, right_index=True)
data_alpha = aux_alpha.merge(aux_labels, left_index=True, right_index=True)
data_beta = aux_beta.merge(aux_labels, left_index=True, right_index=True)
data_gamma = aux_gamma.merge(aux_labels, left_index=True, right_index=True)

frames = [aux_delta, aux_theta, aux_alpha, aux_beta, aux_gamma]
aux_all_bands = pd.concat(frames, axis=1)
all_bands_df = aux_all_bands.merge(aux_labels, left_index=True, right_index=True)

data = all_bands_df.drop(['dominance', 'liking'], axis=1)

conditions = [
    ((data['valence'] <4.5) & (data['arousal']<4.5)),
    ((data['valence'] <4.5) & (data['arousal'] > 4.5)),
    ((data['valence'] >4.5) & (data['arousal']<4.5)),
    ((data['valence'] >4.5) & (data['arousal'] > 4.5))]
choices = ['sadness', 'pleasure', 'anger', 'joy']

data['valence'] = np.select(conditions, choices, default='black')
data = data.rename(columns={'valence' : 'label'})
data = data.drop(columns='arousal')

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['label']]

x = copia.drop(columns=['label'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment6_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)

rf_acc = accuracy_score(yTest, predict)
svm_acc = accuracy_score(yTest, svm_predict)
knn_acc = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_sadness = precision_score(yTest, predict, average="micro", labels=["sadness"])
rf_prec_pleasure = precision_score(yTest, predict, average="micro", labels=["pleasure"])
rf_prec_anger = precision_score(yTest, predict, average="micro", labels=["anger"])
rf_prec_joy = precision_score(yTest, predict, average="micro", labels=["joy"])


svm_prec_sadness = precision_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_prec_pleasure = precision_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_prec_anger = precision_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_prec_joy = precision_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_prec_sadness = precision_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_prec_pleasure = precision_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_prec_anger = precision_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_prec_joy = precision_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import recall_score

rf_rec_sadness = recall_score(yTest, predict, average="micro", labels=["sadness"])
rf_rec_pleasure = recall_score(yTest, predict, average="micro", labels=["pleasure"])
rf_rec_anger = recall_score(yTest, predict, average="micro", labels=["anger"])
rf_rec_joy = recall_score(yTest, predict, average="micro", labels=["joy"])


svm_rec_sadness = recall_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_rec_pleasure = recall_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_rec_anger = recall_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_rec_joy = recall_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_rec_sadness = recall_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_rec_pleasure = recall_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_rec_anger = recall_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_rec_joy = recall_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import f1_score

rf_f1_sadness = f1_score(yTest, predict, average="micro", labels=["sadness"])
rf_f1_pleasure = f1_score(yTest, predict, average="micro", labels=["pleasure"])
rf_f1_anger = f1_score(yTest, predict, average="micro", labels=["anger"])
rf_f1_joy = f1_score(yTest, predict, average="micro", labels=["joy"])


svm_f1_sadness = f1_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_f1_pleasure = f1_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_f1_anger = f1_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_f1_joy = f1_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_f1_sadness = f1_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_f1_pleasure = f1_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_f1_anger = f1_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_f1_joy = f1_score(yTest, knn_predict, average="micro", labels=["joy"])

data_CSV = [subject_id, 6, "STFT", "All bands", "-", "Random Forest", rf_acc, rf_prec_sadness, rf_prec_pleasure, rf_prec_anger, rf_prec_joy, rf_rec_sadness, rf_rec_pleasure, rf_rec_anger, rf_rec_joy, rf_f1_sadness, rf_f1_pleasure, rf_f1_anger, rf_f1_joy], [subject_id, 6, "STFT", "All bands", "-", "SVM", svm_acc, svm_prec_sadness, svm_prec_pleasure, svm_prec_anger, svm_prec_joy, svm_rec_sadness, svm_rec_pleasure, svm_rec_anger, svm_rec_joy, svm_f1_sadness, svm_f1_pleasure, svm_f1_anger, svm_f1_joy], [subject_id, 6, "STFT", "All bands", "-", "kNN", knn_acc, knn_prec_sadness, knn_prec_pleasure, knn_prec_anger, knn_prec_joy, knn_rec_sadness, knn_rec_pleasure, knn_rec_anger, knn_rec_joy, knn_f1_sadness, knn_f1_pleasure, knn_f1_anger, knn_f1_joy]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment6_1.csv", mode="a", header=False, index=False)

f = "data_preprocessed_python\\s05.dat"
subject_id = 5
dictRaw = pd.read_pickle(f)

labels = dictRaw.get('labels')

dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

data = dictRaw.get('data')

df_videos = {}
    #Recorrido de cada una de las columnas de la tabla 3d, y guardado en una una lista de dataframes para cada uno de los videos.
for i in range(40):
    df_videos[i] = pd.DataFrame(data[i])
    df_videos[i] = df_videos[i].transpose()
    df_videos[i].columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature']
    df_videos[i].drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature'],axis = 'columns', inplace=True)

for video in range(0, 40):
    delta_bands = []
    theta_bands = []
    beta_bands = []
    alpha_bands = []
    gamma_bands = []

    for channel in ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']:
        Zxx = do_stft(video, channel)
        eeg_band_fft = make_bands(Zxx)
     #   print(video)
      #  print("\n")
       # print(channel)
        freq = channel_freq(eeg_band_fft)

        delta_bands.append(freq[0])
        theta_bands.append(freq[1])
        alpha_bands.append(freq[2])
        beta_bands.append(freq[3])
        gamma_bands.append(freq[4])

# Banda delta
df = pd.DataFrame(delta_bands)
delta_df = df.transpose()
delta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
delta_df

rows = list()
for _,row in delta_df.iterrows():
    rows += [row]*40
aux_delta = pd.DataFrame(rows).reset_index(drop=True)

# Banda theta
df = pd.DataFrame(theta_bands)
theta_df = df.transpose()
theta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
theta_df

rows = list()
for _,row in theta_df.iterrows():
    rows += [row]*40
aux_theta = pd.DataFrame(rows).reset_index(drop=True)

#Bnada alpha
df = pd.DataFrame(alpha_bands)
alpha_df = df.transpose()
alpha_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
alpha_df

rows = list()
for _,row in alpha_df.iterrows():
    rows += [row]*40
aux_alpha = pd.DataFrame(rows).reset_index(drop=True)

# Banda beta
df = pd.DataFrame(beta_bands)
beta_df = df.transpose()
beta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
beta_df

rows = list()
for _,row in beta_df.iterrows():
    rows += [row]*40
aux_beta = pd.DataFrame(rows).reset_index(drop=True)

# Banda gamma
df = pd.DataFrame(gamma_bands)
gamma_df = df.transpose()
gamma_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
gamma_df

rows = list()
for _,row in gamma_df.iterrows():
    rows += [row]*40
aux_gamma = pd.DataFrame(rows).reset_index(drop=True)

df_labels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])
rows = list()
for _,row in dfLabels.iterrows():
    rows += [row]*64
aux_labels = pd.DataFrame(rows).reset_index(drop=True)

data_delta = aux_delta.merge(aux_labels, left_index=True, right_index=True)
data_theta = aux_theta.merge(aux_labels, left_index=True, right_index=True)
data_alpha = aux_alpha.merge(aux_labels, left_index=True, right_index=True)
data_beta = aux_beta.merge(aux_labels, left_index=True, right_index=True)
data_gamma = aux_gamma.merge(aux_labels, left_index=True, right_index=True)

frames = [aux_delta, aux_theta, aux_alpha, aux_beta, aux_gamma]
aux_all_bands = pd.concat(frames, axis=1)
all_bands_df = aux_all_bands.merge(aux_labels, left_index=True, right_index=True)

data = all_bands_df.drop(['dominance', 'liking'], axis=1)

conditions = [
    ((data['valence'] <4.5) & (data['arousal']<4.5)),
    ((data['valence'] <4.5) & (data['arousal'] > 4.5)),
    ((data['valence'] >4.5) & (data['arousal']<4.5)),
    ((data['valence'] >4.5) & (data['arousal'] > 4.5))]
choices = ['sadness', 'pleasure', 'anger', 'joy']

data['valence'] = np.select(conditions, choices, default='black')
data = data.rename(columns={'valence' : 'label'})
data = data.drop(columns='arousal')

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['label']]

x = copia.drop(columns=['label'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment6_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)
    
rf_acc = accuracy_score(yTest, predict)
svm_acc = accuracy_score(yTest, svm_predict)
knn_acc = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_sadness = precision_score(yTest, predict, average="micro", labels=["sadness"])
rf_prec_pleasure = precision_score(yTest, predict, average="micro", labels=["pleasure"])
rf_prec_anger = precision_score(yTest, predict, average="micro", labels=["anger"])
rf_prec_joy = precision_score(yTest, predict, average="micro", labels=["joy"])


svm_prec_sadness = precision_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_prec_pleasure = precision_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_prec_anger = precision_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_prec_joy = precision_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_prec_sadness = precision_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_prec_pleasure = precision_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_prec_anger = precision_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_prec_joy = precision_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import recall_score

rf_rec_sadness = recall_score(yTest, predict, average="micro", labels=["sadness"])
rf_rec_pleasure = recall_score(yTest, predict, average="micro", labels=["pleasure"])
rf_rec_anger = recall_score(yTest, predict, average="micro", labels=["anger"])
rf_rec_joy = recall_score(yTest, predict, average="micro", labels=["joy"])


svm_rec_sadness = recall_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_rec_pleasure = recall_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_rec_anger = recall_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_rec_joy = recall_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_rec_sadness = recall_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_rec_pleasure = recall_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_rec_anger = recall_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_rec_joy = recall_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import f1_score

rf_f1_sadness = f1_score(yTest, predict, average="micro", labels=["sadness"])
rf_f1_pleasure = f1_score(yTest, predict, average="micro", labels=["pleasure"])
rf_f1_anger = f1_score(yTest, predict, average="micro", labels=["anger"])
rf_f1_joy = f1_score(yTest, predict, average="micro", labels=["joy"])


svm_f1_sadness = f1_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_f1_pleasure = f1_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_f1_anger = f1_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_f1_joy = f1_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_f1_sadness = f1_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_f1_pleasure = f1_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_f1_anger = f1_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_f1_joy = f1_score(yTest, knn_predict, average="micro", labels=["joy"])

data_CSV = [subject_id, 6, "STFT", "All bands", "-", "Random Forest", rf_acc, rf_prec_sadness, rf_prec_pleasure, rf_prec_anger, rf_prec_joy, rf_rec_sadness, rf_rec_pleasure, rf_rec_anger, rf_rec_joy, rf_f1_sadness, rf_f1_pleasure, rf_f1_anger, rf_f1_joy], [subject_id, 6, "STFT", "All bands", "-", "SVM", svm_acc, svm_prec_sadness, svm_prec_pleasure, svm_prec_anger, svm_prec_joy, svm_rec_sadness, svm_rec_pleasure, svm_rec_anger, svm_rec_joy, svm_f1_sadness, svm_f1_pleasure, svm_f1_anger, svm_f1_joy], [subject_id, 6, "STFT", "All bands", "-", "kNN", knn_acc, knn_prec_sadness, knn_prec_pleasure, knn_prec_anger, knn_prec_joy, knn_rec_sadness, knn_rec_pleasure, knn_rec_anger, knn_rec_joy, knn_f1_sadness, knn_f1_pleasure, knn_f1_anger, knn_f1_joy]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment6_1.csv", mode="a", header=False, index=False)

f = "data_preprocessed_python\\s06.dat"
subject_id = 6
dictRaw = pd.read_pickle(f)

labels = dictRaw.get('labels')

dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

data = dictRaw.get('data')

df_videos = {}
    #Recorrido de cada una de las columnas de la tabla 3d, y guardado en una una lista de dataframes para cada uno de los videos.
for i in range(40):
    df_videos[i] = pd.DataFrame(data[i])
    df_videos[i] = df_videos[i].transpose()
    df_videos[i].columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature']
    df_videos[i].drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature'],axis = 'columns', inplace=True)

for video in range(0, 40):
    delta_bands = []
    theta_bands = []
    beta_bands = []
    alpha_bands = []
    gamma_bands = []

    for channel in ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']:
        Zxx = do_stft(video, channel)
        eeg_band_fft = make_bands(Zxx)
     #   print(video)
      #  print("\n")
       # print(channel)
        freq = channel_freq(eeg_band_fft)

        delta_bands.append(freq[0])
        theta_bands.append(freq[1])
        alpha_bands.append(freq[2])
        beta_bands.append(freq[3])
        gamma_bands.append(freq[4])

# Banda delta
df = pd.DataFrame(delta_bands)
delta_df = df.transpose()
delta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
delta_df

rows = list()
for _,row in delta_df.iterrows():
    rows += [row]*40
aux_delta = pd.DataFrame(rows).reset_index(drop=True)

# Banda theta
df = pd.DataFrame(theta_bands)
theta_df = df.transpose()
theta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
theta_df

rows = list()
for _,row in theta_df.iterrows():
    rows += [row]*40
aux_theta = pd.DataFrame(rows).reset_index(drop=True)

#Bnada alpha
df = pd.DataFrame(alpha_bands)
alpha_df = df.transpose()
alpha_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
alpha_df

rows = list()
for _,row in alpha_df.iterrows():
    rows += [row]*40
aux_alpha = pd.DataFrame(rows).reset_index(drop=True)

# Banda beta
df = pd.DataFrame(beta_bands)
beta_df = df.transpose()
beta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
beta_df

rows = list()
for _,row in beta_df.iterrows():
    rows += [row]*40
aux_beta = pd.DataFrame(rows).reset_index(drop=True)

# Banda gamma
df = pd.DataFrame(gamma_bands)
gamma_df = df.transpose()
gamma_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
gamma_df

rows = list()
for _,row in gamma_df.iterrows():
    rows += [row]*40
aux_gamma = pd.DataFrame(rows).reset_index(drop=True)

df_labels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])
rows = list()
for _,row in dfLabels.iterrows():
    rows += [row]*64
aux_labels = pd.DataFrame(rows).reset_index(drop=True)

data_delta = aux_delta.merge(aux_labels, left_index=True, right_index=True)
data_theta = aux_theta.merge(aux_labels, left_index=True, right_index=True)
data_alpha = aux_alpha.merge(aux_labels, left_index=True, right_index=True)
data_beta = aux_beta.merge(aux_labels, left_index=True, right_index=True)
data_gamma = aux_gamma.merge(aux_labels, left_index=True, right_index=True)

frames = [aux_delta, aux_theta, aux_alpha, aux_beta, aux_gamma]
aux_all_bands = pd.concat(frames, axis=1)
all_bands_df = aux_all_bands.merge(aux_labels, left_index=True, right_index=True)

data = all_bands_df.drop(['dominance', 'liking'], axis=1)

conditions = [
    ((data['valence'] <4.5) & (data['arousal']<4.5)),
    ((data['valence'] <4.5) & (data['arousal'] > 4.5)),
    ((data['valence'] >4.5) & (data['arousal']<4.5)),
    ((data['valence'] >4.5) & (data['arousal'] > 4.5))]
choices = ['sadness', 'pleasure', 'anger', 'joy']

data['valence'] = np.select(conditions, choices, default='black')
data = data.rename(columns={'valence' : 'label'})
data = data.drop(columns='arousal')

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['label']]

x = copia.drop(columns=['label'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment6_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)
    
rf_acc = accuracy_score(yTest, predict)
svm_acc = accuracy_score(yTest, svm_predict)
knn_acc = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_sadness = precision_score(yTest, predict, average="micro", labels=["sadness"])
rf_prec_pleasure = precision_score(yTest, predict, average="micro", labels=["pleasure"])
rf_prec_anger = precision_score(yTest, predict, average="micro", labels=["anger"])
rf_prec_joy = precision_score(yTest, predict, average="micro", labels=["joy"])


svm_prec_sadness = precision_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_prec_pleasure = precision_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_prec_anger = precision_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_prec_joy = precision_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_prec_sadness = precision_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_prec_pleasure = precision_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_prec_anger = precision_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_prec_joy = precision_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import recall_score

rf_rec_sadness = recall_score(yTest, predict, average="micro", labels=["sadness"])
rf_rec_pleasure = recall_score(yTest, predict, average="micro", labels=["pleasure"])
rf_rec_anger = recall_score(yTest, predict, average="micro", labels=["anger"])
rf_rec_joy = recall_score(yTest, predict, average="micro", labels=["joy"])


svm_rec_sadness = recall_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_rec_pleasure = recall_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_rec_anger = recall_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_rec_joy = recall_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_rec_sadness = recall_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_rec_pleasure = recall_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_rec_anger = recall_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_rec_joy = recall_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import f1_score

rf_f1_sadness = f1_score(yTest, predict, average="micro", labels=["sadness"])
rf_f1_pleasure = f1_score(yTest, predict, average="micro", labels=["pleasure"])
rf_f1_anger = f1_score(yTest, predict, average="micro", labels=["anger"])
rf_f1_joy = f1_score(yTest, predict, average="micro", labels=["joy"])


svm_f1_sadness = f1_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_f1_pleasure = f1_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_f1_anger = f1_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_f1_joy = f1_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_f1_sadness = f1_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_f1_pleasure = f1_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_f1_anger = f1_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_f1_joy = f1_score(yTest, knn_predict, average="micro", labels=["joy"])

data_CSV = [subject_id, 6, "STFT", "All bands", "-", "Random Forest", rf_acc, rf_prec_sadness, rf_prec_pleasure, rf_prec_anger, rf_prec_joy, rf_rec_sadness, rf_rec_pleasure, rf_rec_anger, rf_rec_joy, rf_f1_sadness, rf_f1_pleasure, rf_f1_anger, rf_f1_joy], [subject_id, 6, "STFT", "All bands", "-", "SVM", svm_acc, svm_prec_sadness, svm_prec_pleasure, svm_prec_anger, svm_prec_joy, svm_rec_sadness, svm_rec_pleasure, svm_rec_anger, svm_rec_joy, svm_f1_sadness, svm_f1_pleasure, svm_f1_anger, svm_f1_joy], [subject_id, 6, "STFT", "All bands", "-", "kNN", knn_acc, knn_prec_sadness, knn_prec_pleasure, knn_prec_anger, knn_prec_joy, knn_rec_sadness, knn_rec_pleasure, knn_rec_anger, knn_rec_joy, knn_f1_sadness, knn_f1_pleasure, knn_f1_anger, knn_f1_joy]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment6_1.csv", mode="a", header=False, index=False)

f = "data_preprocessed_python\\s07.dat"
subject_id = 7
dictRaw = pd.read_pickle(f)

labels = dictRaw.get('labels')

dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

data = dictRaw.get('data')

df_videos = {}
    #Recorrido de cada una de las columnas de la tabla 3d, y guardado en una una lista de dataframes para cada uno de los videos.
for i in range(40):
    df_videos[i] = pd.DataFrame(data[i])
    df_videos[i] = df_videos[i].transpose()
    df_videos[i].columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature']
    df_videos[i].drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature'],axis = 'columns', inplace=True)

for video in range(0, 40):
    delta_bands = []
    theta_bands = []
    beta_bands = []
    alpha_bands = []
    gamma_bands = []

    for channel in ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']:
        Zxx = do_stft(video, channel)
        eeg_band_fft = make_bands(Zxx)
     #   print(video)
      #  print("\n")
       # print(channel)
        freq = channel_freq(eeg_band_fft)

        delta_bands.append(freq[0])
        theta_bands.append(freq[1])
        alpha_bands.append(freq[2])
        beta_bands.append(freq[3])
        gamma_bands.append(freq[4])

# Banda delta
df = pd.DataFrame(delta_bands)
delta_df = df.transpose()
delta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
delta_df

rows = list()
for _,row in delta_df.iterrows():
    rows += [row]*40
aux_delta = pd.DataFrame(rows).reset_index(drop=True)

# Banda theta
df = pd.DataFrame(theta_bands)
theta_df = df.transpose()
theta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
theta_df

rows = list()
for _,row in theta_df.iterrows():
    rows += [row]*40
aux_theta = pd.DataFrame(rows).reset_index(drop=True)

#Bnada alpha
df = pd.DataFrame(alpha_bands)
alpha_df = df.transpose()
alpha_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
alpha_df

rows = list()
for _,row in alpha_df.iterrows():
    rows += [row]*40
aux_alpha = pd.DataFrame(rows).reset_index(drop=True)

# Banda beta
df = pd.DataFrame(beta_bands)
beta_df = df.transpose()
beta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
beta_df

rows = list()
for _,row in beta_df.iterrows():
    rows += [row]*40
aux_beta = pd.DataFrame(rows).reset_index(drop=True)

# Banda gamma
df = pd.DataFrame(gamma_bands)
gamma_df = df.transpose()
gamma_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
gamma_df

rows = list()
for _,row in gamma_df.iterrows():
    rows += [row]*40
aux_gamma = pd.DataFrame(rows).reset_index(drop=True)

df_labels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])
rows = list()
for _,row in dfLabels.iterrows():
    rows += [row]*64
aux_labels = pd.DataFrame(rows).reset_index(drop=True)

data_delta = aux_delta.merge(aux_labels, left_index=True, right_index=True)
data_theta = aux_theta.merge(aux_labels, left_index=True, right_index=True)
data_alpha = aux_alpha.merge(aux_labels, left_index=True, right_index=True)
data_beta = aux_beta.merge(aux_labels, left_index=True, right_index=True)
data_gamma = aux_gamma.merge(aux_labels, left_index=True, right_index=True)

frames = [aux_delta, aux_theta, aux_alpha, aux_beta, aux_gamma]
aux_all_bands = pd.concat(frames, axis=1)
all_bands_df = aux_all_bands.merge(aux_labels, left_index=True, right_index=True)

data = all_bands_df.drop(['dominance', 'liking'], axis=1)

conditions = [
    ((data['valence'] <4.5) & (data['arousal']<4.5)),
    ((data['valence'] <4.5) & (data['arousal'] > 4.5)),
    ((data['valence'] >4.5) & (data['arousal']<4.5)),
    ((data['valence'] >4.5) & (data['arousal'] > 4.5))]
choices = ['sadness', 'pleasure', 'anger', 'joy']

data['valence'] = np.select(conditions, choices, default='black')
data = data.rename(columns={'valence' : 'label'})
data = data.drop(columns='arousal')

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['label']]

x = copia.drop(columns=['label'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment6_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)
    
rf_acc = accuracy_score(yTest, predict)
svm_acc = accuracy_score(yTest, svm_predict)
knn_acc = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_sadness = precision_score(yTest, predict, average="micro", labels=["sadness"])
rf_prec_pleasure = precision_score(yTest, predict, average="micro", labels=["pleasure"])
rf_prec_anger = precision_score(yTest, predict, average="micro", labels=["anger"])
rf_prec_joy = precision_score(yTest, predict, average="micro", labels=["joy"])


svm_prec_sadness = precision_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_prec_pleasure = precision_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_prec_anger = precision_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_prec_joy = precision_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_prec_sadness = precision_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_prec_pleasure = precision_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_prec_anger = precision_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_prec_joy = precision_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import recall_score

rf_rec_sadness = recall_score(yTest, predict, average="micro", labels=["sadness"])
rf_rec_pleasure = recall_score(yTest, predict, average="micro", labels=["pleasure"])
rf_rec_anger = recall_score(yTest, predict, average="micro", labels=["anger"])
rf_rec_joy = recall_score(yTest, predict, average="micro", labels=["joy"])


svm_rec_sadness = recall_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_rec_pleasure = recall_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_rec_anger = recall_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_rec_joy = recall_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_rec_sadness = recall_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_rec_pleasure = recall_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_rec_anger = recall_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_rec_joy = recall_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import f1_score

rf_f1_sadness = f1_score(yTest, predict, average="micro", labels=["sadness"])
rf_f1_pleasure = f1_score(yTest, predict, average="micro", labels=["pleasure"])
rf_f1_anger = f1_score(yTest, predict, average="micro", labels=["anger"])
rf_f1_joy = f1_score(yTest, predict, average="micro", labels=["joy"])


svm_f1_sadness = f1_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_f1_pleasure = f1_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_f1_anger = f1_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_f1_joy = f1_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_f1_sadness = f1_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_f1_pleasure = f1_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_f1_anger = f1_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_f1_joy = f1_score(yTest, knn_predict, average="micro", labels=["joy"])

data_CSV = [subject_id, 6, "STFT", "All bands", "-", "Random Forest", rf_acc, rf_prec_sadness, rf_prec_pleasure, rf_prec_anger, rf_prec_joy, rf_rec_sadness, rf_rec_pleasure, rf_rec_anger, rf_rec_joy, rf_f1_sadness, rf_f1_pleasure, rf_f1_anger, rf_f1_joy], [subject_id, 6, "STFT", "All bands", "-", "SVM", svm_acc, svm_prec_sadness, svm_prec_pleasure, svm_prec_anger, svm_prec_joy, svm_rec_sadness, svm_rec_pleasure, svm_rec_anger, svm_rec_joy, svm_f1_sadness, svm_f1_pleasure, svm_f1_anger, svm_f1_joy], [subject_id, 6, "STFT", "All bands", "-", "kNN", knn_acc, knn_prec_sadness, knn_prec_pleasure, knn_prec_anger, knn_prec_joy, knn_rec_sadness, knn_rec_pleasure, knn_rec_anger, knn_rec_joy, knn_f1_sadness, knn_f1_pleasure, knn_f1_anger, knn_f1_joy]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment6_1.csv", mode="a", header=False, index=False)

f = "data_preprocessed_python\\s08.dat"
subject_id = 8
dictRaw = pd.read_pickle(f)

labels = dictRaw.get('labels')

dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

data = dictRaw.get('data')

df_videos = {}
    #Recorrido de cada una de las columnas de la tabla 3d, y guardado en una una lista de dataframes para cada uno de los videos.
for i in range(40):
    df_videos[i] = pd.DataFrame(data[i])
    df_videos[i] = df_videos[i].transpose()
    df_videos[i].columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature']
    df_videos[i].drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature'],axis = 'columns', inplace=True)

for video in range(0, 40):
    delta_bands = []
    theta_bands = []
    beta_bands = []
    alpha_bands = []
    gamma_bands = []

    for channel in ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']:
        Zxx = do_stft(video, channel)
        eeg_band_fft = make_bands(Zxx)
     #   print(video)
      #  print("\n")
       # print(channel)
        freq = channel_freq(eeg_band_fft)

        delta_bands.append(freq[0])
        theta_bands.append(freq[1])
        alpha_bands.append(freq[2])
        beta_bands.append(freq[3])
        gamma_bands.append(freq[4])

# Banda delta
df = pd.DataFrame(delta_bands)
delta_df = df.transpose()
delta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
delta_df

rows = list()
for _,row in delta_df.iterrows():
    rows += [row]*40
aux_delta = pd.DataFrame(rows).reset_index(drop=True)

# Banda theta
df = pd.DataFrame(theta_bands)
theta_df = df.transpose()
theta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
theta_df

rows = list()
for _,row in theta_df.iterrows():
    rows += [row]*40
aux_theta = pd.DataFrame(rows).reset_index(drop=True)

#Bnada alpha
df = pd.DataFrame(alpha_bands)
alpha_df = df.transpose()
alpha_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
alpha_df

rows = list()
for _,row in alpha_df.iterrows():
    rows += [row]*40
aux_alpha = pd.DataFrame(rows).reset_index(drop=True)

# Banda beta
df = pd.DataFrame(beta_bands)
beta_df = df.transpose()
beta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
beta_df

rows = list()
for _,row in beta_df.iterrows():
    rows += [row]*40
aux_beta = pd.DataFrame(rows).reset_index(drop=True)

# Banda gamma
df = pd.DataFrame(gamma_bands)
gamma_df = df.transpose()
gamma_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
gamma_df

rows = list()
for _,row in gamma_df.iterrows():
    rows += [row]*40
aux_gamma = pd.DataFrame(rows).reset_index(drop=True)

df_labels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])
rows = list()
for _,row in dfLabels.iterrows():
    rows += [row]*64
aux_labels = pd.DataFrame(rows).reset_index(drop=True)

data_delta = aux_delta.merge(aux_labels, left_index=True, right_index=True)
data_theta = aux_theta.merge(aux_labels, left_index=True, right_index=True)
data_alpha = aux_alpha.merge(aux_labels, left_index=True, right_index=True)
data_beta = aux_beta.merge(aux_labels, left_index=True, right_index=True)
data_gamma = aux_gamma.merge(aux_labels, left_index=True, right_index=True)

frames = [aux_delta, aux_theta, aux_alpha, aux_beta, aux_gamma]
aux_all_bands = pd.concat(frames, axis=1)
all_bands_df = aux_all_bands.merge(aux_labels, left_index=True, right_index=True)

data = all_bands_df.drop(['dominance', 'liking'], axis=1)

conditions = [
    ((data['valence'] <4.5) & (data['arousal']<4.5)),
    ((data['valence'] <4.5) & (data['arousal'] > 4.5)),
    ((data['valence'] >4.5) & (data['arousal']<4.5)),
    ((data['valence'] >4.5) & (data['arousal'] > 4.5))]
choices = ['sadness', 'pleasure', 'anger', 'joy']

data['valence'] = np.select(conditions, choices, default='black')
data = data.rename(columns={'valence' : 'label'})
data = data.drop(columns='arousal')

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['label']]

x = copia.drop(columns=['label'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment6_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)
    
rf_acc = accuracy_score(yTest, predict)
svm_acc = accuracy_score(yTest, svm_predict)
knn_acc = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_sadness = precision_score(yTest, predict, average="micro", labels=["sadness"])
rf_prec_pleasure = precision_score(yTest, predict, average="micro", labels=["pleasure"])
rf_prec_anger = precision_score(yTest, predict, average="micro", labels=["anger"])
rf_prec_joy = precision_score(yTest, predict, average="micro", labels=["joy"])


svm_prec_sadness = precision_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_prec_pleasure = precision_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_prec_anger = precision_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_prec_joy = precision_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_prec_sadness = precision_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_prec_pleasure = precision_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_prec_anger = precision_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_prec_joy = precision_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import recall_score

rf_rec_sadness = recall_score(yTest, predict, average="micro", labels=["sadness"])
rf_rec_pleasure = recall_score(yTest, predict, average="micro", labels=["pleasure"])
rf_rec_anger = recall_score(yTest, predict, average="micro", labels=["anger"])
rf_rec_joy = recall_score(yTest, predict, average="micro", labels=["joy"])


svm_rec_sadness = recall_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_rec_pleasure = recall_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_rec_anger = recall_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_rec_joy = recall_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_rec_sadness = recall_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_rec_pleasure = recall_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_rec_anger = recall_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_rec_joy = recall_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import f1_score

rf_f1_sadness = f1_score(yTest, predict, average="micro", labels=["sadness"])
rf_f1_pleasure = f1_score(yTest, predict, average="micro", labels=["pleasure"])
rf_f1_anger = f1_score(yTest, predict, average="micro", labels=["anger"])
rf_f1_joy = f1_score(yTest, predict, average="micro", labels=["joy"])


svm_f1_sadness = f1_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_f1_pleasure = f1_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_f1_anger = f1_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_f1_joy = f1_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_f1_sadness = f1_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_f1_pleasure = f1_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_f1_anger = f1_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_f1_joy = f1_score(yTest, knn_predict, average="micro", labels=["joy"])

data_CSV = [subject_id, 6, "STFT", "All bands", "-", "Random Forest", rf_acc, rf_prec_sadness, rf_prec_pleasure, rf_prec_anger, rf_prec_joy, rf_rec_sadness, rf_rec_pleasure, rf_rec_anger, rf_rec_joy, rf_f1_sadness, rf_f1_pleasure, rf_f1_anger, rf_f1_joy], [subject_id, 6, "STFT", "All bands", "-", "SVM", svm_acc, svm_prec_sadness, svm_prec_pleasure, svm_prec_anger, svm_prec_joy, svm_rec_sadness, svm_rec_pleasure, svm_rec_anger, svm_rec_joy, svm_f1_sadness, svm_f1_pleasure, svm_f1_anger, svm_f1_joy], [subject_id, 6, "STFT", "All bands", "-", "kNN", knn_acc, knn_prec_sadness, knn_prec_pleasure, knn_prec_anger, knn_prec_joy, knn_rec_sadness, knn_rec_pleasure, knn_rec_anger, knn_rec_joy, knn_f1_sadness, knn_f1_pleasure, knn_f1_anger, knn_f1_joy]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment6_1.csv", mode="a", header=False, index=False)

f = "data_preprocessed_python\\s09.dat"
subject_id = 9
dictRaw = pd.read_pickle(f)

labels = dictRaw.get('labels')

dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

data = dictRaw.get('data')

df_videos = {}
    #Recorrido de cada una de las columnas de la tabla 3d, y guardado en una una lista de dataframes para cada uno de los videos.
for i in range(40):
    df_videos[i] = pd.DataFrame(data[i])
    df_videos[i] = df_videos[i].transpose()
    df_videos[i].columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature']
    df_videos[i].drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature'],axis = 'columns', inplace=True)

for video in range(0, 40):
    delta_bands = []
    theta_bands = []
    beta_bands = []
    alpha_bands = []
    gamma_bands = []

    for channel in ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']:
        Zxx = do_stft(video, channel)
        eeg_band_fft = make_bands(Zxx)
     #   print(video)
      #  print("\n")
       # print(channel)
        freq = channel_freq(eeg_band_fft)

        delta_bands.append(freq[0])
        theta_bands.append(freq[1])
        alpha_bands.append(freq[2])
        beta_bands.append(freq[3])
        gamma_bands.append(freq[4])

# Banda delta
df = pd.DataFrame(delta_bands)
delta_df = df.transpose()
delta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
delta_df

rows = list()
for _,row in delta_df.iterrows():
    rows += [row]*40
aux_delta = pd.DataFrame(rows).reset_index(drop=True)

# Banda theta
df = pd.DataFrame(theta_bands)
theta_df = df.transpose()
theta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
theta_df

rows = list()
for _,row in theta_df.iterrows():
    rows += [row]*40
aux_theta = pd.DataFrame(rows).reset_index(drop=True)

#Bnada alpha
df = pd.DataFrame(alpha_bands)
alpha_df = df.transpose()
alpha_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
alpha_df

rows = list()
for _,row in alpha_df.iterrows():
    rows += [row]*40
aux_alpha = pd.DataFrame(rows).reset_index(drop=True)

# Banda beta
df = pd.DataFrame(beta_bands)
beta_df = df.transpose()
beta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
beta_df

rows = list()
for _,row in beta_df.iterrows():
    rows += [row]*40
aux_beta = pd.DataFrame(rows).reset_index(drop=True)

# Banda gamma
df = pd.DataFrame(gamma_bands)
gamma_df = df.transpose()
gamma_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
gamma_df

rows = list()
for _,row in gamma_df.iterrows():
    rows += [row]*40
aux_gamma = pd.DataFrame(rows).reset_index(drop=True)

df_labels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])
rows = list()
for _,row in dfLabels.iterrows():
    rows += [row]*64
aux_labels = pd.DataFrame(rows).reset_index(drop=True)

data_delta = aux_delta.merge(aux_labels, left_index=True, right_index=True)
data_theta = aux_theta.merge(aux_labels, left_index=True, right_index=True)
data_alpha = aux_alpha.merge(aux_labels, left_index=True, right_index=True)
data_beta = aux_beta.merge(aux_labels, left_index=True, right_index=True)
data_gamma = aux_gamma.merge(aux_labels, left_index=True, right_index=True)

frames = [aux_delta, aux_theta, aux_alpha, aux_beta, aux_gamma]
aux_all_bands = pd.concat(frames, axis=1)
all_bands_df = aux_all_bands.merge(aux_labels, left_index=True, right_index=True)

data = all_bands_df.drop(['dominance', 'liking'], axis=1)

conditions = [
    ((data['valence'] <4.5) & (data['arousal']<4.5)),
    ((data['valence'] <4.5) & (data['arousal'] > 4.5)),
    ((data['valence'] >4.5) & (data['arousal']<4.5)),
    ((data['valence'] >4.5) & (data['arousal'] > 4.5))]
choices = ['sadness', 'pleasure', 'anger', 'joy']

data['valence'] = np.select(conditions, choices, default='black')
data = data.rename(columns={'valence' : 'label'})
data = data.drop(columns='arousal')

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['label']]

x = copia.drop(columns=['label'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment6_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)
    
rf_acc = accuracy_score(yTest, predict)
svm_acc = accuracy_score(yTest, svm_predict)
knn_acc = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_sadness = precision_score(yTest, predict, average="micro", labels=["sadness"])
rf_prec_pleasure = precision_score(yTest, predict, average="micro", labels=["pleasure"])
rf_prec_anger = precision_score(yTest, predict, average="micro", labels=["anger"])
rf_prec_joy = precision_score(yTest, predict, average="micro", labels=["joy"])


svm_prec_sadness = precision_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_prec_pleasure = precision_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_prec_anger = precision_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_prec_joy = precision_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_prec_sadness = precision_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_prec_pleasure = precision_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_prec_anger = precision_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_prec_joy = precision_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import recall_score

rf_rec_sadness = recall_score(yTest, predict, average="micro", labels=["sadness"])
rf_rec_pleasure = recall_score(yTest, predict, average="micro", labels=["pleasure"])
rf_rec_anger = recall_score(yTest, predict, average="micro", labels=["anger"])
rf_rec_joy = recall_score(yTest, predict, average="micro", labels=["joy"])


svm_rec_sadness = recall_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_rec_pleasure = recall_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_rec_anger = recall_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_rec_joy = recall_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_rec_sadness = recall_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_rec_pleasure = recall_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_rec_anger = recall_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_rec_joy = recall_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import f1_score

rf_f1_sadness = f1_score(yTest, predict, average="micro", labels=["sadness"])
rf_f1_pleasure = f1_score(yTest, predict, average="micro", labels=["pleasure"])
rf_f1_anger = f1_score(yTest, predict, average="micro", labels=["anger"])
rf_f1_joy = f1_score(yTest, predict, average="micro", labels=["joy"])


svm_f1_sadness = f1_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_f1_pleasure = f1_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_f1_anger = f1_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_f1_joy = f1_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_f1_sadness = f1_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_f1_pleasure = f1_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_f1_anger = f1_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_f1_joy = f1_score(yTest, knn_predict, average="micro", labels=["joy"])

data_CSV = [subject_id, 6, "STFT", "All bands", "-", "Random Forest", rf_acc, rf_prec_sadness, rf_prec_pleasure, rf_prec_anger, rf_prec_joy, rf_rec_sadness, rf_rec_pleasure, rf_rec_anger, rf_rec_joy, rf_f1_sadness, rf_f1_pleasure, rf_f1_anger, rf_f1_joy], [subject_id, 6, "STFT", "All bands", "-", "SVM", svm_acc, svm_prec_sadness, svm_prec_pleasure, svm_prec_anger, svm_prec_joy, svm_rec_sadness, svm_rec_pleasure, svm_rec_anger, svm_rec_joy, svm_f1_sadness, svm_f1_pleasure, svm_f1_anger, svm_f1_joy], [subject_id, 6, "STFT", "All bands", "-", "kNN", knn_acc, knn_prec_sadness, knn_prec_pleasure, knn_prec_anger, knn_prec_joy, knn_rec_sadness, knn_rec_pleasure, knn_rec_anger, knn_rec_joy, knn_f1_sadness, knn_f1_pleasure, knn_f1_anger, knn_f1_joy]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment6_1.csv", mode="a", header=False, index=False)

f = "data_preprocessed_python\\s10.dat"
subject_id = 10
dictRaw = pd.read_pickle(f)

labels = dictRaw.get('labels')

dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

data = dictRaw.get('data')

df_videos = {}
    #Recorrido de cada una de las columnas de la tabla 3d, y guardado en una una lista de dataframes para cada uno de los videos.
for i in range(40):
    df_videos[i] = pd.DataFrame(data[i])
    df_videos[i] = df_videos[i].transpose()
    df_videos[i].columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature']
    df_videos[i].drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature'],axis = 'columns', inplace=True)

for video in range(0, 40):
    delta_bands = []
    theta_bands = []
    beta_bands = []
    alpha_bands = []
    gamma_bands = []

    for channel in ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']:
        Zxx = do_stft(video, channel)
        eeg_band_fft = make_bands(Zxx)
     #   print(video)
      #  print("\n")
       # print(channel)
        freq = channel_freq(eeg_band_fft)

        delta_bands.append(freq[0])
        theta_bands.append(freq[1])
        alpha_bands.append(freq[2])
        beta_bands.append(freq[3])
        gamma_bands.append(freq[4])

# Banda delta
df = pd.DataFrame(delta_bands)
delta_df = df.transpose()
delta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
delta_df

rows = list()
for _,row in delta_df.iterrows():
    rows += [row]*40
aux_delta = pd.DataFrame(rows).reset_index(drop=True)

# Banda theta
df = pd.DataFrame(theta_bands)
theta_df = df.transpose()
theta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
theta_df

rows = list()
for _,row in theta_df.iterrows():
    rows += [row]*40
aux_theta = pd.DataFrame(rows).reset_index(drop=True)

#Bnada alpha
df = pd.DataFrame(alpha_bands)
alpha_df = df.transpose()
alpha_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
alpha_df

rows = list()
for _,row in alpha_df.iterrows():
    rows += [row]*40
aux_alpha = pd.DataFrame(rows).reset_index(drop=True)

# Banda beta
df = pd.DataFrame(beta_bands)
beta_df = df.transpose()
beta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
beta_df

rows = list()
for _,row in beta_df.iterrows():
    rows += [row]*40
aux_beta = pd.DataFrame(rows).reset_index(drop=True)

# Banda gamma
df = pd.DataFrame(gamma_bands)
gamma_df = df.transpose()
gamma_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
gamma_df

rows = list()
for _,row in gamma_df.iterrows():
    rows += [row]*40
aux_gamma = pd.DataFrame(rows).reset_index(drop=True)

df_labels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])
rows = list()
for _,row in dfLabels.iterrows():
    rows += [row]*64
aux_labels = pd.DataFrame(rows).reset_index(drop=True)

data_delta = aux_delta.merge(aux_labels, left_index=True, right_index=True)
data_theta = aux_theta.merge(aux_labels, left_index=True, right_index=True)
data_alpha = aux_alpha.merge(aux_labels, left_index=True, right_index=True)
data_beta = aux_beta.merge(aux_labels, left_index=True, right_index=True)
data_gamma = aux_gamma.merge(aux_labels, left_index=True, right_index=True)

frames = [aux_delta, aux_theta, aux_alpha, aux_beta, aux_gamma]
aux_all_bands = pd.concat(frames, axis=1)
all_bands_df = aux_all_bands.merge(aux_labels, left_index=True, right_index=True)

data = all_bands_df.drop(['dominance', 'liking'], axis=1)

conditions = [
    ((data['valence'] <4.5) & (data['arousal']<4.5)),
    ((data['valence'] <4.5) & (data['arousal'] > 4.5)),
    ((data['valence'] >4.5) & (data['arousal']<4.5)),
    ((data['valence'] >4.5) & (data['arousal'] > 4.5))]
choices = ['sadness', 'pleasure', 'anger', 'joy']

data['valence'] = np.select(conditions, choices, default='black')
data = data.rename(columns={'valence' : 'label'})
data = data.drop(columns='arousal')

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['label']]

x = copia.drop(columns=['label'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment6_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)
    
rf_acc = accuracy_score(yTest, predict)
svm_acc = accuracy_score(yTest, svm_predict)
knn_acc = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_sadness = precision_score(yTest, predict, average="micro", labels=["sadness"])
rf_prec_pleasure = precision_score(yTest, predict, average="micro", labels=["pleasure"])
rf_prec_anger = precision_score(yTest, predict, average="micro", labels=["anger"])
rf_prec_joy = precision_score(yTest, predict, average="micro", labels=["joy"])


svm_prec_sadness = precision_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_prec_pleasure = precision_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_prec_anger = precision_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_prec_joy = precision_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_prec_sadness = precision_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_prec_pleasure = precision_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_prec_anger = precision_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_prec_joy = precision_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import recall_score

rf_rec_sadness = recall_score(yTest, predict, average="micro", labels=["sadness"])
rf_rec_pleasure = recall_score(yTest, predict, average="micro", labels=["pleasure"])
rf_rec_anger = recall_score(yTest, predict, average="micro", labels=["anger"])
rf_rec_joy = recall_score(yTest, predict, average="micro", labels=["joy"])


svm_rec_sadness = recall_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_rec_pleasure = recall_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_rec_anger = recall_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_rec_joy = recall_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_rec_sadness = recall_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_rec_pleasure = recall_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_rec_anger = recall_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_rec_joy = recall_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import f1_score

rf_f1_sadness = f1_score(yTest, predict, average="micro", labels=["sadness"])
rf_f1_pleasure = f1_score(yTest, predict, average="micro", labels=["pleasure"])
rf_f1_anger = f1_score(yTest, predict, average="micro", labels=["anger"])
rf_f1_joy = f1_score(yTest, predict, average="micro", labels=["joy"])


svm_f1_sadness = f1_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_f1_pleasure = f1_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_f1_anger = f1_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_f1_joy = f1_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_f1_sadness = f1_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_f1_pleasure = f1_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_f1_anger = f1_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_f1_joy = f1_score(yTest, knn_predict, average="micro", labels=["joy"])

data_CSV = [subject_id, 6, "STFT", "All bands", "-", "Random Forest", rf_acc, rf_prec_sadness, rf_prec_pleasure, rf_prec_anger, rf_prec_joy, rf_rec_sadness, rf_rec_pleasure, rf_rec_anger, rf_rec_joy, rf_f1_sadness, rf_f1_pleasure, rf_f1_anger, rf_f1_joy], [subject_id, 6, "STFT", "All bands", "-", "SVM", svm_acc, svm_prec_sadness, svm_prec_pleasure, svm_prec_anger, svm_prec_joy, svm_rec_sadness, svm_rec_pleasure, svm_rec_anger, svm_rec_joy, svm_f1_sadness, svm_f1_pleasure, svm_f1_anger, svm_f1_joy], [subject_id, 6, "STFT", "All bands", "-", "kNN", knn_acc, knn_prec_sadness, knn_prec_pleasure, knn_prec_anger, knn_prec_joy, knn_rec_sadness, knn_rec_pleasure, knn_rec_anger, knn_rec_joy, knn_f1_sadness, knn_f1_pleasure, knn_f1_anger, knn_f1_joy]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment6_1.csv", mode="a", header=False, index=False)

f = "data_preprocessed_python\\s11.dat"
subject_id = 11
dictRaw = pd.read_pickle(f)

labels = dictRaw.get('labels')

dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

data = dictRaw.get('data')

df_videos = {}
    #Recorrido de cada una de las columnas de la tabla 3d, y guardado en una una lista de dataframes para cada uno de los videos.
for i in range(40):
    df_videos[i] = pd.DataFrame(data[i])
    df_videos[i] = df_videos[i].transpose()
    df_videos[i].columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature']
    df_videos[i].drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature'],axis = 'columns', inplace=True)

for video in range(0, 40):
    delta_bands = []
    theta_bands = []
    beta_bands = []
    alpha_bands = []
    gamma_bands = []

    for channel in ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']:
        Zxx = do_stft(video, channel)
        eeg_band_fft = make_bands(Zxx)
     #   print(video)
      #  print("\n")
       # print(channel)
        freq = channel_freq(eeg_band_fft)

        delta_bands.append(freq[0])
        theta_bands.append(freq[1])
        alpha_bands.append(freq[2])
        beta_bands.append(freq[3])
        gamma_bands.append(freq[4])

# Banda delta
df = pd.DataFrame(delta_bands)
delta_df = df.transpose()
delta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
delta_df

rows = list()
for _,row in delta_df.iterrows():
    rows += [row]*40
aux_delta = pd.DataFrame(rows).reset_index(drop=True)

# Banda theta
df = pd.DataFrame(theta_bands)
theta_df = df.transpose()
theta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
theta_df

rows = list()
for _,row in theta_df.iterrows():
    rows += [row]*40
aux_theta = pd.DataFrame(rows).reset_index(drop=True)

#Bnada alpha
df = pd.DataFrame(alpha_bands)
alpha_df = df.transpose()
alpha_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
alpha_df

rows = list()
for _,row in alpha_df.iterrows():
    rows += [row]*40
aux_alpha = pd.DataFrame(rows).reset_index(drop=True)

# Banda beta
df = pd.DataFrame(beta_bands)
beta_df = df.transpose()
beta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
beta_df

rows = list()
for _,row in beta_df.iterrows():
    rows += [row]*40
aux_beta = pd.DataFrame(rows).reset_index(drop=True)

# Banda gamma
df = pd.DataFrame(gamma_bands)
gamma_df = df.transpose()
gamma_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
gamma_df

rows = list()
for _,row in gamma_df.iterrows():
    rows += [row]*40
aux_gamma = pd.DataFrame(rows).reset_index(drop=True)

df_labels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])
rows = list()
for _,row in dfLabels.iterrows():
    rows += [row]*64
aux_labels = pd.DataFrame(rows).reset_index(drop=True)

data_delta = aux_delta.merge(aux_labels, left_index=True, right_index=True)
data_theta = aux_theta.merge(aux_labels, left_index=True, right_index=True)
data_alpha = aux_alpha.merge(aux_labels, left_index=True, right_index=True)
data_beta = aux_beta.merge(aux_labels, left_index=True, right_index=True)
data_gamma = aux_gamma.merge(aux_labels, left_index=True, right_index=True)

frames = [aux_delta, aux_theta, aux_alpha, aux_beta, aux_gamma]
aux_all_bands = pd.concat(frames, axis=1)
all_bands_df = aux_all_bands.merge(aux_labels, left_index=True, right_index=True)

data = all_bands_df.drop(['dominance', 'liking'], axis=1)

conditions = [
    ((data['valence'] <4.5) & (data['arousal']<4.5)),
    ((data['valence'] <4.5) & (data['arousal'] > 4.5)),
    ((data['valence'] >4.5) & (data['arousal']<4.5)),
    ((data['valence'] >4.5) & (data['arousal'] > 4.5))]
choices = ['sadness', 'pleasure', 'anger', 'joy']

data['valence'] = np.select(conditions, choices, default='black')
data = data.rename(columns={'valence' : 'label'})
data = data.drop(columns='arousal')

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['label']]

x = copia.drop(columns=['label'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment6_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)
    
rf_acc = accuracy_score(yTest, predict)
svm_acc = accuracy_score(yTest, svm_predict)
knn_acc = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_sadness = precision_score(yTest, predict, average="micro", labels=["sadness"])
rf_prec_pleasure = precision_score(yTest, predict, average="micro", labels=["pleasure"])
rf_prec_anger = precision_score(yTest, predict, average="micro", labels=["anger"])
rf_prec_joy = precision_score(yTest, predict, average="micro", labels=["joy"])


svm_prec_sadness = precision_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_prec_pleasure = precision_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_prec_anger = precision_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_prec_joy = precision_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_prec_sadness = precision_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_prec_pleasure = precision_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_prec_anger = precision_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_prec_joy = precision_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import recall_score

rf_rec_sadness = recall_score(yTest, predict, average="micro", labels=["sadness"])
rf_rec_pleasure = recall_score(yTest, predict, average="micro", labels=["pleasure"])
rf_rec_anger = recall_score(yTest, predict, average="micro", labels=["anger"])
rf_rec_joy = recall_score(yTest, predict, average="micro", labels=["joy"])


svm_rec_sadness = recall_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_rec_pleasure = recall_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_rec_anger = recall_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_rec_joy = recall_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_rec_sadness = recall_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_rec_pleasure = recall_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_rec_anger = recall_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_rec_joy = recall_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import f1_score

rf_f1_sadness = f1_score(yTest, predict, average="micro", labels=["sadness"])
rf_f1_pleasure = f1_score(yTest, predict, average="micro", labels=["pleasure"])
rf_f1_anger = f1_score(yTest, predict, average="micro", labels=["anger"])
rf_f1_joy = f1_score(yTest, predict, average="micro", labels=["joy"])


svm_f1_sadness = f1_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_f1_pleasure = f1_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_f1_anger = f1_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_f1_joy = f1_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_f1_sadness = f1_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_f1_pleasure = f1_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_f1_anger = f1_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_f1_joy = f1_score(yTest, knn_predict, average="micro", labels=["joy"])

data_CSV = [subject_id, 6, "STFT", "All bands", "-", "Random Forest", rf_acc, rf_prec_sadness, rf_prec_pleasure, rf_prec_anger, rf_prec_joy, rf_rec_sadness, rf_rec_pleasure, rf_rec_anger, rf_rec_joy, rf_f1_sadness, rf_f1_pleasure, rf_f1_anger, rf_f1_joy], [subject_id, 6, "STFT", "All bands", "-", "SVM", svm_acc, svm_prec_sadness, svm_prec_pleasure, svm_prec_anger, svm_prec_joy, svm_rec_sadness, svm_rec_pleasure, svm_rec_anger, svm_rec_joy, svm_f1_sadness, svm_f1_pleasure, svm_f1_anger, svm_f1_joy], [subject_id, 6, "STFT", "All bands", "-", "kNN", knn_acc, knn_prec_sadness, knn_prec_pleasure, knn_prec_anger, knn_prec_joy, knn_rec_sadness, knn_rec_pleasure, knn_rec_anger, knn_rec_joy, knn_f1_sadness, knn_f1_pleasure, knn_f1_anger, knn_f1_joy]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment6_1.csv", mode="a", header=False, index=False)

f = "data_preprocessed_python\\s12.dat"
subject_id = 12
dictRaw = pd.read_pickle(f)

labels = dictRaw.get('labels')

dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

data = dictRaw.get('data')

df_videos = {}
    #Recorrido de cada una de las columnas de la tabla 3d, y guardado en una una lista de dataframes para cada uno de los videos.
for i in range(40):
    df_videos[i] = pd.DataFrame(data[i])
    df_videos[i] = df_videos[i].transpose()
    df_videos[i].columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature']
    df_videos[i].drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature'],axis = 'columns', inplace=True)

for video in range(0, 40):
    delta_bands = []
    theta_bands = []
    beta_bands = []
    alpha_bands = []
    gamma_bands = []

    for channel in ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']:
        Zxx = do_stft(video, channel)
        eeg_band_fft = make_bands(Zxx)
     #   print(video)
      #  print("\n")
       # print(channel)
        freq = channel_freq(eeg_band_fft)

        delta_bands.append(freq[0])
        theta_bands.append(freq[1])
        alpha_bands.append(freq[2])
        beta_bands.append(freq[3])
        gamma_bands.append(freq[4])

# Banda delta
df = pd.DataFrame(delta_bands)
delta_df = df.transpose()
delta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
delta_df

rows = list()
for _,row in delta_df.iterrows():
    rows += [row]*40
aux_delta = pd.DataFrame(rows).reset_index(drop=True)

# Banda theta
df = pd.DataFrame(theta_bands)
theta_df = df.transpose()
theta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
theta_df

rows = list()
for _,row in theta_df.iterrows():
    rows += [row]*40
aux_theta = pd.DataFrame(rows).reset_index(drop=True)

#Bnada alpha
df = pd.DataFrame(alpha_bands)
alpha_df = df.transpose()
alpha_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
alpha_df

rows = list()
for _,row in alpha_df.iterrows():
    rows += [row]*40
aux_alpha = pd.DataFrame(rows).reset_index(drop=True)

# Banda beta
df = pd.DataFrame(beta_bands)
beta_df = df.transpose()
beta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
beta_df

rows = list()
for _,row in beta_df.iterrows():
    rows += [row]*40
aux_beta = pd.DataFrame(rows).reset_index(drop=True)

# Banda gamma
df = pd.DataFrame(gamma_bands)
gamma_df = df.transpose()
gamma_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
gamma_df

rows = list()
for _,row in gamma_df.iterrows():
    rows += [row]*40
aux_gamma = pd.DataFrame(rows).reset_index(drop=True)

df_labels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])
rows = list()
for _,row in dfLabels.iterrows():
    rows += [row]*64
aux_labels = pd.DataFrame(rows).reset_index(drop=True)

data_delta = aux_delta.merge(aux_labels, left_index=True, right_index=True)
data_theta = aux_theta.merge(aux_labels, left_index=True, right_index=True)
data_alpha = aux_alpha.merge(aux_labels, left_index=True, right_index=True)
data_beta = aux_beta.merge(aux_labels, left_index=True, right_index=True)
data_gamma = aux_gamma.merge(aux_labels, left_index=True, right_index=True)

frames = [aux_delta, aux_theta, aux_alpha, aux_beta, aux_gamma]
aux_all_bands = pd.concat(frames, axis=1)
all_bands_df = aux_all_bands.merge(aux_labels, left_index=True, right_index=True)

data = all_bands_df.drop(['dominance', 'liking'], axis=1)

conditions = [
    ((data['valence'] <4.5) & (data['arousal']<4.5)),
    ((data['valence'] <4.5) & (data['arousal'] > 4.5)),
    ((data['valence'] >4.5) & (data['arousal']<4.5)),
    ((data['valence'] >4.5) & (data['arousal'] > 4.5))]
choices = ['sadness', 'pleasure', 'anger', 'joy']

data['valence'] = np.select(conditions, choices, default='black')
data = data.rename(columns={'valence' : 'label'})
data = data.drop(columns='arousal')

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['label']]

x = copia.drop(columns=['label'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment6_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)
    
rf_acc = accuracy_score(yTest, predict)
svm_acc = accuracy_score(yTest, svm_predict)
knn_acc = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_sadness = precision_score(yTest, predict, average="micro", labels=["sadness"])
rf_prec_pleasure = precision_score(yTest, predict, average="micro", labels=["pleasure"])
rf_prec_anger = precision_score(yTest, predict, average="micro", labels=["anger"])
rf_prec_joy = precision_score(yTest, predict, average="micro", labels=["joy"])


svm_prec_sadness = precision_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_prec_pleasure = precision_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_prec_anger = precision_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_prec_joy = precision_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_prec_sadness = precision_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_prec_pleasure = precision_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_prec_anger = precision_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_prec_joy = precision_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import recall_score

rf_rec_sadness = recall_score(yTest, predict, average="micro", labels=["sadness"])
rf_rec_pleasure = recall_score(yTest, predict, average="micro", labels=["pleasure"])
rf_rec_anger = recall_score(yTest, predict, average="micro", labels=["anger"])
rf_rec_joy = recall_score(yTest, predict, average="micro", labels=["joy"])


svm_rec_sadness = recall_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_rec_pleasure = recall_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_rec_anger = recall_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_rec_joy = recall_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_rec_sadness = recall_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_rec_pleasure = recall_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_rec_anger = recall_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_rec_joy = recall_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import f1_score

rf_f1_sadness = f1_score(yTest, predict, average="micro", labels=["sadness"])
rf_f1_pleasure = f1_score(yTest, predict, average="micro", labels=["pleasure"])
rf_f1_anger = f1_score(yTest, predict, average="micro", labels=["anger"])
rf_f1_joy = f1_score(yTest, predict, average="micro", labels=["joy"])


svm_f1_sadness = f1_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_f1_pleasure = f1_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_f1_anger = f1_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_f1_joy = f1_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_f1_sadness = f1_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_f1_pleasure = f1_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_f1_anger = f1_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_f1_joy = f1_score(yTest, knn_predict, average="micro", labels=["joy"])

data_CSV = [subject_id, 6, "STFT", "All bands", "-", "Random Forest", rf_acc, rf_prec_sadness, rf_prec_pleasure, rf_prec_anger, rf_prec_joy, rf_rec_sadness, rf_rec_pleasure, rf_rec_anger, rf_rec_joy, rf_f1_sadness, rf_f1_pleasure, rf_f1_anger, rf_f1_joy], [subject_id, 6, "STFT", "All bands", "-", "SVM", svm_acc, svm_prec_sadness, svm_prec_pleasure, svm_prec_anger, svm_prec_joy, svm_rec_sadness, svm_rec_pleasure, svm_rec_anger, svm_rec_joy, svm_f1_sadness, svm_f1_pleasure, svm_f1_anger, svm_f1_joy], [subject_id, 6, "STFT", "All bands", "-", "kNN", knn_acc, knn_prec_sadness, knn_prec_pleasure, knn_prec_anger, knn_prec_joy, knn_rec_sadness, knn_rec_pleasure, knn_rec_anger, knn_rec_joy, knn_f1_sadness, knn_f1_pleasure, knn_f1_anger, knn_f1_joy]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment6_1.csv", mode="a", header=False, index=False)

f = "data_preprocessed_python\\s13.dat"
subject_id = 13
dictRaw = pd.read_pickle(f)

labels = dictRaw.get('labels')

dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

data = dictRaw.get('data')

df_videos = {}
    #Recorrido de cada una de las columnas de la tabla 3d, y guardado en una una lista de dataframes para cada uno de los videos.
for i in range(40):
    df_videos[i] = pd.DataFrame(data[i])
    df_videos[i] = df_videos[i].transpose()
    df_videos[i].columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature']
    df_videos[i].drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature'],axis = 'columns', inplace=True)

for video in range(0, 40):
    delta_bands = []
    theta_bands = []
    beta_bands = []
    alpha_bands = []
    gamma_bands = []

    for channel in ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']:
        Zxx = do_stft(video, channel)
        eeg_band_fft = make_bands(Zxx)
     #   print(video)
      #  print("\n")
       # print(channel)
        freq = channel_freq(eeg_band_fft)

        delta_bands.append(freq[0])
        theta_bands.append(freq[1])
        alpha_bands.append(freq[2])
        beta_bands.append(freq[3])
        gamma_bands.append(freq[4])

# Banda delta
df = pd.DataFrame(delta_bands)
delta_df = df.transpose()
delta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
delta_df

rows = list()
for _,row in delta_df.iterrows():
    rows += [row]*40
aux_delta = pd.DataFrame(rows).reset_index(drop=True)

# Banda theta
df = pd.DataFrame(theta_bands)
theta_df = df.transpose()
theta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
theta_df

rows = list()
for _,row in theta_df.iterrows():
    rows += [row]*40
aux_theta = pd.DataFrame(rows).reset_index(drop=True)

#Bnada alpha
df = pd.DataFrame(alpha_bands)
alpha_df = df.transpose()
alpha_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
alpha_df

rows = list()
for _,row in alpha_df.iterrows():
    rows += [row]*40
aux_alpha = pd.DataFrame(rows).reset_index(drop=True)

# Banda beta
df = pd.DataFrame(beta_bands)
beta_df = df.transpose()
beta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
beta_df

rows = list()
for _,row in beta_df.iterrows():
    rows += [row]*40
aux_beta = pd.DataFrame(rows).reset_index(drop=True)

# Banda gamma
df = pd.DataFrame(gamma_bands)
gamma_df = df.transpose()
gamma_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
gamma_df

rows = list()
for _,row in gamma_df.iterrows():
    rows += [row]*40
aux_gamma = pd.DataFrame(rows).reset_index(drop=True)

df_labels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])
rows = list()
for _,row in dfLabels.iterrows():
    rows += [row]*64
aux_labels = pd.DataFrame(rows).reset_index(drop=True)

data_delta = aux_delta.merge(aux_labels, left_index=True, right_index=True)
data_theta = aux_theta.merge(aux_labels, left_index=True, right_index=True)
data_alpha = aux_alpha.merge(aux_labels, left_index=True, right_index=True)
data_beta = aux_beta.merge(aux_labels, left_index=True, right_index=True)
data_gamma = aux_gamma.merge(aux_labels, left_index=True, right_index=True)

frames = [aux_delta, aux_theta, aux_alpha, aux_beta, aux_gamma]
aux_all_bands = pd.concat(frames, axis=1)
all_bands_df = aux_all_bands.merge(aux_labels, left_index=True, right_index=True)

data = all_bands_df.drop(['dominance', 'liking'], axis=1)

conditions = [
    ((data['valence'] <4.5) & (data['arousal']<4.5)),
    ((data['valence'] <4.5) & (data['arousal'] > 4.5)),
    ((data['valence'] >4.5) & (data['arousal']<4.5)),
    ((data['valence'] >4.5) & (data['arousal'] > 4.5))]
choices = ['sadness', 'pleasure', 'anger', 'joy']

data['valence'] = np.select(conditions, choices, default='black')
data = data.rename(columns={'valence' : 'label'})
data = data.drop(columns='arousal')

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['label']]

x = copia.drop(columns=['label'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment6_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)
    
rf_acc = accuracy_score(yTest, predict)
svm_acc = accuracy_score(yTest, svm_predict)
knn_acc = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_sadness = precision_score(yTest, predict, average="micro", labels=["sadness"])
rf_prec_pleasure = precision_score(yTest, predict, average="micro", labels=["pleasure"])
rf_prec_anger = precision_score(yTest, predict, average="micro", labels=["anger"])
rf_prec_joy = precision_score(yTest, predict, average="micro", labels=["joy"])


svm_prec_sadness = precision_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_prec_pleasure = precision_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_prec_anger = precision_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_prec_joy = precision_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_prec_sadness = precision_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_prec_pleasure = precision_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_prec_anger = precision_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_prec_joy = precision_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import recall_score

rf_rec_sadness = recall_score(yTest, predict, average="micro", labels=["sadness"])
rf_rec_pleasure = recall_score(yTest, predict, average="micro", labels=["pleasure"])
rf_rec_anger = recall_score(yTest, predict, average="micro", labels=["anger"])
rf_rec_joy = recall_score(yTest, predict, average="micro", labels=["joy"])


svm_rec_sadness = recall_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_rec_pleasure = recall_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_rec_anger = recall_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_rec_joy = recall_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_rec_sadness = recall_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_rec_pleasure = recall_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_rec_anger = recall_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_rec_joy = recall_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import f1_score

rf_f1_sadness = f1_score(yTest, predict, average="micro", labels=["sadness"])
rf_f1_pleasure = f1_score(yTest, predict, average="micro", labels=["pleasure"])
rf_f1_anger = f1_score(yTest, predict, average="micro", labels=["anger"])
rf_f1_joy = f1_score(yTest, predict, average="micro", labels=["joy"])


svm_f1_sadness = f1_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_f1_pleasure = f1_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_f1_anger = f1_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_f1_joy = f1_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_f1_sadness = f1_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_f1_pleasure = f1_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_f1_anger = f1_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_f1_joy = f1_score(yTest, knn_predict, average="micro", labels=["joy"])

data_CSV = [subject_id, 6, "STFT", "All bands", "-", "Random Forest", rf_acc, rf_prec_sadness, rf_prec_pleasure, rf_prec_anger, rf_prec_joy, rf_rec_sadness, rf_rec_pleasure, rf_rec_anger, rf_rec_joy, rf_f1_sadness, rf_f1_pleasure, rf_f1_anger, rf_f1_joy], [subject_id, 6, "STFT", "All bands", "-", "SVM", svm_acc, svm_prec_sadness, svm_prec_pleasure, svm_prec_anger, svm_prec_joy, svm_rec_sadness, svm_rec_pleasure, svm_rec_anger, svm_rec_joy, svm_f1_sadness, svm_f1_pleasure, svm_f1_anger, svm_f1_joy], [subject_id, 6, "STFT", "All bands", "-", "kNN", knn_acc, knn_prec_sadness, knn_prec_pleasure, knn_prec_anger, knn_prec_joy, knn_rec_sadness, knn_rec_pleasure, knn_rec_anger, knn_rec_joy, knn_f1_sadness, knn_f1_pleasure, knn_f1_anger, knn_f1_joy]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment6_1.csv", mode="a", header=False, index=False)

f = "data_preprocessed_python\\s14.dat"
subject_id = 14
dictRaw = pd.read_pickle(f)

labels = dictRaw.get('labels')

dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

data = dictRaw.get('data')

df_videos = {}
    #Recorrido de cada una de las columnas de la tabla 3d, y guardado en una una lista de dataframes para cada uno de los videos.
for i in range(40):
    df_videos[i] = pd.DataFrame(data[i])
    df_videos[i] = df_videos[i].transpose()
    df_videos[i].columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature']
    df_videos[i].drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature'],axis = 'columns', inplace=True)

for video in range(0, 40):
    delta_bands = []
    theta_bands = []
    beta_bands = []
    alpha_bands = []
    gamma_bands = []

    for channel in ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']:
        Zxx = do_stft(video, channel)
        eeg_band_fft = make_bands(Zxx)
     #   print(video)
      #  print("\n")
       # print(channel)
        freq = channel_freq(eeg_band_fft)

        delta_bands.append(freq[0])
        theta_bands.append(freq[1])
        alpha_bands.append(freq[2])
        beta_bands.append(freq[3])
        gamma_bands.append(freq[4])

# Banda delta
df = pd.DataFrame(delta_bands)
delta_df = df.transpose()
delta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
delta_df

rows = list()
for _,row in delta_df.iterrows():
    rows += [row]*40
aux_delta = pd.DataFrame(rows).reset_index(drop=True)

# Banda theta
df = pd.DataFrame(theta_bands)
theta_df = df.transpose()
theta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
theta_df

rows = list()
for _,row in theta_df.iterrows():
    rows += [row]*40
aux_theta = pd.DataFrame(rows).reset_index(drop=True)

#Bnada alpha
df = pd.DataFrame(alpha_bands)
alpha_df = df.transpose()
alpha_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
alpha_df

rows = list()
for _,row in alpha_df.iterrows():
    rows += [row]*40
aux_alpha = pd.DataFrame(rows).reset_index(drop=True)

# Banda beta
df = pd.DataFrame(beta_bands)
beta_df = df.transpose()
beta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
beta_df

rows = list()
for _,row in beta_df.iterrows():
    rows += [row]*40
aux_beta = pd.DataFrame(rows).reset_index(drop=True)

# Banda gamma
df = pd.DataFrame(gamma_bands)
gamma_df = df.transpose()
gamma_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
gamma_df

rows = list()
for _,row in gamma_df.iterrows():
    rows += [row]*40
aux_gamma = pd.DataFrame(rows).reset_index(drop=True)

df_labels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])
rows = list()
for _,row in dfLabels.iterrows():
    rows += [row]*64
aux_labels = pd.DataFrame(rows).reset_index(drop=True)

data_delta = aux_delta.merge(aux_labels, left_index=True, right_index=True)
data_theta = aux_theta.merge(aux_labels, left_index=True, right_index=True)
data_alpha = aux_alpha.merge(aux_labels, left_index=True, right_index=True)
data_beta = aux_beta.merge(aux_labels, left_index=True, right_index=True)
data_gamma = aux_gamma.merge(aux_labels, left_index=True, right_index=True)

frames = [aux_delta, aux_theta, aux_alpha, aux_beta, aux_gamma]
aux_all_bands = pd.concat(frames, axis=1)
all_bands_df = aux_all_bands.merge(aux_labels, left_index=True, right_index=True)

data = all_bands_df.drop(['dominance', 'liking'], axis=1)

conditions = [
    ((data['valence'] <4.5) & (data['arousal']<4.5)),
    ((data['valence'] <4.5) & (data['arousal'] > 4.5)),
    ((data['valence'] >4.5) & (data['arousal']<4.5)),
    ((data['valence'] >4.5) & (data['arousal'] > 4.5))]
choices = ['sadness', 'pleasure', 'anger', 'joy']

data['valence'] = np.select(conditions, choices, default='black')
data = data.rename(columns={'valence' : 'label'})
data = data.drop(columns='arousal')

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['label']]

x = copia.drop(columns=['label'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment6_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)
    
rf_acc = accuracy_score(yTest, predict)
svm_acc = accuracy_score(yTest, svm_predict)
knn_acc = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_sadness = precision_score(yTest, predict, average="micro", labels=["sadness"])
rf_prec_pleasure = precision_score(yTest, predict, average="micro", labels=["pleasure"])
rf_prec_anger = precision_score(yTest, predict, average="micro", labels=["anger"])
rf_prec_joy = precision_score(yTest, predict, average="micro", labels=["joy"])


svm_prec_sadness = precision_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_prec_pleasure = precision_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_prec_anger = precision_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_prec_joy = precision_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_prec_sadness = precision_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_prec_pleasure = precision_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_prec_anger = precision_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_prec_joy = precision_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import recall_score

rf_rec_sadness = recall_score(yTest, predict, average="micro", labels=["sadness"])
rf_rec_pleasure = recall_score(yTest, predict, average="micro", labels=["pleasure"])
rf_rec_anger = recall_score(yTest, predict, average="micro", labels=["anger"])
rf_rec_joy = recall_score(yTest, predict, average="micro", labels=["joy"])


svm_rec_sadness = recall_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_rec_pleasure = recall_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_rec_anger = recall_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_rec_joy = recall_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_rec_sadness = recall_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_rec_pleasure = recall_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_rec_anger = recall_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_rec_joy = recall_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import f1_score

rf_f1_sadness = f1_score(yTest, predict, average="micro", labels=["sadness"])
rf_f1_pleasure = f1_score(yTest, predict, average="micro", labels=["pleasure"])
rf_f1_anger = f1_score(yTest, predict, average="micro", labels=["anger"])
rf_f1_joy = f1_score(yTest, predict, average="micro", labels=["joy"])


svm_f1_sadness = f1_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_f1_pleasure = f1_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_f1_anger = f1_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_f1_joy = f1_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_f1_sadness = f1_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_f1_pleasure = f1_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_f1_anger = f1_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_f1_joy = f1_score(yTest, knn_predict, average="micro", labels=["joy"])

data_CSV = [subject_id, 6, "STFT", "All bands", "-", "Random Forest", rf_acc, rf_prec_sadness, rf_prec_pleasure, rf_prec_anger, rf_prec_joy, rf_rec_sadness, rf_rec_pleasure, rf_rec_anger, rf_rec_joy, rf_f1_sadness, rf_f1_pleasure, rf_f1_anger, rf_f1_joy], [subject_id, 6, "STFT", "All bands", "-", "SVM", svm_acc, svm_prec_sadness, svm_prec_pleasure, svm_prec_anger, svm_prec_joy, svm_rec_sadness, svm_rec_pleasure, svm_rec_anger, svm_rec_joy, svm_f1_sadness, svm_f1_pleasure, svm_f1_anger, svm_f1_joy], [subject_id, 6, "STFT", "All bands", "-", "kNN", knn_acc, knn_prec_sadness, knn_prec_pleasure, knn_prec_anger, knn_prec_joy, knn_rec_sadness, knn_rec_pleasure, knn_rec_anger, knn_rec_joy, knn_f1_sadness, knn_f1_pleasure, knn_f1_anger, knn_f1_joy]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment6_1.csv", mode="a", header=False, index=False)

f = "data_preprocessed_python\\s15.dat"
subject_id = 15
dictRaw = pd.read_pickle(f)

labels = dictRaw.get('labels')

dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

data = dictRaw.get('data')

df_videos = {}
    #Recorrido de cada una de las columnas de la tabla 3d, y guardado en una una lista de dataframes para cada uno de los videos.
for i in range(40):
    df_videos[i] = pd.DataFrame(data[i])
    df_videos[i] = df_videos[i].transpose()
    df_videos[i].columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature']
    df_videos[i].drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature'],axis = 'columns', inplace=True)

for video in range(0, 40):
    delta_bands = []
    theta_bands = []
    beta_bands = []
    alpha_bands = []
    gamma_bands = []

    for channel in ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']:
        Zxx = do_stft(video, channel)
        eeg_band_fft = make_bands(Zxx)
     #   print(video)
      #  print("\n")
       # print(channel)
        freq = channel_freq(eeg_band_fft)

        delta_bands.append(freq[0])
        theta_bands.append(freq[1])
        alpha_bands.append(freq[2])
        beta_bands.append(freq[3])
        gamma_bands.append(freq[4])

# Banda delta
df = pd.DataFrame(delta_bands)
delta_df = df.transpose()
delta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
delta_df

rows = list()
for _,row in delta_df.iterrows():
    rows += [row]*40
aux_delta = pd.DataFrame(rows).reset_index(drop=True)

# Banda theta
df = pd.DataFrame(theta_bands)
theta_df = df.transpose()
theta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
theta_df

rows = list()
for _,row in theta_df.iterrows():
    rows += [row]*40
aux_theta = pd.DataFrame(rows).reset_index(drop=True)

#Bnada alpha
df = pd.DataFrame(alpha_bands)
alpha_df = df.transpose()
alpha_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
alpha_df

rows = list()
for _,row in alpha_df.iterrows():
    rows += [row]*40
aux_alpha = pd.DataFrame(rows).reset_index(drop=True)

# Banda beta
df = pd.DataFrame(beta_bands)
beta_df = df.transpose()
beta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
beta_df

rows = list()
for _,row in beta_df.iterrows():
    rows += [row]*40
aux_beta = pd.DataFrame(rows).reset_index(drop=True)

# Banda gamma
df = pd.DataFrame(gamma_bands)
gamma_df = df.transpose()
gamma_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
gamma_df

rows = list()
for _,row in gamma_df.iterrows():
    rows += [row]*40
aux_gamma = pd.DataFrame(rows).reset_index(drop=True)

df_labels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])
rows = list()
for _,row in dfLabels.iterrows():
    rows += [row]*64
aux_labels = pd.DataFrame(rows).reset_index(drop=True)

data_delta = aux_delta.merge(aux_labels, left_index=True, right_index=True)
data_theta = aux_theta.merge(aux_labels, left_index=True, right_index=True)
data_alpha = aux_alpha.merge(aux_labels, left_index=True, right_index=True)
data_beta = aux_beta.merge(aux_labels, left_index=True, right_index=True)
data_gamma = aux_gamma.merge(aux_labels, left_index=True, right_index=True)

frames = [aux_delta, aux_theta, aux_alpha, aux_beta, aux_gamma]
aux_all_bands = pd.concat(frames, axis=1)
all_bands_df = aux_all_bands.merge(aux_labels, left_index=True, right_index=True)

data = all_bands_df.drop(['dominance', 'liking'], axis=1)

conditions = [
    ((data['valence'] <4.5) & (data['arousal']<4.5)),
    ((data['valence'] <4.5) & (data['arousal'] > 4.5)),
    ((data['valence'] >4.5) & (data['arousal']<4.5)),
    ((data['valence'] >4.5) & (data['arousal'] > 4.5))]
choices = ['sadness', 'pleasure', 'anger', 'joy']

data['valence'] = np.select(conditions, choices, default='black')
data = data.rename(columns={'valence' : 'label'})
data = data.drop(columns='arousal')

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['label']]

x = copia.drop(columns=['label'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment6_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)
    
rf_acc = accuracy_score(yTest, predict)
svm_acc = accuracy_score(yTest, svm_predict)
knn_acc = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_sadness = precision_score(yTest, predict, average="micro", labels=["sadness"])
rf_prec_pleasure = precision_score(yTest, predict, average="micro", labels=["pleasure"])
rf_prec_anger = precision_score(yTest, predict, average="micro", labels=["anger"])
rf_prec_joy = precision_score(yTest, predict, average="micro", labels=["joy"])


svm_prec_sadness = precision_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_prec_pleasure = precision_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_prec_anger = precision_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_prec_joy = precision_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_prec_sadness = precision_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_prec_pleasure = precision_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_prec_anger = precision_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_prec_joy = precision_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import recall_score

rf_rec_sadness = recall_score(yTest, predict, average="micro", labels=["sadness"])
rf_rec_pleasure = recall_score(yTest, predict, average="micro", labels=["pleasure"])
rf_rec_anger = recall_score(yTest, predict, average="micro", labels=["anger"])
rf_rec_joy = recall_score(yTest, predict, average="micro", labels=["joy"])


svm_rec_sadness = recall_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_rec_pleasure = recall_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_rec_anger = recall_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_rec_joy = recall_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_rec_sadness = recall_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_rec_pleasure = recall_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_rec_anger = recall_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_rec_joy = recall_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import f1_score

rf_f1_sadness = f1_score(yTest, predict, average="micro", labels=["sadness"])
rf_f1_pleasure = f1_score(yTest, predict, average="micro", labels=["pleasure"])
rf_f1_anger = f1_score(yTest, predict, average="micro", labels=["anger"])
rf_f1_joy = f1_score(yTest, predict, average="micro", labels=["joy"])


svm_f1_sadness = f1_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_f1_pleasure = f1_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_f1_anger = f1_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_f1_joy = f1_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_f1_sadness = f1_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_f1_pleasure = f1_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_f1_anger = f1_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_f1_joy = f1_score(yTest, knn_predict, average="micro", labels=["joy"])

data_CSV = [subject_id, 6, "STFT", "All bands", "-", "Random Forest", rf_acc, rf_prec_sadness, rf_prec_pleasure, rf_prec_anger, rf_prec_joy, rf_rec_sadness, rf_rec_pleasure, rf_rec_anger, rf_rec_joy, rf_f1_sadness, rf_f1_pleasure, rf_f1_anger, rf_f1_joy], [subject_id, 6, "STFT", "All bands", "-", "SVM", svm_acc, svm_prec_sadness, svm_prec_pleasure, svm_prec_anger, svm_prec_joy, svm_rec_sadness, svm_rec_pleasure, svm_rec_anger, svm_rec_joy, svm_f1_sadness, svm_f1_pleasure, svm_f1_anger, svm_f1_joy], [subject_id, 6, "STFT", "All bands", "-", "kNN", knn_acc, knn_prec_sadness, knn_prec_pleasure, knn_prec_anger, knn_prec_joy, knn_rec_sadness, knn_rec_pleasure, knn_rec_anger, knn_rec_joy, knn_f1_sadness, knn_f1_pleasure, knn_f1_anger, knn_f1_joy]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment6_1.csv", mode="a", header=False, index=False)

f = "data_preprocessed_python\\s16.dat"
subject_id = 16
dictRaw = pd.read_pickle(f)

labels = dictRaw.get('labels')

dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

data = dictRaw.get('data')

df_videos = {}
    #Recorrido de cada una de las columnas de la tabla 3d, y guardado en una una lista de dataframes para cada uno de los videos.
for i in range(40):
    df_videos[i] = pd.DataFrame(data[i])
    df_videos[i] = df_videos[i].transpose()
    df_videos[i].columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature']
    df_videos[i].drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature'],axis = 'columns', inplace=True)

for video in range(0, 40):
    delta_bands = []
    theta_bands = []
    beta_bands = []
    alpha_bands = []
    gamma_bands = []

    for channel in ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']:
        Zxx = do_stft(video, channel)
        eeg_band_fft = make_bands(Zxx)
     #   print(video)
      #  print("\n")
       # print(channel)
        freq = channel_freq(eeg_band_fft)

        delta_bands.append(freq[0])
        theta_bands.append(freq[1])
        alpha_bands.append(freq[2])
        beta_bands.append(freq[3])
        gamma_bands.append(freq[4])

# Banda delta
df = pd.DataFrame(delta_bands)
delta_df = df.transpose()
delta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
delta_df

rows = list()
for _,row in delta_df.iterrows():
    rows += [row]*40
aux_delta = pd.DataFrame(rows).reset_index(drop=True)

# Banda theta
df = pd.DataFrame(theta_bands)
theta_df = df.transpose()
theta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
theta_df

rows = list()
for _,row in theta_df.iterrows():
    rows += [row]*40
aux_theta = pd.DataFrame(rows).reset_index(drop=True)

#Bnada alpha
df = pd.DataFrame(alpha_bands)
alpha_df = df.transpose()
alpha_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
alpha_df

rows = list()
for _,row in alpha_df.iterrows():
    rows += [row]*40
aux_alpha = pd.DataFrame(rows).reset_index(drop=True)

# Banda beta
df = pd.DataFrame(beta_bands)
beta_df = df.transpose()
beta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
beta_df

rows = list()
for _,row in beta_df.iterrows():
    rows += [row]*40
aux_beta = pd.DataFrame(rows).reset_index(drop=True)

# Banda gamma
df = pd.DataFrame(gamma_bands)
gamma_df = df.transpose()
gamma_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
gamma_df

rows = list()
for _,row in gamma_df.iterrows():
    rows += [row]*40
aux_gamma = pd.DataFrame(rows).reset_index(drop=True)

df_labels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])
rows = list()
for _,row in dfLabels.iterrows():
    rows += [row]*64
aux_labels = pd.DataFrame(rows).reset_index(drop=True)

data_delta = aux_delta.merge(aux_labels, left_index=True, right_index=True)
data_theta = aux_theta.merge(aux_labels, left_index=True, right_index=True)
data_alpha = aux_alpha.merge(aux_labels, left_index=True, right_index=True)
data_beta = aux_beta.merge(aux_labels, left_index=True, right_index=True)
data_gamma = aux_gamma.merge(aux_labels, left_index=True, right_index=True)

frames = [aux_delta, aux_theta, aux_alpha, aux_beta, aux_gamma]
aux_all_bands = pd.concat(frames, axis=1)
all_bands_df = aux_all_bands.merge(aux_labels, left_index=True, right_index=True)

data = all_bands_df.drop(['dominance', 'liking'], axis=1)

conditions = [
    ((data['valence'] <4.5) & (data['arousal']<4.5)),
    ((data['valence'] <4.5) & (data['arousal'] > 4.5)),
    ((data['valence'] >4.5) & (data['arousal']<4.5)),
    ((data['valence'] >4.5) & (data['arousal'] > 4.5))]
choices = ['sadness', 'pleasure', 'anger', 'joy']

data['valence'] = np.select(conditions, choices, default='black')
data = data.rename(columns={'valence' : 'label'})
data = data.drop(columns='arousal')

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['label']]

x = copia.drop(columns=['label'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment6_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)
    
rf_acc = accuracy_score(yTest, predict)
svm_acc = accuracy_score(yTest, svm_predict)
knn_acc = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_sadness = precision_score(yTest, predict, average="micro", labels=["sadness"])
rf_prec_pleasure = precision_score(yTest, predict, average="micro", labels=["pleasure"])
rf_prec_anger = precision_score(yTest, predict, average="micro", labels=["anger"])
rf_prec_joy = precision_score(yTest, predict, average="micro", labels=["joy"])


svm_prec_sadness = precision_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_prec_pleasure = precision_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_prec_anger = precision_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_prec_joy = precision_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_prec_sadness = precision_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_prec_pleasure = precision_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_prec_anger = precision_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_prec_joy = precision_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import recall_score

rf_rec_sadness = recall_score(yTest, predict, average="micro", labels=["sadness"])
rf_rec_pleasure = recall_score(yTest, predict, average="micro", labels=["pleasure"])
rf_rec_anger = recall_score(yTest, predict, average="micro", labels=["anger"])
rf_rec_joy = recall_score(yTest, predict, average="micro", labels=["joy"])


svm_rec_sadness = recall_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_rec_pleasure = recall_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_rec_anger = recall_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_rec_joy = recall_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_rec_sadness = recall_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_rec_pleasure = recall_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_rec_anger = recall_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_rec_joy = recall_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import f1_score

rf_f1_sadness = f1_score(yTest, predict, average="micro", labels=["sadness"])
rf_f1_pleasure = f1_score(yTest, predict, average="micro", labels=["pleasure"])
rf_f1_anger = f1_score(yTest, predict, average="micro", labels=["anger"])
rf_f1_joy = f1_score(yTest, predict, average="micro", labels=["joy"])


svm_f1_sadness = f1_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_f1_pleasure = f1_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_f1_anger = f1_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_f1_joy = f1_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_f1_sadness = f1_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_f1_pleasure = f1_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_f1_anger = f1_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_f1_joy = f1_score(yTest, knn_predict, average="micro", labels=["joy"])

data_CSV = [subject_id, 6, "STFT", "All bands", "-", "Random Forest", rf_acc, rf_prec_sadness, rf_prec_pleasure, rf_prec_anger, rf_prec_joy, rf_rec_sadness, rf_rec_pleasure, rf_rec_anger, rf_rec_joy, rf_f1_sadness, rf_f1_pleasure, rf_f1_anger, rf_f1_joy], [subject_id, 6, "STFT", "All bands", "-", "SVM", svm_acc, svm_prec_sadness, svm_prec_pleasure, svm_prec_anger, svm_prec_joy, svm_rec_sadness, svm_rec_pleasure, svm_rec_anger, svm_rec_joy, svm_f1_sadness, svm_f1_pleasure, svm_f1_anger, svm_f1_joy], [subject_id, 6, "STFT", "All bands", "-", "kNN", knn_acc, knn_prec_sadness, knn_prec_pleasure, knn_prec_anger, knn_prec_joy, knn_rec_sadness, knn_rec_pleasure, knn_rec_anger, knn_rec_joy, knn_f1_sadness, knn_f1_pleasure, knn_f1_anger, knn_f1_joy]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment6_1.csv", mode="a", header=False, index=False)

<ipython-input-18-e7d0bc791510>:120: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['valence'] = np.select(conditions, choices, default='black')
[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  80 tasks      | elapsed:    0.2s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    0.3s finished
[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  80 tasks      | elapsed:    0.1s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:  

In [ ]:
f = "data_preprocessed_python\\s17.dat"
subject_id = 17
dictRaw = pd.read_pickle(f)

labels = dictRaw.get('labels')

dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

data = dictRaw.get('data')

df_videos = {}
    #Recorrido de cada una de las columnas de la tabla 3d, y guardado en una una lista de dataframes para cada uno de los videos.
for i in range(40):
    df_videos[i] = pd.DataFrame(data[i])
    df_videos[i] = df_videos[i].transpose()
    df_videos[i].columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature']
    df_videos[i].drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature'],axis = 'columns', inplace=True)

for video in range(0, 40):
    delta_bands = []
    theta_bands = []
    beta_bands = []
    alpha_bands = []
    gamma_bands = []

    for channel in ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']:
        Zxx = do_stft(video, channel)
        eeg_band_fft = make_bands(Zxx)
     #   print(video)
      #  print("\n")
       # print(channel)
        freq = channel_freq(eeg_band_fft)

        delta_bands.append(freq[0])
        theta_bands.append(freq[1])
        alpha_bands.append(freq[2])
        beta_bands.append(freq[3])
        gamma_bands.append(freq[4])

# Banda delta
df = pd.DataFrame(delta_bands)
delta_df = df.transpose()
delta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
delta_df

rows = list()
for _,row in delta_df.iterrows():
    rows += [row]*40
aux_delta = pd.DataFrame(rows).reset_index(drop=True)

# Banda theta
df = pd.DataFrame(theta_bands)
theta_df = df.transpose()
theta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
theta_df

rows = list()
for _,row in theta_df.iterrows():
    rows += [row]*40
aux_theta = pd.DataFrame(rows).reset_index(drop=True)

#Bnada alpha
df = pd.DataFrame(alpha_bands)
alpha_df = df.transpose()
alpha_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
alpha_df

rows = list()
for _,row in alpha_df.iterrows():
    rows += [row]*40
aux_alpha = pd.DataFrame(rows).reset_index(drop=True)

# Banda beta
df = pd.DataFrame(beta_bands)
beta_df = df.transpose()
beta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
beta_df

rows = list()
for _,row in beta_df.iterrows():
    rows += [row]*40
aux_beta = pd.DataFrame(rows).reset_index(drop=True)

# Banda gamma
df = pd.DataFrame(gamma_bands)
gamma_df = df.transpose()
gamma_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
gamma_df

rows = list()
for _,row in gamma_df.iterrows():
    rows += [row]*40
aux_gamma = pd.DataFrame(rows).reset_index(drop=True)

df_labels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])
rows = list()
for _,row in dfLabels.iterrows():
    rows += [row]*64
aux_labels = pd.DataFrame(rows).reset_index(drop=True)

data_delta = aux_delta.merge(aux_labels, left_index=True, right_index=True)
data_theta = aux_theta.merge(aux_labels, left_index=True, right_index=True)
data_alpha = aux_alpha.merge(aux_labels, left_index=True, right_index=True)
data_beta = aux_beta.merge(aux_labels, left_index=True, right_index=True)
data_gamma = aux_gamma.merge(aux_labels, left_index=True, right_index=True)

frames = [aux_delta, aux_theta, aux_alpha, aux_beta, aux_gamma]
aux_all_bands = pd.concat(frames, axis=1)
all_bands_df = aux_all_bands.merge(aux_labels, left_index=True, right_index=True)

data = all_bands_df.drop(['dominance', 'liking'], axis=1)

conditions = [
    ((data['valence'] <4.5) & (data['arousal']<4.5)),
    ((data['valence'] <4.5) & (data['arousal'] > 4.5)),
    ((data['valence'] >4.5) & (data['arousal']<4.5)),
    ((data['valence'] >4.5) & (data['arousal'] > 4.5))]
choices = ['sadness', 'pleasure', 'anger', 'joy']

data['valence'] = np.select(conditions, choices, default='black')
data = data.rename(columns={'valence' : 'label'})
data = data.drop(columns='arousal')

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['label']]

x = copia.drop(columns=['label'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment6_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)
    
rf_acc = accuracy_score(yTest, predict)
svm_acc = accuracy_score(yTest, svm_predict)
knn_acc = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_sadness = precision_score(yTest, predict, average="micro", labels=["sadness"])
rf_prec_pleasure = precision_score(yTest, predict, average="micro", labels=["pleasure"])
rf_prec_anger = precision_score(yTest, predict, average="micro", labels=["anger"])
rf_prec_joy = precision_score(yTest, predict, average="micro", labels=["joy"])


svm_prec_sadness = precision_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_prec_pleasure = precision_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_prec_anger = precision_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_prec_joy = precision_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_prec_sadness = precision_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_prec_pleasure = precision_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_prec_anger = precision_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_prec_joy = precision_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import recall_score

rf_rec_sadness = recall_score(yTest, predict, average="micro", labels=["sadness"])
rf_rec_pleasure = recall_score(yTest, predict, average="micro", labels=["pleasure"])
rf_rec_anger = recall_score(yTest, predict, average="micro", labels=["anger"])
rf_rec_joy = recall_score(yTest, predict, average="micro", labels=["joy"])


svm_rec_sadness = recall_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_rec_pleasure = recall_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_rec_anger = recall_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_rec_joy = recall_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_rec_sadness = recall_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_rec_pleasure = recall_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_rec_anger = recall_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_rec_joy = recall_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import f1_score

rf_f1_sadness = f1_score(yTest, predict, average="micro", labels=["sadness"])
rf_f1_pleasure = f1_score(yTest, predict, average="micro", labels=["pleasure"])
rf_f1_anger = f1_score(yTest, predict, average="micro", labels=["anger"])
rf_f1_joy = f1_score(yTest, predict, average="micro", labels=["joy"])


svm_f1_sadness = f1_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_f1_pleasure = f1_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_f1_anger = f1_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_f1_joy = f1_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_f1_sadness = f1_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_f1_pleasure = f1_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_f1_anger = f1_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_f1_joy = f1_score(yTest, knn_predict, average="micro", labels=["joy"])

data_CSV = [subject_id, 6, "STFT", "All bands", "-", "Random Forest", rf_acc, rf_prec_sadness, rf_prec_pleasure, rf_prec_anger, rf_prec_joy, rf_rec_sadness, rf_rec_pleasure, rf_rec_anger, rf_rec_joy, rf_f1_sadness, rf_f1_pleasure, rf_f1_anger, rf_f1_joy], [subject_id, 6, "STFT", "All bands", "-", "SVM", svm_acc, svm_prec_sadness, svm_prec_pleasure, svm_prec_anger, svm_prec_joy, svm_rec_sadness, svm_rec_pleasure, svm_rec_anger, svm_rec_joy, svm_f1_sadness, svm_f1_pleasure, svm_f1_anger, svm_f1_joy], [subject_id, 6, "STFT", "All bands", "-", "kNN", knn_acc, knn_prec_sadness, knn_prec_pleasure, knn_prec_anger, knn_prec_joy, knn_rec_sadness, knn_rec_pleasure, knn_rec_anger, knn_rec_joy, knn_f1_sadness, knn_f1_pleasure, knn_f1_anger, knn_f1_joy]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment6_1.csv", mode="a", header=False, index=False)

f = "data_preprocessed_python\\s18.dat"
subject_id = 18
dictRaw = pd.read_pickle(f)

labels = dictRaw.get('labels')

dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

data = dictRaw.get('data')

df_videos = {}
    #Recorrido de cada una de las columnas de la tabla 3d, y guardado en una una lista de dataframes para cada uno de los videos.
for i in range(40):
    df_videos[i] = pd.DataFrame(data[i])
    df_videos[i] = df_videos[i].transpose()
    df_videos[i].columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature']
    df_videos[i].drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature'],axis = 'columns', inplace=True)

for video in range(0, 40):
    delta_bands = []
    theta_bands = []
    beta_bands = []
    alpha_bands = []
    gamma_bands = []

    for channel in ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']:
        Zxx = do_stft(video, channel)
        eeg_band_fft = make_bands(Zxx)
     #   print(video)
      #  print("\n")
       # print(channel)
        freq = channel_freq(eeg_band_fft)

        delta_bands.append(freq[0])
        theta_bands.append(freq[1])
        alpha_bands.append(freq[2])
        beta_bands.append(freq[3])
        gamma_bands.append(freq[4])

# Banda delta
df = pd.DataFrame(delta_bands)
delta_df = df.transpose()
delta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
delta_df

rows = list()
for _,row in delta_df.iterrows():
    rows += [row]*40
aux_delta = pd.DataFrame(rows).reset_index(drop=True)

# Banda theta
df = pd.DataFrame(theta_bands)
theta_df = df.transpose()
theta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
theta_df

rows = list()
for _,row in theta_df.iterrows():
    rows += [row]*40
aux_theta = pd.DataFrame(rows).reset_index(drop=True)

#Bnada alpha
df = pd.DataFrame(alpha_bands)
alpha_df = df.transpose()
alpha_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
alpha_df

rows = list()
for _,row in alpha_df.iterrows():
    rows += [row]*40
aux_alpha = pd.DataFrame(rows).reset_index(drop=True)

# Banda beta
df = pd.DataFrame(beta_bands)
beta_df = df.transpose()
beta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
beta_df

rows = list()
for _,row in beta_df.iterrows():
    rows += [row]*40
aux_beta = pd.DataFrame(rows).reset_index(drop=True)

# Banda gamma
df = pd.DataFrame(gamma_bands)
gamma_df = df.transpose()
gamma_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
gamma_df

rows = list()
for _,row in gamma_df.iterrows():
    rows += [row]*40
aux_gamma = pd.DataFrame(rows).reset_index(drop=True)

df_labels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])
rows = list()
for _,row in dfLabels.iterrows():
    rows += [row]*64
aux_labels = pd.DataFrame(rows).reset_index(drop=True)

data_delta = aux_delta.merge(aux_labels, left_index=True, right_index=True)
data_theta = aux_theta.merge(aux_labels, left_index=True, right_index=True)
data_alpha = aux_alpha.merge(aux_labels, left_index=True, right_index=True)
data_beta = aux_beta.merge(aux_labels, left_index=True, right_index=True)
data_gamma = aux_gamma.merge(aux_labels, left_index=True, right_index=True)

frames = [aux_delta, aux_theta, aux_alpha, aux_beta, aux_gamma]
aux_all_bands = pd.concat(frames, axis=1)
all_bands_df = aux_all_bands.merge(aux_labels, left_index=True, right_index=True)

data = all_bands_df.drop(['dominance', 'liking'], axis=1)

conditions = [
    ((data['valence'] <4.5) & (data['arousal']<4.5)),
    ((data['valence'] <4.5) & (data['arousal'] > 4.5)),
    ((data['valence'] >4.5) & (data['arousal']<4.5)),
    ((data['valence'] >4.5) & (data['arousal'] > 4.5))]
choices = ['sadness', 'pleasure', 'anger', 'joy']

data['valence'] = np.select(conditions, choices, default='black')
data = data.rename(columns={'valence' : 'label'})
data = data.drop(columns='arousal')

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['label']]

x = copia.drop(columns=['label'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment6_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)
    
rf_acc = accuracy_score(yTest, predict)
svm_acc = accuracy_score(yTest, svm_predict)
knn_acc = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_sadness = precision_score(yTest, predict, average="micro", labels=["sadness"])
rf_prec_pleasure = precision_score(yTest, predict, average="micro", labels=["pleasure"])
rf_prec_anger = precision_score(yTest, predict, average="micro", labels=["anger"])
rf_prec_joy = precision_score(yTest, predict, average="micro", labels=["joy"])


svm_prec_sadness = precision_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_prec_pleasure = precision_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_prec_anger = precision_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_prec_joy = precision_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_prec_sadness = precision_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_prec_pleasure = precision_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_prec_anger = precision_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_prec_joy = precision_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import recall_score

rf_rec_sadness = recall_score(yTest, predict, average="micro", labels=["sadness"])
rf_rec_pleasure = recall_score(yTest, predict, average="micro", labels=["pleasure"])
rf_rec_anger = recall_score(yTest, predict, average="micro", labels=["anger"])
rf_rec_joy = recall_score(yTest, predict, average="micro", labels=["joy"])


svm_rec_sadness = recall_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_rec_pleasure = recall_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_rec_anger = recall_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_rec_joy = recall_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_rec_sadness = recall_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_rec_pleasure = recall_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_rec_anger = recall_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_rec_joy = recall_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import f1_score

rf_f1_sadness = f1_score(yTest, predict, average="micro", labels=["sadness"])
rf_f1_pleasure = f1_score(yTest, predict, average="micro", labels=["pleasure"])
rf_f1_anger = f1_score(yTest, predict, average="micro", labels=["anger"])
rf_f1_joy = f1_score(yTest, predict, average="micro", labels=["joy"])


svm_f1_sadness = f1_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_f1_pleasure = f1_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_f1_anger = f1_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_f1_joy = f1_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_f1_sadness = f1_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_f1_pleasure = f1_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_f1_anger = f1_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_f1_joy = f1_score(yTest, knn_predict, average="micro", labels=["joy"])

data_CSV = [subject_id, 6, "STFT", "All bands", "-", "Random Forest", rf_acc, rf_prec_sadness, rf_prec_pleasure, rf_prec_anger, rf_prec_joy, rf_rec_sadness, rf_rec_pleasure, rf_rec_anger, rf_rec_joy, rf_f1_sadness, rf_f1_pleasure, rf_f1_anger, rf_f1_joy], [subject_id, 6, "STFT", "All bands", "-", "SVM", svm_acc, svm_prec_sadness, svm_prec_pleasure, svm_prec_anger, svm_prec_joy, svm_rec_sadness, svm_rec_pleasure, svm_rec_anger, svm_rec_joy, svm_f1_sadness, svm_f1_pleasure, svm_f1_anger, svm_f1_joy], [subject_id, 6, "STFT", "All bands", "-", "kNN", knn_acc, knn_prec_sadness, knn_prec_pleasure, knn_prec_anger, knn_prec_joy, knn_rec_sadness, knn_rec_pleasure, knn_rec_anger, knn_rec_joy, knn_f1_sadness, knn_f1_pleasure, knn_f1_anger, knn_f1_joy]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment6_1.csv", mode="a", header=False, index=False)

f = "data_preprocessed_python\\s19.dat"
subject_id = 19
dictRaw = pd.read_pickle(f)

labels = dictRaw.get('labels')

dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

data = dictRaw.get('data')

df_videos = {}
    #Recorrido de cada una de las columnas de la tabla 3d, y guardado en una una lista de dataframes para cada uno de los videos.
for i in range(40):
    df_videos[i] = pd.DataFrame(data[i])
    df_videos[i] = df_videos[i].transpose()
    df_videos[i].columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature']
    df_videos[i].drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature'],axis = 'columns', inplace=True)

for video in range(0, 40):
    delta_bands = []
    theta_bands = []
    beta_bands = []
    alpha_bands = []
    gamma_bands = []

    for channel in ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']:
        Zxx = do_stft(video, channel)
        eeg_band_fft = make_bands(Zxx)
     #   print(video)
      #  print("\n")
       # print(channel)
        freq = channel_freq(eeg_band_fft)

        delta_bands.append(freq[0])
        theta_bands.append(freq[1])
        alpha_bands.append(freq[2])
        beta_bands.append(freq[3])
        gamma_bands.append(freq[4])

# Banda delta
df = pd.DataFrame(delta_bands)
delta_df = df.transpose()
delta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
delta_df

rows = list()
for _,row in delta_df.iterrows():
    rows += [row]*40
aux_delta = pd.DataFrame(rows).reset_index(drop=True)

# Banda theta
df = pd.DataFrame(theta_bands)
theta_df = df.transpose()
theta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
theta_df

rows = list()
for _,row in theta_df.iterrows():
    rows += [row]*40
aux_theta = pd.DataFrame(rows).reset_index(drop=True)

#Bnada alpha
df = pd.DataFrame(alpha_bands)
alpha_df = df.transpose()
alpha_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
alpha_df

rows = list()
for _,row in alpha_df.iterrows():
    rows += [row]*40
aux_alpha = pd.DataFrame(rows).reset_index(drop=True)

# Banda beta
df = pd.DataFrame(beta_bands)
beta_df = df.transpose()
beta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
beta_df

rows = list()
for _,row in beta_df.iterrows():
    rows += [row]*40
aux_beta = pd.DataFrame(rows).reset_index(drop=True)

# Banda gamma
df = pd.DataFrame(gamma_bands)
gamma_df = df.transpose()
gamma_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
gamma_df

rows = list()
for _,row in gamma_df.iterrows():
    rows += [row]*40
aux_gamma = pd.DataFrame(rows).reset_index(drop=True)

df_labels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])
rows = list()
for _,row in dfLabels.iterrows():
    rows += [row]*64
aux_labels = pd.DataFrame(rows).reset_index(drop=True)

data_delta = aux_delta.merge(aux_labels, left_index=True, right_index=True)
data_theta = aux_theta.merge(aux_labels, left_index=True, right_index=True)
data_alpha = aux_alpha.merge(aux_labels, left_index=True, right_index=True)
data_beta = aux_beta.merge(aux_labels, left_index=True, right_index=True)
data_gamma = aux_gamma.merge(aux_labels, left_index=True, right_index=True)

frames = [aux_delta, aux_theta, aux_alpha, aux_beta, aux_gamma]
aux_all_bands = pd.concat(frames, axis=1)
all_bands_df = aux_all_bands.merge(aux_labels, left_index=True, right_index=True)

data = all_bands_df.drop(['dominance', 'liking'], axis=1)

conditions = [
    ((data['valence'] <4.5) & (data['arousal']<4.5)),
    ((data['valence'] <4.5) & (data['arousal'] > 4.5)),
    ((data['valence'] >4.5) & (data['arousal']<4.5)),
    ((data['valence'] >4.5) & (data['arousal'] > 4.5))]
choices = ['sadness', 'pleasure', 'anger', 'joy']

data['valence'] = np.select(conditions, choices, default='black')
data = data.rename(columns={'valence' : 'label'})
data = data.drop(columns='arousal')

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['label']]

x = copia.drop(columns=['label'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment6_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)
    
rf_acc = accuracy_score(yTest, predict)
svm_acc = accuracy_score(yTest, svm_predict)
knn_acc = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_sadness = precision_score(yTest, predict, average="micro", labels=["sadness"])
rf_prec_pleasure = precision_score(yTest, predict, average="micro", labels=["pleasure"])
rf_prec_anger = precision_score(yTest, predict, average="micro", labels=["anger"])
rf_prec_joy = precision_score(yTest, predict, average="micro", labels=["joy"])


svm_prec_sadness = precision_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_prec_pleasure = precision_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_prec_anger = precision_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_prec_joy = precision_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_prec_sadness = precision_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_prec_pleasure = precision_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_prec_anger = precision_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_prec_joy = precision_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import recall_score

rf_rec_sadness = recall_score(yTest, predict, average="micro", labels=["sadness"])
rf_rec_pleasure = recall_score(yTest, predict, average="micro", labels=["pleasure"])
rf_rec_anger = recall_score(yTest, predict, average="micro", labels=["anger"])
rf_rec_joy = recall_score(yTest, predict, average="micro", labels=["joy"])


svm_rec_sadness = recall_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_rec_pleasure = recall_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_rec_anger = recall_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_rec_joy = recall_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_rec_sadness = recall_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_rec_pleasure = recall_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_rec_anger = recall_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_rec_joy = recall_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import f1_score

rf_f1_sadness = f1_score(yTest, predict, average="micro", labels=["sadness"])
rf_f1_pleasure = f1_score(yTest, predict, average="micro", labels=["pleasure"])
rf_f1_anger = f1_score(yTest, predict, average="micro", labels=["anger"])
rf_f1_joy = f1_score(yTest, predict, average="micro", labels=["joy"])


svm_f1_sadness = f1_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_f1_pleasure = f1_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_f1_anger = f1_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_f1_joy = f1_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_f1_sadness = f1_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_f1_pleasure = f1_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_f1_anger = f1_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_f1_joy = f1_score(yTest, knn_predict, average="micro", labels=["joy"])

data_CSV = [subject_id, 6, "STFT", "All bands", "-", "Random Forest", rf_acc, rf_prec_sadness, rf_prec_pleasure, rf_prec_anger, rf_prec_joy, rf_rec_sadness, rf_rec_pleasure, rf_rec_anger, rf_rec_joy, rf_f1_sadness, rf_f1_pleasure, rf_f1_anger, rf_f1_joy], [subject_id, 6, "STFT", "All bands", "-", "SVM", svm_acc, svm_prec_sadness, svm_prec_pleasure, svm_prec_anger, svm_prec_joy, svm_rec_sadness, svm_rec_pleasure, svm_rec_anger, svm_rec_joy, svm_f1_sadness, svm_f1_pleasure, svm_f1_anger, svm_f1_joy], [subject_id, 6, "STFT", "All bands", "-", "kNN", knn_acc, knn_prec_sadness, knn_prec_pleasure, knn_prec_anger, knn_prec_joy, knn_rec_sadness, knn_rec_pleasure, knn_rec_anger, knn_rec_joy, knn_f1_sadness, knn_f1_pleasure, knn_f1_anger, knn_f1_joy]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment6_1.csv", mode="a", header=False, index=False)

f = "data_preprocessed_python\\s20.dat"
subject_id = 20
dictRaw = pd.read_pickle(f)

labels = dictRaw.get('labels')

dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

data = dictRaw.get('data')

df_videos = {}
    #Recorrido de cada una de las columnas de la tabla 3d, y guardado en una una lista de dataframes para cada uno de los videos.
for i in range(40):
    df_videos[i] = pd.DataFrame(data[i])
    df_videos[i] = df_videos[i].transpose()
    df_videos[i].columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature']
    df_videos[i].drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature'],axis = 'columns', inplace=True)

for video in range(0, 40):
    delta_bands = []
    theta_bands = []
    beta_bands = []
    alpha_bands = []
    gamma_bands = []

    for channel in ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']:
        Zxx = do_stft(video, channel)
        eeg_band_fft = make_bands(Zxx)
     #   print(video)
      #  print("\n")
       # print(channel)
        freq = channel_freq(eeg_band_fft)

        delta_bands.append(freq[0])
        theta_bands.append(freq[1])
        alpha_bands.append(freq[2])
        beta_bands.append(freq[3])
        gamma_bands.append(freq[4])

# Banda delta
df = pd.DataFrame(delta_bands)
delta_df = df.transpose()
delta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
delta_df

rows = list()
for _,row in delta_df.iterrows():
    rows += [row]*40
aux_delta = pd.DataFrame(rows).reset_index(drop=True)

# Banda theta
df = pd.DataFrame(theta_bands)
theta_df = df.transpose()
theta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
theta_df

rows = list()
for _,row in theta_df.iterrows():
    rows += [row]*40
aux_theta = pd.DataFrame(rows).reset_index(drop=True)

#Bnada alpha
df = pd.DataFrame(alpha_bands)
alpha_df = df.transpose()
alpha_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
alpha_df

rows = list()
for _,row in alpha_df.iterrows():
    rows += [row]*40
aux_alpha = pd.DataFrame(rows).reset_index(drop=True)

# Banda beta
df = pd.DataFrame(beta_bands)
beta_df = df.transpose()
beta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
beta_df

rows = list()
for _,row in beta_df.iterrows():
    rows += [row]*40
aux_beta = pd.DataFrame(rows).reset_index(drop=True)

# Banda gamma
df = pd.DataFrame(gamma_bands)
gamma_df = df.transpose()
gamma_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
gamma_df

rows = list()
for _,row in gamma_df.iterrows():
    rows += [row]*40
aux_gamma = pd.DataFrame(rows).reset_index(drop=True)

df_labels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])
rows = list()
for _,row in dfLabels.iterrows():
    rows += [row]*64
aux_labels = pd.DataFrame(rows).reset_index(drop=True)

data_delta = aux_delta.merge(aux_labels, left_index=True, right_index=True)
data_theta = aux_theta.merge(aux_labels, left_index=True, right_index=True)
data_alpha = aux_alpha.merge(aux_labels, left_index=True, right_index=True)
data_beta = aux_beta.merge(aux_labels, left_index=True, right_index=True)
data_gamma = aux_gamma.merge(aux_labels, left_index=True, right_index=True)

frames = [aux_delta, aux_theta, aux_alpha, aux_beta, aux_gamma]
aux_all_bands = pd.concat(frames, axis=1)
all_bands_df = aux_all_bands.merge(aux_labels, left_index=True, right_index=True)

data = all_bands_df.drop(['dominance', 'liking'], axis=1)

conditions = [
    ((data['valence'] <4.5) & (data['arousal']<4.5)),
    ((data['valence'] <4.5) & (data['arousal'] > 4.5)),
    ((data['valence'] >4.5) & (data['arousal']<4.5)),
    ((data['valence'] >4.5) & (data['arousal'] > 4.5))]
choices = ['sadness', 'pleasure', 'anger', 'joy']

data['valence'] = np.select(conditions, choices, default='black')
data = data.rename(columns={'valence' : 'label'})
data = data.drop(columns='arousal')

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['label']]

x = copia.drop(columns=['label'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment6_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)
    
rf_acc = accuracy_score(yTest, predict)
svm_acc = accuracy_score(yTest, svm_predict)
knn_acc = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_sadness = precision_score(yTest, predict, average="micro", labels=["sadness"])
rf_prec_pleasure = precision_score(yTest, predict, average="micro", labels=["pleasure"])
rf_prec_anger = precision_score(yTest, predict, average="micro", labels=["anger"])
rf_prec_joy = precision_score(yTest, predict, average="micro", labels=["joy"])


svm_prec_sadness = precision_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_prec_pleasure = precision_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_prec_anger = precision_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_prec_joy = precision_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_prec_sadness = precision_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_prec_pleasure = precision_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_prec_anger = precision_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_prec_joy = precision_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import recall_score

rf_rec_sadness = recall_score(yTest, predict, average="micro", labels=["sadness"])
rf_rec_pleasure = recall_score(yTest, predict, average="micro", labels=["pleasure"])
rf_rec_anger = recall_score(yTest, predict, average="micro", labels=["anger"])
rf_rec_joy = recall_score(yTest, predict, average="micro", labels=["joy"])


svm_rec_sadness = recall_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_rec_pleasure = recall_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_rec_anger = recall_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_rec_joy = recall_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_rec_sadness = recall_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_rec_pleasure = recall_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_rec_anger = recall_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_rec_joy = recall_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import f1_score

rf_f1_sadness = f1_score(yTest, predict, average="micro", labels=["sadness"])
rf_f1_pleasure = f1_score(yTest, predict, average="micro", labels=["pleasure"])
rf_f1_anger = f1_score(yTest, predict, average="micro", labels=["anger"])
rf_f1_joy = f1_score(yTest, predict, average="micro", labels=["joy"])


svm_f1_sadness = f1_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_f1_pleasure = f1_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_f1_anger = f1_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_f1_joy = f1_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_f1_sadness = f1_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_f1_pleasure = f1_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_f1_anger = f1_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_f1_joy = f1_score(yTest, knn_predict, average="micro", labels=["joy"])

data_CSV = [subject_id, 6, "STFT", "All bands", "-", "Random Forest", rf_acc, rf_prec_sadness, rf_prec_pleasure, rf_prec_anger, rf_prec_joy, rf_rec_sadness, rf_rec_pleasure, rf_rec_anger, rf_rec_joy, rf_f1_sadness, rf_f1_pleasure, rf_f1_anger, rf_f1_joy], [subject_id, 6, "STFT", "All bands", "-", "SVM", svm_acc, svm_prec_sadness, svm_prec_pleasure, svm_prec_anger, svm_prec_joy, svm_rec_sadness, svm_rec_pleasure, svm_rec_anger, svm_rec_joy, svm_f1_sadness, svm_f1_pleasure, svm_f1_anger, svm_f1_joy], [subject_id, 6, "STFT", "All bands", "-", "kNN", knn_acc, knn_prec_sadness, knn_prec_pleasure, knn_prec_anger, knn_prec_joy, knn_rec_sadness, knn_rec_pleasure, knn_rec_anger, knn_rec_joy, knn_f1_sadness, knn_f1_pleasure, knn_f1_anger, knn_f1_joy]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment6_1.csv", mode="a", header=False, index=False)

f = "data_preprocessed_python\\s21.dat"
subject_id = 21
dictRaw = pd.read_pickle(f)

labels = dictRaw.get('labels')

dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

data = dictRaw.get('data')

df_videos = {}
    #Recorrido de cada una de las columnas de la tabla 3d, y guardado en una una lista de dataframes para cada uno de los videos.
for i in range(40):
    df_videos[i] = pd.DataFrame(data[i])
    df_videos[i] = df_videos[i].transpose()
    df_videos[i].columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature']
    df_videos[i].drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature'],axis = 'columns', inplace=True)

for video in range(0, 40):
    delta_bands = []
    theta_bands = []
    beta_bands = []
    alpha_bands = []
    gamma_bands = []

    for channel in ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']:
        Zxx = do_stft(video, channel)
        eeg_band_fft = make_bands(Zxx)
     #   print(video)
      #  print("\n")
       # print(channel)
        freq = channel_freq(eeg_band_fft)

        delta_bands.append(freq[0])
        theta_bands.append(freq[1])
        alpha_bands.append(freq[2])
        beta_bands.append(freq[3])
        gamma_bands.append(freq[4])

# Banda delta
df = pd.DataFrame(delta_bands)
delta_df = df.transpose()
delta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
delta_df

rows = list()
for _,row in delta_df.iterrows():
    rows += [row]*40
aux_delta = pd.DataFrame(rows).reset_index(drop=True)

# Banda theta
df = pd.DataFrame(theta_bands)
theta_df = df.transpose()
theta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
theta_df

rows = list()
for _,row in theta_df.iterrows():
    rows += [row]*40
aux_theta = pd.DataFrame(rows).reset_index(drop=True)

#Bnada alpha
df = pd.DataFrame(alpha_bands)
alpha_df = df.transpose()
alpha_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
alpha_df

rows = list()
for _,row in alpha_df.iterrows():
    rows += [row]*40
aux_alpha = pd.DataFrame(rows).reset_index(drop=True)

# Banda beta
df = pd.DataFrame(beta_bands)
beta_df = df.transpose()
beta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
beta_df

rows = list()
for _,row in beta_df.iterrows():
    rows += [row]*40
aux_beta = pd.DataFrame(rows).reset_index(drop=True)

# Banda gamma
df = pd.DataFrame(gamma_bands)
gamma_df = df.transpose()
gamma_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
gamma_df

rows = list()
for _,row in gamma_df.iterrows():
    rows += [row]*40
aux_gamma = pd.DataFrame(rows).reset_index(drop=True)

df_labels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])
rows = list()
for _,row in dfLabels.iterrows():
    rows += [row]*64
aux_labels = pd.DataFrame(rows).reset_index(drop=True)

data_delta = aux_delta.merge(aux_labels, left_index=True, right_index=True)
data_theta = aux_theta.merge(aux_labels, left_index=True, right_index=True)
data_alpha = aux_alpha.merge(aux_labels, left_index=True, right_index=True)
data_beta = aux_beta.merge(aux_labels, left_index=True, right_index=True)
data_gamma = aux_gamma.merge(aux_labels, left_index=True, right_index=True)

frames = [aux_delta, aux_theta, aux_alpha, aux_beta, aux_gamma]
aux_all_bands = pd.concat(frames, axis=1)
all_bands_df = aux_all_bands.merge(aux_labels, left_index=True, right_index=True)

data = all_bands_df.drop(['dominance', 'liking'], axis=1)

conditions = [
    ((data['valence'] <4.5) & (data['arousal']<4.5)),
    ((data['valence'] <4.5) & (data['arousal'] > 4.5)),
    ((data['valence'] >4.5) & (data['arousal']<4.5)),
    ((data['valence'] >4.5) & (data['arousal'] > 4.5))]
choices = ['sadness', 'pleasure', 'anger', 'joy']

data['valence'] = np.select(conditions, choices, default='black')
data = data.rename(columns={'valence' : 'label'})
data = data.drop(columns='arousal')

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['label']]

x = copia.drop(columns=['label'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment6_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)
    
rf_acc = accuracy_score(yTest, predict)
svm_acc = accuracy_score(yTest, svm_predict)
knn_acc = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_sadness = precision_score(yTest, predict, average="micro", labels=["sadness"])
rf_prec_pleasure = precision_score(yTest, predict, average="micro", labels=["pleasure"])
rf_prec_anger = precision_score(yTest, predict, average="micro", labels=["anger"])
rf_prec_joy = precision_score(yTest, predict, average="micro", labels=["joy"])


svm_prec_sadness = precision_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_prec_pleasure = precision_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_prec_anger = precision_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_prec_joy = precision_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_prec_sadness = precision_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_prec_pleasure = precision_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_prec_anger = precision_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_prec_joy = precision_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import recall_score

rf_rec_sadness = recall_score(yTest, predict, average="micro", labels=["sadness"])
rf_rec_pleasure = recall_score(yTest, predict, average="micro", labels=["pleasure"])
rf_rec_anger = recall_score(yTest, predict, average="micro", labels=["anger"])
rf_rec_joy = recall_score(yTest, predict, average="micro", labels=["joy"])


svm_rec_sadness = recall_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_rec_pleasure = recall_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_rec_anger = recall_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_rec_joy = recall_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_rec_sadness = recall_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_rec_pleasure = recall_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_rec_anger = recall_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_rec_joy = recall_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import f1_score

rf_f1_sadness = f1_score(yTest, predict, average="micro", labels=["sadness"])
rf_f1_pleasure = f1_score(yTest, predict, average="micro", labels=["pleasure"])
rf_f1_anger = f1_score(yTest, predict, average="micro", labels=["anger"])
rf_f1_joy = f1_score(yTest, predict, average="micro", labels=["joy"])


svm_f1_sadness = f1_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_f1_pleasure = f1_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_f1_anger = f1_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_f1_joy = f1_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_f1_sadness = f1_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_f1_pleasure = f1_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_f1_anger = f1_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_f1_joy = f1_score(yTest, knn_predict, average="micro", labels=["joy"])

data_CSV = [subject_id, 6, "STFT", "All bands", "-", "Random Forest", rf_acc, rf_prec_sadness, rf_prec_pleasure, rf_prec_anger, rf_prec_joy, rf_rec_sadness, rf_rec_pleasure, rf_rec_anger, rf_rec_joy, rf_f1_sadness, rf_f1_pleasure, rf_f1_anger, rf_f1_joy], [subject_id, 6, "STFT", "All bands", "-", "SVM", svm_acc, svm_prec_sadness, svm_prec_pleasure, svm_prec_anger, svm_prec_joy, svm_rec_sadness, svm_rec_pleasure, svm_rec_anger, svm_rec_joy, svm_f1_sadness, svm_f1_pleasure, svm_f1_anger, svm_f1_joy], [subject_id, 6, "STFT", "All bands", "-", "kNN", knn_acc, knn_prec_sadness, knn_prec_pleasure, knn_prec_anger, knn_prec_joy, knn_rec_sadness, knn_rec_pleasure, knn_rec_anger, knn_rec_joy, knn_f1_sadness, knn_f1_pleasure, knn_f1_anger, knn_f1_joy]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment6_1.csv", mode="a", header=False, index=False)

f = "data_preprocessed_python\\s22.dat"
subject_id = 22
dictRaw = pd.read_pickle(f)

labels = dictRaw.get('labels')

dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

data = dictRaw.get('data')

df_videos = {}
    #Recorrido de cada una de las columnas de la tabla 3d, y guardado en una una lista de dataframes para cada uno de los videos.
for i in range(40):
    df_videos[i] = pd.DataFrame(data[i])
    df_videos[i] = df_videos[i].transpose()
    df_videos[i].columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature']
    df_videos[i].drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature'],axis = 'columns', inplace=True)

for video in range(0, 40):
    delta_bands = []
    theta_bands = []
    beta_bands = []
    alpha_bands = []
    gamma_bands = []

    for channel in ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']:
        Zxx = do_stft(video, channel)
        eeg_band_fft = make_bands(Zxx)
     #   print(video)
      #  print("\n")
       # print(channel)
        freq = channel_freq(eeg_band_fft)

        delta_bands.append(freq[0])
        theta_bands.append(freq[1])
        alpha_bands.append(freq[2])
        beta_bands.append(freq[3])
        gamma_bands.append(freq[4])

# Banda delta
df = pd.DataFrame(delta_bands)
delta_df = df.transpose()
delta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
delta_df

rows = list()
for _,row in delta_df.iterrows():
    rows += [row]*40
aux_delta = pd.DataFrame(rows).reset_index(drop=True)

# Banda theta
df = pd.DataFrame(theta_bands)
theta_df = df.transpose()
theta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
theta_df

rows = list()
for _,row in theta_df.iterrows():
    rows += [row]*40
aux_theta = pd.DataFrame(rows).reset_index(drop=True)

#Bnada alpha
df = pd.DataFrame(alpha_bands)
alpha_df = df.transpose()
alpha_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
alpha_df

rows = list()
for _,row in alpha_df.iterrows():
    rows += [row]*40
aux_alpha = pd.DataFrame(rows).reset_index(drop=True)

# Banda beta
df = pd.DataFrame(beta_bands)
beta_df = df.transpose()
beta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
beta_df

rows = list()
for _,row in beta_df.iterrows():
    rows += [row]*40
aux_beta = pd.DataFrame(rows).reset_index(drop=True)

# Banda gamma
df = pd.DataFrame(gamma_bands)
gamma_df = df.transpose()
gamma_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
gamma_df

rows = list()
for _,row in gamma_df.iterrows():
    rows += [row]*40
aux_gamma = pd.DataFrame(rows).reset_index(drop=True)

df_labels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])
rows = list()
for _,row in dfLabels.iterrows():
    rows += [row]*64
aux_labels = pd.DataFrame(rows).reset_index(drop=True)

data_delta = aux_delta.merge(aux_labels, left_index=True, right_index=True)
data_theta = aux_theta.merge(aux_labels, left_index=True, right_index=True)
data_alpha = aux_alpha.merge(aux_labels, left_index=True, right_index=True)
data_beta = aux_beta.merge(aux_labels, left_index=True, right_index=True)
data_gamma = aux_gamma.merge(aux_labels, left_index=True, right_index=True)

frames = [aux_delta, aux_theta, aux_alpha, aux_beta, aux_gamma]
aux_all_bands = pd.concat(frames, axis=1)
all_bands_df = aux_all_bands.merge(aux_labels, left_index=True, right_index=True)

data = all_bands_df.drop(['dominance', 'liking'], axis=1)

conditions = [
    ((data['valence'] <4.5) & (data['arousal']<4.5)),
    ((data['valence'] <4.5) & (data['arousal'] > 4.5)),
    ((data['valence'] >4.5) & (data['arousal']<4.5)),
    ((data['valence'] >4.5) & (data['arousal'] > 4.5))]
choices = ['sadness', 'pleasure', 'anger', 'joy']

data['valence'] = np.select(conditions, choices, default='black')
data = data.rename(columns={'valence' : 'label'})
data = data.drop(columns='arousal')

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['label']]

x = copia.drop(columns=['label'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment6_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)
    
rf_acc = accuracy_score(yTest, predict)
svm_acc = accuracy_score(yTest, svm_predict)
knn_acc = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_sadness = precision_score(yTest, predict, average="micro", labels=["sadness"])
rf_prec_pleasure = precision_score(yTest, predict, average="micro", labels=["pleasure"])
rf_prec_anger = precision_score(yTest, predict, average="micro", labels=["anger"])
rf_prec_joy = precision_score(yTest, predict, average="micro", labels=["joy"])


svm_prec_sadness = precision_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_prec_pleasure = precision_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_prec_anger = precision_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_prec_joy = precision_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_prec_sadness = precision_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_prec_pleasure = precision_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_prec_anger = precision_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_prec_joy = precision_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import recall_score

rf_rec_sadness = recall_score(yTest, predict, average="micro", labels=["sadness"])
rf_rec_pleasure = recall_score(yTest, predict, average="micro", labels=["pleasure"])
rf_rec_anger = recall_score(yTest, predict, average="micro", labels=["anger"])
rf_rec_joy = recall_score(yTest, predict, average="micro", labels=["joy"])


svm_rec_sadness = recall_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_rec_pleasure = recall_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_rec_anger = recall_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_rec_joy = recall_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_rec_sadness = recall_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_rec_pleasure = recall_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_rec_anger = recall_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_rec_joy = recall_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import f1_score

rf_f1_sadness = f1_score(yTest, predict, average="micro", labels=["sadness"])
rf_f1_pleasure = f1_score(yTest, predict, average="micro", labels=["pleasure"])
rf_f1_anger = f1_score(yTest, predict, average="micro", labels=["anger"])
rf_f1_joy = f1_score(yTest, predict, average="micro", labels=["joy"])


svm_f1_sadness = f1_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_f1_pleasure = f1_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_f1_anger = f1_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_f1_joy = f1_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_f1_sadness = f1_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_f1_pleasure = f1_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_f1_anger = f1_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_f1_joy = f1_score(yTest, knn_predict, average="micro", labels=["joy"])

data_CSV = [subject_id, 6, "STFT", "All bands", "-", "Random Forest", rf_acc, rf_prec_sadness, rf_prec_pleasure, rf_prec_anger, rf_prec_joy, rf_rec_sadness, rf_rec_pleasure, rf_rec_anger, rf_rec_joy, rf_f1_sadness, rf_f1_pleasure, rf_f1_anger, rf_f1_joy], [subject_id, 6, "STFT", "All bands", "-", "SVM", svm_acc, svm_prec_sadness, svm_prec_pleasure, svm_prec_anger, svm_prec_joy, svm_rec_sadness, svm_rec_pleasure, svm_rec_anger, svm_rec_joy, svm_f1_sadness, svm_f1_pleasure, svm_f1_anger, svm_f1_joy], [subject_id, 6, "STFT", "All bands", "-", "kNN", knn_acc, knn_prec_sadness, knn_prec_pleasure, knn_prec_anger, knn_prec_joy, knn_rec_sadness, knn_rec_pleasure, knn_rec_anger, knn_rec_joy, knn_f1_sadness, knn_f1_pleasure, knn_f1_anger, knn_f1_joy]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment6_1.csv", mode="a", header=False, index=False)

f = "data_preprocessed_python\\s23.dat"
subject_id = 23
dictRaw = pd.read_pickle(f)

labels = dictRaw.get('labels')

dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

data = dictRaw.get('data')

df_videos = {}
    #Recorrido de cada una de las columnas de la tabla 3d, y guardado en una una lista de dataframes para cada uno de los videos.
for i in range(40):
    df_videos[i] = pd.DataFrame(data[i])
    df_videos[i] = df_videos[i].transpose()
    df_videos[i].columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature']
    df_videos[i].drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature'],axis = 'columns', inplace=True)

for video in range(0, 40):
    delta_bands = []
    theta_bands = []
    beta_bands = []
    alpha_bands = []
    gamma_bands = []

    for channel in ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']:
        Zxx = do_stft(video, channel)
        eeg_band_fft = make_bands(Zxx)
     #   print(video)
      #  print("\n")
       # print(channel)
        freq = channel_freq(eeg_band_fft)

        delta_bands.append(freq[0])
        theta_bands.append(freq[1])
        alpha_bands.append(freq[2])
        beta_bands.append(freq[3])
        gamma_bands.append(freq[4])

# Banda delta
df = pd.DataFrame(delta_bands)
delta_df = df.transpose()
delta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
delta_df

rows = list()
for _,row in delta_df.iterrows():
    rows += [row]*40
aux_delta = pd.DataFrame(rows).reset_index(drop=True)

# Banda theta
df = pd.DataFrame(theta_bands)
theta_df = df.transpose()
theta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
theta_df

rows = list()
for _,row in theta_df.iterrows():
    rows += [row]*40
aux_theta = pd.DataFrame(rows).reset_index(drop=True)

#Bnada alpha
df = pd.DataFrame(alpha_bands)
alpha_df = df.transpose()
alpha_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
alpha_df

rows = list()
for _,row in alpha_df.iterrows():
    rows += [row]*40
aux_alpha = pd.DataFrame(rows).reset_index(drop=True)

# Banda beta
df = pd.DataFrame(beta_bands)
beta_df = df.transpose()
beta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
beta_df

rows = list()
for _,row in beta_df.iterrows():
    rows += [row]*40
aux_beta = pd.DataFrame(rows).reset_index(drop=True)

# Banda gamma
df = pd.DataFrame(gamma_bands)
gamma_df = df.transpose()
gamma_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
gamma_df

rows = list()
for _,row in gamma_df.iterrows():
    rows += [row]*40
aux_gamma = pd.DataFrame(rows).reset_index(drop=True)

df_labels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])
rows = list()
for _,row in dfLabels.iterrows():
    rows += [row]*64
aux_labels = pd.DataFrame(rows).reset_index(drop=True)

data_delta = aux_delta.merge(aux_labels, left_index=True, right_index=True)
data_theta = aux_theta.merge(aux_labels, left_index=True, right_index=True)
data_alpha = aux_alpha.merge(aux_labels, left_index=True, right_index=True)
data_beta = aux_beta.merge(aux_labels, left_index=True, right_index=True)
data_gamma = aux_gamma.merge(aux_labels, left_index=True, right_index=True)

frames = [aux_delta, aux_theta, aux_alpha, aux_beta, aux_gamma]
aux_all_bands = pd.concat(frames, axis=1)
all_bands_df = aux_all_bands.merge(aux_labels, left_index=True, right_index=True)

data = all_bands_df.drop(['dominance', 'liking'], axis=1)

conditions = [
    ((data['valence'] <4.5) & (data['arousal']<4.5)),
    ((data['valence'] <4.5) & (data['arousal'] > 4.5)),
    ((data['valence'] >4.5) & (data['arousal']<4.5)),
    ((data['valence'] >4.5) & (data['arousal'] > 4.5))]
choices = ['sadness', 'pleasure', 'anger', 'joy']

data['valence'] = np.select(conditions, choices, default='black')
data = data.rename(columns={'valence' : 'label'})
data = data.drop(columns='arousal')

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['label']]

x = copia.drop(columns=['label'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment6_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)
    
rf_acc = accuracy_score(yTest, predict)
svm_acc = accuracy_score(yTest, svm_predict)
knn_acc = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_sadness = precision_score(yTest, predict, average="micro", labels=["sadness"])
rf_prec_pleasure = precision_score(yTest, predict, average="micro", labels=["pleasure"])
rf_prec_anger = precision_score(yTest, predict, average="micro", labels=["anger"])
rf_prec_joy = precision_score(yTest, predict, average="micro", labels=["joy"])


svm_prec_sadness = precision_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_prec_pleasure = precision_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_prec_anger = precision_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_prec_joy = precision_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_prec_sadness = precision_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_prec_pleasure = precision_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_prec_anger = precision_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_prec_joy = precision_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import recall_score

rf_rec_sadness = recall_score(yTest, predict, average="micro", labels=["sadness"])
rf_rec_pleasure = recall_score(yTest, predict, average="micro", labels=["pleasure"])
rf_rec_anger = recall_score(yTest, predict, average="micro", labels=["anger"])
rf_rec_joy = recall_score(yTest, predict, average="micro", labels=["joy"])


svm_rec_sadness = recall_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_rec_pleasure = recall_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_rec_anger = recall_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_rec_joy = recall_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_rec_sadness = recall_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_rec_pleasure = recall_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_rec_anger = recall_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_rec_joy = recall_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import f1_score

rf_f1_sadness = f1_score(yTest, predict, average="micro", labels=["sadness"])
rf_f1_pleasure = f1_score(yTest, predict, average="micro", labels=["pleasure"])
rf_f1_anger = f1_score(yTest, predict, average="micro", labels=["anger"])
rf_f1_joy = f1_score(yTest, predict, average="micro", labels=["joy"])


svm_f1_sadness = f1_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_f1_pleasure = f1_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_f1_anger = f1_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_f1_joy = f1_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_f1_sadness = f1_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_f1_pleasure = f1_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_f1_anger = f1_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_f1_joy = f1_score(yTest, knn_predict, average="micro", labels=["joy"])

data_CSV = [subject_id, 6, "STFT", "All bands", "-", "Random Forest", rf_acc, rf_prec_sadness, rf_prec_pleasure, rf_prec_anger, rf_prec_joy, rf_rec_sadness, rf_rec_pleasure, rf_rec_anger, rf_rec_joy, rf_f1_sadness, rf_f1_pleasure, rf_f1_anger, rf_f1_joy], [subject_id, 6, "STFT", "All bands", "-", "SVM", svm_acc, svm_prec_sadness, svm_prec_pleasure, svm_prec_anger, svm_prec_joy, svm_rec_sadness, svm_rec_pleasure, svm_rec_anger, svm_rec_joy, svm_f1_sadness, svm_f1_pleasure, svm_f1_anger, svm_f1_joy], [subject_id, 6, "STFT", "All bands", "-", "kNN", knn_acc, knn_prec_sadness, knn_prec_pleasure, knn_prec_anger, knn_prec_joy, knn_rec_sadness, knn_rec_pleasure, knn_rec_anger, knn_rec_joy, knn_f1_sadness, knn_f1_pleasure, knn_f1_anger, knn_f1_joy]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment6_1.csv", mode="a", header=False, index=False)

f = "data_preprocessed_python\\s24.dat"
subject_id = 24
dictRaw = pd.read_pickle(f)

labels = dictRaw.get('labels')

dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

data = dictRaw.get('data')

df_videos = {}
    #Recorrido de cada una de las columnas de la tabla 3d, y guardado en una una lista de dataframes para cada uno de los videos.
for i in range(40):
    df_videos[i] = pd.DataFrame(data[i])
    df_videos[i] = df_videos[i].transpose()
    df_videos[i].columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature']
    df_videos[i].drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature'],axis = 'columns', inplace=True)

for video in range(0, 40):
    delta_bands = []
    theta_bands = []
    beta_bands = []
    alpha_bands = []
    gamma_bands = []

    for channel in ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']:
        Zxx = do_stft(video, channel)
        eeg_band_fft = make_bands(Zxx)
     #   print(video)
      #  print("\n")
       # print(channel)
        freq = channel_freq(eeg_band_fft)

        delta_bands.append(freq[0])
        theta_bands.append(freq[1])
        alpha_bands.append(freq[2])
        beta_bands.append(freq[3])
        gamma_bands.append(freq[4])

# Banda delta
df = pd.DataFrame(delta_bands)
delta_df = df.transpose()
delta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
delta_df

rows = list()
for _,row in delta_df.iterrows():
    rows += [row]*40
aux_delta = pd.DataFrame(rows).reset_index(drop=True)

# Banda theta
df = pd.DataFrame(theta_bands)
theta_df = df.transpose()
theta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
theta_df

rows = list()
for _,row in theta_df.iterrows():
    rows += [row]*40
aux_theta = pd.DataFrame(rows).reset_index(drop=True)

#Bnada alpha
df = pd.DataFrame(alpha_bands)
alpha_df = df.transpose()
alpha_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
alpha_df

rows = list()
for _,row in alpha_df.iterrows():
    rows += [row]*40
aux_alpha = pd.DataFrame(rows).reset_index(drop=True)

# Banda beta
df = pd.DataFrame(beta_bands)
beta_df = df.transpose()
beta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
beta_df

rows = list()
for _,row in beta_df.iterrows():
    rows += [row]*40
aux_beta = pd.DataFrame(rows).reset_index(drop=True)

# Banda gamma
df = pd.DataFrame(gamma_bands)
gamma_df = df.transpose()
gamma_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
gamma_df

rows = list()
for _,row in gamma_df.iterrows():
    rows += [row]*40
aux_gamma = pd.DataFrame(rows).reset_index(drop=True)

df_labels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])
rows = list()
for _,row in dfLabels.iterrows():
    rows += [row]*64
aux_labels = pd.DataFrame(rows).reset_index(drop=True)

data_delta = aux_delta.merge(aux_labels, left_index=True, right_index=True)
data_theta = aux_theta.merge(aux_labels, left_index=True, right_index=True)
data_alpha = aux_alpha.merge(aux_labels, left_index=True, right_index=True)
data_beta = aux_beta.merge(aux_labels, left_index=True, right_index=True)
data_gamma = aux_gamma.merge(aux_labels, left_index=True, right_index=True)

frames = [aux_delta, aux_theta, aux_alpha, aux_beta, aux_gamma]
aux_all_bands = pd.concat(frames, axis=1)
all_bands_df = aux_all_bands.merge(aux_labels, left_index=True, right_index=True)

data = all_bands_df.drop(['dominance', 'liking'], axis=1)

conditions = [
    ((data['valence'] <4.5) & (data['arousal']<4.5)),
    ((data['valence'] <4.5) & (data['arousal'] > 4.5)),
    ((data['valence'] >4.5) & (data['arousal']<4.5)),
    ((data['valence'] >4.5) & (data['arousal'] > 4.5))]
choices = ['sadness', 'pleasure', 'anger', 'joy']

data['valence'] = np.select(conditions, choices, default='black')
data = data.rename(columns={'valence' : 'label'})
data = data.drop(columns='arousal')

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['label']]

x = copia.drop(columns=['label'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment6_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)
    
rf_acc = accuracy_score(yTest, predict)
svm_acc = accuracy_score(yTest, svm_predict)
knn_acc = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_sadness = precision_score(yTest, predict, average="micro", labels=["sadness"])
rf_prec_pleasure = precision_score(yTest, predict, average="micro", labels=["pleasure"])
rf_prec_anger = precision_score(yTest, predict, average="micro", labels=["anger"])
rf_prec_joy = precision_score(yTest, predict, average="micro", labels=["joy"])


svm_prec_sadness = precision_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_prec_pleasure = precision_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_prec_anger = precision_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_prec_joy = precision_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_prec_sadness = precision_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_prec_pleasure = precision_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_prec_anger = precision_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_prec_joy = precision_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import recall_score

rf_rec_sadness = recall_score(yTest, predict, average="micro", labels=["sadness"])
rf_rec_pleasure = recall_score(yTest, predict, average="micro", labels=["pleasure"])
rf_rec_anger = recall_score(yTest, predict, average="micro", labels=["anger"])
rf_rec_joy = recall_score(yTest, predict, average="micro", labels=["joy"])


svm_rec_sadness = recall_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_rec_pleasure = recall_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_rec_anger = recall_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_rec_joy = recall_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_rec_sadness = recall_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_rec_pleasure = recall_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_rec_anger = recall_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_rec_joy = recall_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import f1_score

rf_f1_sadness = f1_score(yTest, predict, average="micro", labels=["sadness"])
rf_f1_pleasure = f1_score(yTest, predict, average="micro", labels=["pleasure"])
rf_f1_anger = f1_score(yTest, predict, average="micro", labels=["anger"])
rf_f1_joy = f1_score(yTest, predict, average="micro", labels=["joy"])


svm_f1_sadness = f1_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_f1_pleasure = f1_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_f1_anger = f1_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_f1_joy = f1_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_f1_sadness = f1_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_f1_pleasure = f1_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_f1_anger = f1_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_f1_joy = f1_score(yTest, knn_predict, average="micro", labels=["joy"])

data_CSV = [subject_id, 6, "STFT", "All bands", "-", "Random Forest", rf_acc, rf_prec_sadness, rf_prec_pleasure, rf_prec_anger, rf_prec_joy, rf_rec_sadness, rf_rec_pleasure, rf_rec_anger, rf_rec_joy, rf_f1_sadness, rf_f1_pleasure, rf_f1_anger, rf_f1_joy], [subject_id, 6, "STFT", "All bands", "-", "SVM", svm_acc, svm_prec_sadness, svm_prec_pleasure, svm_prec_anger, svm_prec_joy, svm_rec_sadness, svm_rec_pleasure, svm_rec_anger, svm_rec_joy, svm_f1_sadness, svm_f1_pleasure, svm_f1_anger, svm_f1_joy], [subject_id, 6, "STFT", "All bands", "-", "kNN", knn_acc, knn_prec_sadness, knn_prec_pleasure, knn_prec_anger, knn_prec_joy, knn_rec_sadness, knn_rec_pleasure, knn_rec_anger, knn_rec_joy, knn_f1_sadness, knn_f1_pleasure, knn_f1_anger, knn_f1_joy]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment6_1.csv", mode="a", header=False, index=False)

f = "data_preprocessed_python\\s25.dat"
subject_id = 25
dictRaw = pd.read_pickle(f)

labels = dictRaw.get('labels')

dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

data = dictRaw.get('data')

df_videos = {}
    #Recorrido de cada una de las columnas de la tabla 3d, y guardado en una una lista de dataframes para cada uno de los videos.
for i in range(40):
    df_videos[i] = pd.DataFrame(data[i])
    df_videos[i] = df_videos[i].transpose()
    df_videos[i].columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature']
    df_videos[i].drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature'],axis = 'columns', inplace=True)

for video in range(0, 40):
    delta_bands = []
    theta_bands = []
    beta_bands = []
    alpha_bands = []
    gamma_bands = []

    for channel in ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']:
        Zxx = do_stft(video, channel)
        eeg_band_fft = make_bands(Zxx)
     #   print(video)
      #  print("\n")
       # print(channel)
        freq = channel_freq(eeg_band_fft)

        delta_bands.append(freq[0])
        theta_bands.append(freq[1])
        alpha_bands.append(freq[2])
        beta_bands.append(freq[3])
        gamma_bands.append(freq[4])

# Banda delta
df = pd.DataFrame(delta_bands)
delta_df = df.transpose()
delta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
delta_df

rows = list()
for _,row in delta_df.iterrows():
    rows += [row]*40
aux_delta = pd.DataFrame(rows).reset_index(drop=True)

# Banda theta
df = pd.DataFrame(theta_bands)
theta_df = df.transpose()
theta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
theta_df

rows = list()
for _,row in theta_df.iterrows():
    rows += [row]*40
aux_theta = pd.DataFrame(rows).reset_index(drop=True)

#Bnada alpha
df = pd.DataFrame(alpha_bands)
alpha_df = df.transpose()
alpha_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
alpha_df

rows = list()
for _,row in alpha_df.iterrows():
    rows += [row]*40
aux_alpha = pd.DataFrame(rows).reset_index(drop=True)

# Banda beta
df = pd.DataFrame(beta_bands)
beta_df = df.transpose()
beta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
beta_df

rows = list()
for _,row in beta_df.iterrows():
    rows += [row]*40
aux_beta = pd.DataFrame(rows).reset_index(drop=True)

# Banda gamma
df = pd.DataFrame(gamma_bands)
gamma_df = df.transpose()
gamma_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
gamma_df

rows = list()
for _,row in gamma_df.iterrows():
    rows += [row]*40
aux_gamma = pd.DataFrame(rows).reset_index(drop=True)

df_labels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])
rows = list()
for _,row in dfLabels.iterrows():
    rows += [row]*64
aux_labels = pd.DataFrame(rows).reset_index(drop=True)

data_delta = aux_delta.merge(aux_labels, left_index=True, right_index=True)
data_theta = aux_theta.merge(aux_labels, left_index=True, right_index=True)
data_alpha = aux_alpha.merge(aux_labels, left_index=True, right_index=True)
data_beta = aux_beta.merge(aux_labels, left_index=True, right_index=True)
data_gamma = aux_gamma.merge(aux_labels, left_index=True, right_index=True)

frames = [aux_delta, aux_theta, aux_alpha, aux_beta, aux_gamma]
aux_all_bands = pd.concat(frames, axis=1)
all_bands_df = aux_all_bands.merge(aux_labels, left_index=True, right_index=True)

data = all_bands_df.drop(['dominance', 'liking'], axis=1)

conditions = [
    ((data['valence'] <4.5) & (data['arousal']<4.5)),
    ((data['valence'] <4.5) & (data['arousal'] > 4.5)),
    ((data['valence'] >4.5) & (data['arousal']<4.5)),
    ((data['valence'] >4.5) & (data['arousal'] > 4.5))]
choices = ['sadness', 'pleasure', 'anger', 'joy']

data['valence'] = np.select(conditions, choices, default='black')
data = data.rename(columns={'valence' : 'label'})
data = data.drop(columns='arousal')

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['label']]

x = copia.drop(columns=['label'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment6_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)
    
rf_acc = accuracy_score(yTest, predict)
svm_acc = accuracy_score(yTest, svm_predict)
knn_acc = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_sadness = precision_score(yTest, predict, average="micro", labels=["sadness"])
rf_prec_pleasure = precision_score(yTest, predict, average="micro", labels=["pleasure"])
rf_prec_anger = precision_score(yTest, predict, average="micro", labels=["anger"])
rf_prec_joy = precision_score(yTest, predict, average="micro", labels=["joy"])


svm_prec_sadness = precision_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_prec_pleasure = precision_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_prec_anger = precision_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_prec_joy = precision_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_prec_sadness = precision_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_prec_pleasure = precision_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_prec_anger = precision_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_prec_joy = precision_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import recall_score

rf_rec_sadness = recall_score(yTest, predict, average="micro", labels=["sadness"])
rf_rec_pleasure = recall_score(yTest, predict, average="micro", labels=["pleasure"])
rf_rec_anger = recall_score(yTest, predict, average="micro", labels=["anger"])
rf_rec_joy = recall_score(yTest, predict, average="micro", labels=["joy"])


svm_rec_sadness = recall_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_rec_pleasure = recall_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_rec_anger = recall_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_rec_joy = recall_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_rec_sadness = recall_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_rec_pleasure = recall_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_rec_anger = recall_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_rec_joy = recall_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import f1_score

rf_f1_sadness = f1_score(yTest, predict, average="micro", labels=["sadness"])
rf_f1_pleasure = f1_score(yTest, predict, average="micro", labels=["pleasure"])
rf_f1_anger = f1_score(yTest, predict, average="micro", labels=["anger"])
rf_f1_joy = f1_score(yTest, predict, average="micro", labels=["joy"])


svm_f1_sadness = f1_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_f1_pleasure = f1_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_f1_anger = f1_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_f1_joy = f1_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_f1_sadness = f1_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_f1_pleasure = f1_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_f1_anger = f1_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_f1_joy = f1_score(yTest, knn_predict, average="micro", labels=["joy"])

data_CSV = [subject_id, 6, "STFT", "All bands", "-", "Random Forest", rf_acc, rf_prec_sadness, rf_prec_pleasure, rf_prec_anger, rf_prec_joy, rf_rec_sadness, rf_rec_pleasure, rf_rec_anger, rf_rec_joy, rf_f1_sadness, rf_f1_pleasure, rf_f1_anger, rf_f1_joy], [subject_id, 6, "STFT", "All bands", "-", "SVM", svm_acc, svm_prec_sadness, svm_prec_pleasure, svm_prec_anger, svm_prec_joy, svm_rec_sadness, svm_rec_pleasure, svm_rec_anger, svm_rec_joy, svm_f1_sadness, svm_f1_pleasure, svm_f1_anger, svm_f1_joy], [subject_id, 6, "STFT", "All bands", "-", "kNN", knn_acc, knn_prec_sadness, knn_prec_pleasure, knn_prec_anger, knn_prec_joy, knn_rec_sadness, knn_rec_pleasure, knn_rec_anger, knn_rec_joy, knn_f1_sadness, knn_f1_pleasure, knn_f1_anger, knn_f1_joy]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment6_1.csv", mode="a", header=False, index=False)

f = "data_preprocessed_python\\s26.dat"
subject_id = 26
dictRaw = pd.read_pickle(f)

labels = dictRaw.get('labels')

dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

data = dictRaw.get('data')

df_videos = {}
    #Recorrido de cada una de las columnas de la tabla 3d, y guardado en una una lista de dataframes para cada uno de los videos.
for i in range(40):
    df_videos[i] = pd.DataFrame(data[i])
    df_videos[i] = df_videos[i].transpose()
    df_videos[i].columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature']
    df_videos[i].drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature'],axis = 'columns', inplace=True)

for video in range(0, 40):
    delta_bands = []
    theta_bands = []
    beta_bands = []
    alpha_bands = []
    gamma_bands = []

    for channel in ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']:
        Zxx = do_stft(video, channel)
        eeg_band_fft = make_bands(Zxx)
     #   print(video)
      #  print("\n")
       # print(channel)
        freq = channel_freq(eeg_band_fft)

        delta_bands.append(freq[0])
        theta_bands.append(freq[1])
        alpha_bands.append(freq[2])
        beta_bands.append(freq[3])
        gamma_bands.append(freq[4])

# Banda delta
df = pd.DataFrame(delta_bands)
delta_df = df.transpose()
delta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
delta_df

rows = list()
for _,row in delta_df.iterrows():
    rows += [row]*40
aux_delta = pd.DataFrame(rows).reset_index(drop=True)

# Banda theta
df = pd.DataFrame(theta_bands)
theta_df = df.transpose()
theta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
theta_df

rows = list()
for _,row in theta_df.iterrows():
    rows += [row]*40
aux_theta = pd.DataFrame(rows).reset_index(drop=True)

#Bnada alpha
df = pd.DataFrame(alpha_bands)
alpha_df = df.transpose()
alpha_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
alpha_df

rows = list()
for _,row in alpha_df.iterrows():
    rows += [row]*40
aux_alpha = pd.DataFrame(rows).reset_index(drop=True)

# Banda beta
df = pd.DataFrame(beta_bands)
beta_df = df.transpose()
beta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
beta_df

rows = list()
for _,row in beta_df.iterrows():
    rows += [row]*40
aux_beta = pd.DataFrame(rows).reset_index(drop=True)

# Banda gamma
df = pd.DataFrame(gamma_bands)
gamma_df = df.transpose()
gamma_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
gamma_df

rows = list()
for _,row in gamma_df.iterrows():
    rows += [row]*40
aux_gamma = pd.DataFrame(rows).reset_index(drop=True)

df_labels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])
rows = list()
for _,row in dfLabels.iterrows():
    rows += [row]*64
aux_labels = pd.DataFrame(rows).reset_index(drop=True)

data_delta = aux_delta.merge(aux_labels, left_index=True, right_index=True)
data_theta = aux_theta.merge(aux_labels, left_index=True, right_index=True)
data_alpha = aux_alpha.merge(aux_labels, left_index=True, right_index=True)
data_beta = aux_beta.merge(aux_labels, left_index=True, right_index=True)
data_gamma = aux_gamma.merge(aux_labels, left_index=True, right_index=True)

frames = [aux_delta, aux_theta, aux_alpha, aux_beta, aux_gamma]
aux_all_bands = pd.concat(frames, axis=1)
all_bands_df = aux_all_bands.merge(aux_labels, left_index=True, right_index=True)

data = all_bands_df.drop(['dominance', 'liking'], axis=1)

conditions = [
    ((data['valence'] <4.5) & (data['arousal']<4.5)),
    ((data['valence'] <4.5) & (data['arousal'] > 4.5)),
    ((data['valence'] >4.5) & (data['arousal']<4.5)),
    ((data['valence'] >4.5) & (data['arousal'] > 4.5))]
choices = ['sadness', 'pleasure', 'anger', 'joy']

data['valence'] = np.select(conditions, choices, default='black')
data = data.rename(columns={'valence' : 'label'})
data = data.drop(columns='arousal')

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['label']]

x = copia.drop(columns=['label'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment6_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)
    
rf_acc = accuracy_score(yTest, predict)
svm_acc = accuracy_score(yTest, svm_predict)
knn_acc = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_sadness = precision_score(yTest, predict, average="micro", labels=["sadness"])
rf_prec_pleasure = precision_score(yTest, predict, average="micro", labels=["pleasure"])
rf_prec_anger = precision_score(yTest, predict, average="micro", labels=["anger"])
rf_prec_joy = precision_score(yTest, predict, average="micro", labels=["joy"])


svm_prec_sadness = precision_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_prec_pleasure = precision_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_prec_anger = precision_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_prec_joy = precision_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_prec_sadness = precision_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_prec_pleasure = precision_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_prec_anger = precision_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_prec_joy = precision_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import recall_score

rf_rec_sadness = recall_score(yTest, predict, average="micro", labels=["sadness"])
rf_rec_pleasure = recall_score(yTest, predict, average="micro", labels=["pleasure"])
rf_rec_anger = recall_score(yTest, predict, average="micro", labels=["anger"])
rf_rec_joy = recall_score(yTest, predict, average="micro", labels=["joy"])


svm_rec_sadness = recall_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_rec_pleasure = recall_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_rec_anger = recall_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_rec_joy = recall_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_rec_sadness = recall_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_rec_pleasure = recall_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_rec_anger = recall_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_rec_joy = recall_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import f1_score

rf_f1_sadness = f1_score(yTest, predict, average="micro", labels=["sadness"])
rf_f1_pleasure = f1_score(yTest, predict, average="micro", labels=["pleasure"])
rf_f1_anger = f1_score(yTest, predict, average="micro", labels=["anger"])
rf_f1_joy = f1_score(yTest, predict, average="micro", labels=["joy"])


svm_f1_sadness = f1_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_f1_pleasure = f1_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_f1_anger = f1_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_f1_joy = f1_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_f1_sadness = f1_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_f1_pleasure = f1_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_f1_anger = f1_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_f1_joy = f1_score(yTest, knn_predict, average="micro", labels=["joy"])

data_CSV = [subject_id, 6, "STFT", "All bands", "-", "Random Forest", rf_acc, rf_prec_sadness, rf_prec_pleasure, rf_prec_anger, rf_prec_joy, rf_rec_sadness, rf_rec_pleasure, rf_rec_anger, rf_rec_joy, rf_f1_sadness, rf_f1_pleasure, rf_f1_anger, rf_f1_joy], [subject_id, 6, "STFT", "All bands", "-", "SVM", svm_acc, svm_prec_sadness, svm_prec_pleasure, svm_prec_anger, svm_prec_joy, svm_rec_sadness, svm_rec_pleasure, svm_rec_anger, svm_rec_joy, svm_f1_sadness, svm_f1_pleasure, svm_f1_anger, svm_f1_joy], [subject_id, 6, "STFT", "All bands", "-", "kNN", knn_acc, knn_prec_sadness, knn_prec_pleasure, knn_prec_anger, knn_prec_joy, knn_rec_sadness, knn_rec_pleasure, knn_rec_anger, knn_rec_joy, knn_f1_sadness, knn_f1_pleasure, knn_f1_anger, knn_f1_joy]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment6_1.csv", mode="a", header=False, index=False)

f = "data_preprocessed_python\\s27.dat"
subject_id = 27
dictRaw = pd.read_pickle(f)

labels = dictRaw.get('labels')

dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

data = dictRaw.get('data')

df_videos = {}
    #Recorrido de cada una de las columnas de la tabla 3d, y guardado en una una lista de dataframes para cada uno de los videos.
for i in range(40):
    df_videos[i] = pd.DataFrame(data[i])
    df_videos[i] = df_videos[i].transpose()
    df_videos[i].columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature']
    df_videos[i].drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature'],axis = 'columns', inplace=True)

for video in range(0, 40):
    delta_bands = []
    theta_bands = []
    beta_bands = []
    alpha_bands = []
    gamma_bands = []

    for channel in ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']:
        Zxx = do_stft(video, channel)
        eeg_band_fft = make_bands(Zxx)
     #   print(video)
      #  print("\n")
       # print(channel)
        freq = channel_freq(eeg_band_fft)

        delta_bands.append(freq[0])
        theta_bands.append(freq[1])
        alpha_bands.append(freq[2])
        beta_bands.append(freq[3])
        gamma_bands.append(freq[4])

# Banda delta
df = pd.DataFrame(delta_bands)
delta_df = df.transpose()
delta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
delta_df

rows = list()
for _,row in delta_df.iterrows():
    rows += [row]*40
aux_delta = pd.DataFrame(rows).reset_index(drop=True)

# Banda theta
df = pd.DataFrame(theta_bands)
theta_df = df.transpose()
theta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
theta_df

rows = list()
for _,row in theta_df.iterrows():
    rows += [row]*40
aux_theta = pd.DataFrame(rows).reset_index(drop=True)

#Bnada alpha
df = pd.DataFrame(alpha_bands)
alpha_df = df.transpose()
alpha_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
alpha_df

rows = list()
for _,row in alpha_df.iterrows():
    rows += [row]*40
aux_alpha = pd.DataFrame(rows).reset_index(drop=True)

# Banda beta
df = pd.DataFrame(beta_bands)
beta_df = df.transpose()
beta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
beta_df

rows = list()
for _,row in beta_df.iterrows():
    rows += [row]*40
aux_beta = pd.DataFrame(rows).reset_index(drop=True)

# Banda gamma
df = pd.DataFrame(gamma_bands)
gamma_df = df.transpose()
gamma_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
gamma_df

rows = list()
for _,row in gamma_df.iterrows():
    rows += [row]*40
aux_gamma = pd.DataFrame(rows).reset_index(drop=True)

df_labels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])
rows = list()
for _,row in dfLabels.iterrows():
    rows += [row]*64
aux_labels = pd.DataFrame(rows).reset_index(drop=True)

data_delta = aux_delta.merge(aux_labels, left_index=True, right_index=True)
data_theta = aux_theta.merge(aux_labels, left_index=True, right_index=True)
data_alpha = aux_alpha.merge(aux_labels, left_index=True, right_index=True)
data_beta = aux_beta.merge(aux_labels, left_index=True, right_index=True)
data_gamma = aux_gamma.merge(aux_labels, left_index=True, right_index=True)

frames = [aux_delta, aux_theta, aux_alpha, aux_beta, aux_gamma]
aux_all_bands = pd.concat(frames, axis=1)
all_bands_df = aux_all_bands.merge(aux_labels, left_index=True, right_index=True)

data = all_bands_df.drop(['dominance', 'liking'], axis=1)

conditions = [
    ((data['valence'] <4.5) & (data['arousal']<4.5)),
    ((data['valence'] <4.5) & (data['arousal'] > 4.5)),
    ((data['valence'] >4.5) & (data['arousal']<4.5)),
    ((data['valence'] >4.5) & (data['arousal'] > 4.5))]
choices = ['sadness', 'pleasure', 'anger', 'joy']

data['valence'] = np.select(conditions, choices, default='black')
data = data.rename(columns={'valence' : 'label'})
data = data.drop(columns='arousal')

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['label']]

x = copia.drop(columns=['label'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment6_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)
    
rf_acc = accuracy_score(yTest, predict)
svm_acc = accuracy_score(yTest, svm_predict)
knn_acc = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_sadness = precision_score(yTest, predict, average="micro", labels=["sadness"])
rf_prec_pleasure = precision_score(yTest, predict, average="micro", labels=["pleasure"])
rf_prec_anger = precision_score(yTest, predict, average="micro", labels=["anger"])
rf_prec_joy = precision_score(yTest, predict, average="micro", labels=["joy"])


svm_prec_sadness = precision_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_prec_pleasure = precision_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_prec_anger = precision_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_prec_joy = precision_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_prec_sadness = precision_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_prec_pleasure = precision_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_prec_anger = precision_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_prec_joy = precision_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import recall_score

rf_rec_sadness = recall_score(yTest, predict, average="micro", labels=["sadness"])
rf_rec_pleasure = recall_score(yTest, predict, average="micro", labels=["pleasure"])
rf_rec_anger = recall_score(yTest, predict, average="micro", labels=["anger"])
rf_rec_joy = recall_score(yTest, predict, average="micro", labels=["joy"])


svm_rec_sadness = recall_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_rec_pleasure = recall_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_rec_anger = recall_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_rec_joy = recall_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_rec_sadness = recall_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_rec_pleasure = recall_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_rec_anger = recall_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_rec_joy = recall_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import f1_score

rf_f1_sadness = f1_score(yTest, predict, average="micro", labels=["sadness"])
rf_f1_pleasure = f1_score(yTest, predict, average="micro", labels=["pleasure"])
rf_f1_anger = f1_score(yTest, predict, average="micro", labels=["anger"])
rf_f1_joy = f1_score(yTest, predict, average="micro", labels=["joy"])


svm_f1_sadness = f1_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_f1_pleasure = f1_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_f1_anger = f1_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_f1_joy = f1_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_f1_sadness = f1_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_f1_pleasure = f1_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_f1_anger = f1_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_f1_joy = f1_score(yTest, knn_predict, average="micro", labels=["joy"])

data_CSV = [subject_id, 6, "STFT", "All bands", "-", "Random Forest", rf_acc, rf_prec_sadness, rf_prec_pleasure, rf_prec_anger, rf_prec_joy, rf_rec_sadness, rf_rec_pleasure, rf_rec_anger, rf_rec_joy, rf_f1_sadness, rf_f1_pleasure, rf_f1_anger, rf_f1_joy], [subject_id, 6, "STFT", "All bands", "-", "SVM", svm_acc, svm_prec_sadness, svm_prec_pleasure, svm_prec_anger, svm_prec_joy, svm_rec_sadness, svm_rec_pleasure, svm_rec_anger, svm_rec_joy, svm_f1_sadness, svm_f1_pleasure, svm_f1_anger, svm_f1_joy], [subject_id, 6, "STFT", "All bands", "-", "kNN", knn_acc, knn_prec_sadness, knn_prec_pleasure, knn_prec_anger, knn_prec_joy, knn_rec_sadness, knn_rec_pleasure, knn_rec_anger, knn_rec_joy, knn_f1_sadness, knn_f1_pleasure, knn_f1_anger, knn_f1_joy]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment6_1.csv", mode="a", header=False, index=False)

f = "data_preprocessed_python\\s28.dat"
subject_id = 28
dictRaw = pd.read_pickle(f)

labels = dictRaw.get('labels')

dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

data = dictRaw.get('data')

df_videos = {}
    #Recorrido de cada una de las columnas de la tabla 3d, y guardado en una una lista de dataframes para cada uno de los videos.
for i in range(40):
    df_videos[i] = pd.DataFrame(data[i])
    df_videos[i] = df_videos[i].transpose()
    df_videos[i].columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature']
    df_videos[i].drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature'],axis = 'columns', inplace=True)

for video in range(0, 40):
    delta_bands = []
    theta_bands = []
    beta_bands = []
    alpha_bands = []
    gamma_bands = []

    for channel in ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']:
        Zxx = do_stft(video, channel)
        eeg_band_fft = make_bands(Zxx)
     #   print(video)
      #  print("\n")
       # print(channel)
        freq = channel_freq(eeg_band_fft)

        delta_bands.append(freq[0])
        theta_bands.append(freq[1])
        alpha_bands.append(freq[2])
        beta_bands.append(freq[3])
        gamma_bands.append(freq[4])

# Banda delta
df = pd.DataFrame(delta_bands)
delta_df = df.transpose()
delta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
delta_df

rows = list()
for _,row in delta_df.iterrows():
    rows += [row]*40
aux_delta = pd.DataFrame(rows).reset_index(drop=True)

# Banda theta
df = pd.DataFrame(theta_bands)
theta_df = df.transpose()
theta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
theta_df

rows = list()
for _,row in theta_df.iterrows():
    rows += [row]*40
aux_theta = pd.DataFrame(rows).reset_index(drop=True)

#Bnada alpha
df = pd.DataFrame(alpha_bands)
alpha_df = df.transpose()
alpha_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
alpha_df

rows = list()
for _,row in alpha_df.iterrows():
    rows += [row]*40
aux_alpha = pd.DataFrame(rows).reset_index(drop=True)

# Banda beta
df = pd.DataFrame(beta_bands)
beta_df = df.transpose()
beta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
beta_df

rows = list()
for _,row in beta_df.iterrows():
    rows += [row]*40
aux_beta = pd.DataFrame(rows).reset_index(drop=True)

# Banda gamma
df = pd.DataFrame(gamma_bands)
gamma_df = df.transpose()
gamma_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
gamma_df

rows = list()
for _,row in gamma_df.iterrows():
    rows += [row]*40
aux_gamma = pd.DataFrame(rows).reset_index(drop=True)

df_labels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])
rows = list()
for _,row in dfLabels.iterrows():
    rows += [row]*64
aux_labels = pd.DataFrame(rows).reset_index(drop=True)

data_delta = aux_delta.merge(aux_labels, left_index=True, right_index=True)
data_theta = aux_theta.merge(aux_labels, left_index=True, right_index=True)
data_alpha = aux_alpha.merge(aux_labels, left_index=True, right_index=True)
data_beta = aux_beta.merge(aux_labels, left_index=True, right_index=True)
data_gamma = aux_gamma.merge(aux_labels, left_index=True, right_index=True)

frames = [aux_delta, aux_theta, aux_alpha, aux_beta, aux_gamma]
aux_all_bands = pd.concat(frames, axis=1)
all_bands_df = aux_all_bands.merge(aux_labels, left_index=True, right_index=True)

data = all_bands_df.drop(['dominance', 'liking'], axis=1)

conditions = [
    ((data['valence'] <4.5) & (data['arousal']<4.5)),
    ((data['valence'] <4.5) & (data['arousal'] > 4.5)),
    ((data['valence'] >4.5) & (data['arousal']<4.5)),
    ((data['valence'] >4.5) & (data['arousal'] > 4.5))]
choices = ['sadness', 'pleasure', 'anger', 'joy']

data['valence'] = np.select(conditions, choices, default='black')
data = data.rename(columns={'valence' : 'label'})
data = data.drop(columns='arousal')

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['label']]

x = copia.drop(columns=['label'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment6_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)
    
rf_acc = accuracy_score(yTest, predict)
svm_acc = accuracy_score(yTest, svm_predict)
knn_acc = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_sadness = precision_score(yTest, predict, average="micro", labels=["sadness"])
rf_prec_pleasure = precision_score(yTest, predict, average="micro", labels=["pleasure"])
rf_prec_anger = precision_score(yTest, predict, average="micro", labels=["anger"])
rf_prec_joy = precision_score(yTest, predict, average="micro", labels=["joy"])


svm_prec_sadness = precision_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_prec_pleasure = precision_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_prec_anger = precision_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_prec_joy = precision_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_prec_sadness = precision_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_prec_pleasure = precision_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_prec_anger = precision_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_prec_joy = precision_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import recall_score

rf_rec_sadness = recall_score(yTest, predict, average="micro", labels=["sadness"])
rf_rec_pleasure = recall_score(yTest, predict, average="micro", labels=["pleasure"])
rf_rec_anger = recall_score(yTest, predict, average="micro", labels=["anger"])
rf_rec_joy = recall_score(yTest, predict, average="micro", labels=["joy"])


svm_rec_sadness = recall_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_rec_pleasure = recall_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_rec_anger = recall_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_rec_joy = recall_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_rec_sadness = recall_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_rec_pleasure = recall_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_rec_anger = recall_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_rec_joy = recall_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import f1_score

rf_f1_sadness = f1_score(yTest, predict, average="micro", labels=["sadness"])
rf_f1_pleasure = f1_score(yTest, predict, average="micro", labels=["pleasure"])
rf_f1_anger = f1_score(yTest, predict, average="micro", labels=["anger"])
rf_f1_joy = f1_score(yTest, predict, average="micro", labels=["joy"])


svm_f1_sadness = f1_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_f1_pleasure = f1_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_f1_anger = f1_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_f1_joy = f1_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_f1_sadness = f1_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_f1_pleasure = f1_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_f1_anger = f1_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_f1_joy = f1_score(yTest, knn_predict, average="micro", labels=["joy"])

data_CSV = [subject_id, 6, "STFT", "All bands", "-", "Random Forest", rf_acc, rf_prec_sadness, rf_prec_pleasure, rf_prec_anger, rf_prec_joy, rf_rec_sadness, rf_rec_pleasure, rf_rec_anger, rf_rec_joy, rf_f1_sadness, rf_f1_pleasure, rf_f1_anger, rf_f1_joy], [subject_id, 6, "STFT", "All bands", "-", "SVM", svm_acc, svm_prec_sadness, svm_prec_pleasure, svm_prec_anger, svm_prec_joy, svm_rec_sadness, svm_rec_pleasure, svm_rec_anger, svm_rec_joy, svm_f1_sadness, svm_f1_pleasure, svm_f1_anger, svm_f1_joy], [subject_id, 6, "STFT", "All bands", "-", "kNN", knn_acc, knn_prec_sadness, knn_prec_pleasure, knn_prec_anger, knn_prec_joy, knn_rec_sadness, knn_rec_pleasure, knn_rec_anger, knn_rec_joy, knn_f1_sadness, knn_f1_pleasure, knn_f1_anger, knn_f1_joy]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment6_1.csv", mode="a", header=False, index=False)

f = "data_preprocessed_python\\s29.dat"
subject_id = 29
dictRaw = pd.read_pickle(f)

labels = dictRaw.get('labels')

dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

data = dictRaw.get('data')

df_videos = {}
    #Recorrido de cada una de las columnas de la tabla 3d, y guardado en una una lista de dataframes para cada uno de los videos.
for i in range(40):
    df_videos[i] = pd.DataFrame(data[i])
    df_videos[i] = df_videos[i].transpose()
    df_videos[i].columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature']
    df_videos[i].drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature'],axis = 'columns', inplace=True)

for video in range(0, 40):
    delta_bands = []
    theta_bands = []
    beta_bands = []
    alpha_bands = []
    gamma_bands = []

    for channel in ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']:
        Zxx = do_stft(video, channel)
        eeg_band_fft = make_bands(Zxx)
     #   print(video)
      #  print("\n")
       # print(channel)
        freq = channel_freq(eeg_band_fft)

        delta_bands.append(freq[0])
        theta_bands.append(freq[1])
        alpha_bands.append(freq[2])
        beta_bands.append(freq[3])
        gamma_bands.append(freq[4])

# Banda delta
df = pd.DataFrame(delta_bands)
delta_df = df.transpose()
delta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
delta_df

rows = list()
for _,row in delta_df.iterrows():
    rows += [row]*40
aux_delta = pd.DataFrame(rows).reset_index(drop=True)

# Banda theta
df = pd.DataFrame(theta_bands)
theta_df = df.transpose()
theta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
theta_df

rows = list()
for _,row in theta_df.iterrows():
    rows += [row]*40
aux_theta = pd.DataFrame(rows).reset_index(drop=True)

#Bnada alpha
df = pd.DataFrame(alpha_bands)
alpha_df = df.transpose()
alpha_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
alpha_df

rows = list()
for _,row in alpha_df.iterrows():
    rows += [row]*40
aux_alpha = pd.DataFrame(rows).reset_index(drop=True)

# Banda beta
df = pd.DataFrame(beta_bands)
beta_df = df.transpose()
beta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
beta_df

rows = list()
for _,row in beta_df.iterrows():
    rows += [row]*40
aux_beta = pd.DataFrame(rows).reset_index(drop=True)

# Banda gamma
df = pd.DataFrame(gamma_bands)
gamma_df = df.transpose()
gamma_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
gamma_df

rows = list()
for _,row in gamma_df.iterrows():
    rows += [row]*40
aux_gamma = pd.DataFrame(rows).reset_index(drop=True)

df_labels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])
rows = list()
for _,row in dfLabels.iterrows():
    rows += [row]*64
aux_labels = pd.DataFrame(rows).reset_index(drop=True)

data_delta = aux_delta.merge(aux_labels, left_index=True, right_index=True)
data_theta = aux_theta.merge(aux_labels, left_index=True, right_index=True)
data_alpha = aux_alpha.merge(aux_labels, left_index=True, right_index=True)
data_beta = aux_beta.merge(aux_labels, left_index=True, right_index=True)
data_gamma = aux_gamma.merge(aux_labels, left_index=True, right_index=True)

frames = [aux_delta, aux_theta, aux_alpha, aux_beta, aux_gamma]
aux_all_bands = pd.concat(frames, axis=1)
all_bands_df = aux_all_bands.merge(aux_labels, left_index=True, right_index=True)

data = all_bands_df.drop(['dominance', 'liking'], axis=1)

conditions = [
    ((data['valence'] <4.5) & (data['arousal']<4.5)),
    ((data['valence'] <4.5) & (data['arousal'] > 4.5)),
    ((data['valence'] >4.5) & (data['arousal']<4.5)),
    ((data['valence'] >4.5) & (data['arousal'] > 4.5))]
choices = ['sadness', 'pleasure', 'anger', 'joy']

data['valence'] = np.select(conditions, choices, default='black')
data = data.rename(columns={'valence' : 'label'})
data = data.drop(columns='arousal')

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['label']]

x = copia.drop(columns=['label'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment6_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)
    
rf_acc = accuracy_score(yTest, predict)
svm_acc = accuracy_score(yTest, svm_predict)
knn_acc = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_sadness = precision_score(yTest, predict, average="micro", labels=["sadness"])
rf_prec_pleasure = precision_score(yTest, predict, average="micro", labels=["pleasure"])
rf_prec_anger = precision_score(yTest, predict, average="micro", labels=["anger"])
rf_prec_joy = precision_score(yTest, predict, average="micro", labels=["joy"])


svm_prec_sadness = precision_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_prec_pleasure = precision_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_prec_anger = precision_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_prec_joy = precision_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_prec_sadness = precision_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_prec_pleasure = precision_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_prec_anger = precision_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_prec_joy = precision_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import recall_score

rf_rec_sadness = recall_score(yTest, predict, average="micro", labels=["sadness"])
rf_rec_pleasure = recall_score(yTest, predict, average="micro", labels=["pleasure"])
rf_rec_anger = recall_score(yTest, predict, average="micro", labels=["anger"])
rf_rec_joy = recall_score(yTest, predict, average="micro", labels=["joy"])


svm_rec_sadness = recall_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_rec_pleasure = recall_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_rec_anger = recall_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_rec_joy = recall_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_rec_sadness = recall_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_rec_pleasure = recall_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_rec_anger = recall_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_rec_joy = recall_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import f1_score

rf_f1_sadness = f1_score(yTest, predict, average="micro", labels=["sadness"])
rf_f1_pleasure = f1_score(yTest, predict, average="micro", labels=["pleasure"])
rf_f1_anger = f1_score(yTest, predict, average="micro", labels=["anger"])
rf_f1_joy = f1_score(yTest, predict, average="micro", labels=["joy"])


svm_f1_sadness = f1_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_f1_pleasure = f1_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_f1_anger = f1_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_f1_joy = f1_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_f1_sadness = f1_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_f1_pleasure = f1_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_f1_anger = f1_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_f1_joy = f1_score(yTest, knn_predict, average="micro", labels=["joy"])

data_CSV = [subject_id, 6, "STFT", "All bands", "-", "Random Forest", rf_acc, rf_prec_sadness, rf_prec_pleasure, rf_prec_anger, rf_prec_joy, rf_rec_sadness, rf_rec_pleasure, rf_rec_anger, rf_rec_joy, rf_f1_sadness, rf_f1_pleasure, rf_f1_anger, rf_f1_joy], [subject_id, 6, "STFT", "All bands", "-", "SVM", svm_acc, svm_prec_sadness, svm_prec_pleasure, svm_prec_anger, svm_prec_joy, svm_rec_sadness, svm_rec_pleasure, svm_rec_anger, svm_rec_joy, svm_f1_sadness, svm_f1_pleasure, svm_f1_anger, svm_f1_joy], [subject_id, 6, "STFT", "All bands", "-", "kNN", knn_acc, knn_prec_sadness, knn_prec_pleasure, knn_prec_anger, knn_prec_joy, knn_rec_sadness, knn_rec_pleasure, knn_rec_anger, knn_rec_joy, knn_f1_sadness, knn_f1_pleasure, knn_f1_anger, knn_f1_joy]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment6_1.csv", mode="a", header=False, index=False)

f = "data_preprocessed_python\\s30.dat"
subject_id = 30
dictRaw = pd.read_pickle(f)

labels = dictRaw.get('labels')

dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

data = dictRaw.get('data')

df_videos = {}
    #Recorrido de cada una de las columnas de la tabla 3d, y guardado en una una lista de dataframes para cada uno de los videos.
for i in range(40):
    df_videos[i] = pd.DataFrame(data[i])
    df_videos[i] = df_videos[i].transpose()
    df_videos[i].columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature']
    df_videos[i].drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature'],axis = 'columns', inplace=True)

for video in range(0, 40):
    delta_bands = []
    theta_bands = []
    beta_bands = []
    alpha_bands = []
    gamma_bands = []

    for channel in ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']:
        Zxx = do_stft(video, channel)
        eeg_band_fft = make_bands(Zxx)
     #   print(video)
      #  print("\n")
       # print(channel)
        freq = channel_freq(eeg_band_fft)

        delta_bands.append(freq[0])
        theta_bands.append(freq[1])
        alpha_bands.append(freq[2])
        beta_bands.append(freq[3])
        gamma_bands.append(freq[4])

# Banda delta
df = pd.DataFrame(delta_bands)
delta_df = df.transpose()
delta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
delta_df

rows = list()
for _,row in delta_df.iterrows():
    rows += [row]*40
aux_delta = pd.DataFrame(rows).reset_index(drop=True)

# Banda theta
df = pd.DataFrame(theta_bands)
theta_df = df.transpose()
theta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
theta_df

rows = list()
for _,row in theta_df.iterrows():
    rows += [row]*40
aux_theta = pd.DataFrame(rows).reset_index(drop=True)

#Bnada alpha
df = pd.DataFrame(alpha_bands)
alpha_df = df.transpose()
alpha_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
alpha_df

rows = list()
for _,row in alpha_df.iterrows():
    rows += [row]*40
aux_alpha = pd.DataFrame(rows).reset_index(drop=True)

# Banda beta
df = pd.DataFrame(beta_bands)
beta_df = df.transpose()
beta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
beta_df

rows = list()
for _,row in beta_df.iterrows():
    rows += [row]*40
aux_beta = pd.DataFrame(rows).reset_index(drop=True)

# Banda gamma
df = pd.DataFrame(gamma_bands)
gamma_df = df.transpose()
gamma_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
gamma_df

rows = list()
for _,row in gamma_df.iterrows():
    rows += [row]*40
aux_gamma = pd.DataFrame(rows).reset_index(drop=True)

df_labels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])
rows = list()
for _,row in dfLabels.iterrows():
    rows += [row]*64
aux_labels = pd.DataFrame(rows).reset_index(drop=True)

data_delta = aux_delta.merge(aux_labels, left_index=True, right_index=True)
data_theta = aux_theta.merge(aux_labels, left_index=True, right_index=True)
data_alpha = aux_alpha.merge(aux_labels, left_index=True, right_index=True)
data_beta = aux_beta.merge(aux_labels, left_index=True, right_index=True)
data_gamma = aux_gamma.merge(aux_labels, left_index=True, right_index=True)

frames = [aux_delta, aux_theta, aux_alpha, aux_beta, aux_gamma]
aux_all_bands = pd.concat(frames, axis=1)
all_bands_df = aux_all_bands.merge(aux_labels, left_index=True, right_index=True)

data = all_bands_df.drop(['dominance', 'liking'], axis=1)

conditions = [
    ((data['valence'] <4.5) & (data['arousal']<4.5)),
    ((data['valence'] <4.5) & (data['arousal'] > 4.5)),
    ((data['valence'] >4.5) & (data['arousal']<4.5)),
    ((data['valence'] >4.5) & (data['arousal'] > 4.5))]
choices = ['sadness', 'pleasure', 'anger', 'joy']

data['valence'] = np.select(conditions, choices, default='black')
data = data.rename(columns={'valence' : 'label'})
data = data.drop(columns='arousal')

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['label']]

x = copia.drop(columns=['label'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment6_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)
    
rf_acc = accuracy_score(yTest, predict)
svm_acc = accuracy_score(yTest, svm_predict)
knn_acc = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_sadness = precision_score(yTest, predict, average="micro", labels=["sadness"])
rf_prec_pleasure = precision_score(yTest, predict, average="micro", labels=["pleasure"])
rf_prec_anger = precision_score(yTest, predict, average="micro", labels=["anger"])
rf_prec_joy = precision_score(yTest, predict, average="micro", labels=["joy"])


svm_prec_sadness = precision_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_prec_pleasure = precision_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_prec_anger = precision_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_prec_joy = precision_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_prec_sadness = precision_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_prec_pleasure = precision_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_prec_anger = precision_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_prec_joy = precision_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import recall_score

rf_rec_sadness = recall_score(yTest, predict, average="micro", labels=["sadness"])
rf_rec_pleasure = recall_score(yTest, predict, average="micro", labels=["pleasure"])
rf_rec_anger = recall_score(yTest, predict, average="micro", labels=["anger"])
rf_rec_joy = recall_score(yTest, predict, average="micro", labels=["joy"])


svm_rec_sadness = recall_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_rec_pleasure = recall_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_rec_anger = recall_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_rec_joy = recall_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_rec_sadness = recall_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_rec_pleasure = recall_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_rec_anger = recall_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_rec_joy = recall_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import f1_score

rf_f1_sadness = f1_score(yTest, predict, average="micro", labels=["sadness"])
rf_f1_pleasure = f1_score(yTest, predict, average="micro", labels=["pleasure"])
rf_f1_anger = f1_score(yTest, predict, average="micro", labels=["anger"])
rf_f1_joy = f1_score(yTest, predict, average="micro", labels=["joy"])


svm_f1_sadness = f1_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_f1_pleasure = f1_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_f1_anger = f1_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_f1_joy = f1_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_f1_sadness = f1_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_f1_pleasure = f1_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_f1_anger = f1_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_f1_joy = f1_score(yTest, knn_predict, average="micro", labels=["joy"])

data_CSV = [subject_id, 6, "STFT", "All bands", "-", "Random Forest", rf_acc, rf_prec_sadness, rf_prec_pleasure, rf_prec_anger, rf_prec_joy, rf_rec_sadness, rf_rec_pleasure, rf_rec_anger, rf_rec_joy, rf_f1_sadness, rf_f1_pleasure, rf_f1_anger, rf_f1_joy], [subject_id, 6, "STFT", "All bands", "-", "SVM", svm_acc, svm_prec_sadness, svm_prec_pleasure, svm_prec_anger, svm_prec_joy, svm_rec_sadness, svm_rec_pleasure, svm_rec_anger, svm_rec_joy, svm_f1_sadness, svm_f1_pleasure, svm_f1_anger, svm_f1_joy], [subject_id, 6, "STFT", "All bands", "-", "kNN", knn_acc, knn_prec_sadness, knn_prec_pleasure, knn_prec_anger, knn_prec_joy, knn_rec_sadness, knn_rec_pleasure, knn_rec_anger, knn_rec_joy, knn_f1_sadness, knn_f1_pleasure, knn_f1_anger, knn_f1_joy]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment6_1.csv", mode="a", header=False, index=False)

f = "data_preprocessed_python\\s31.dat"
subject_id = 31
dictRaw = pd.read_pickle(f)

labels = dictRaw.get('labels')

dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

data = dictRaw.get('data')

df_videos = {}
    #Recorrido de cada una de las columnas de la tabla 3d, y guardado en una una lista de dataframes para cada uno de los videos.
for i in range(40):
    df_videos[i] = pd.DataFrame(data[i])
    df_videos[i] = df_videos[i].transpose()
    df_videos[i].columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature']
    df_videos[i].drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature'],axis = 'columns', inplace=True)

for video in range(0, 40):
    delta_bands = []
    theta_bands = []
    beta_bands = []
    alpha_bands = []
    gamma_bands = []

    for channel in ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']:
        Zxx = do_stft(video, channel)
        eeg_band_fft = make_bands(Zxx)
     #   print(video)
      #  print("\n")
       # print(channel)
        freq = channel_freq(eeg_band_fft)

        delta_bands.append(freq[0])
        theta_bands.append(freq[1])
        alpha_bands.append(freq[2])
        beta_bands.append(freq[3])
        gamma_bands.append(freq[4])

# Banda delta
df = pd.DataFrame(delta_bands)
delta_df = df.transpose()
delta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
delta_df

rows = list()
for _,row in delta_df.iterrows():
    rows += [row]*40
aux_delta = pd.DataFrame(rows).reset_index(drop=True)

# Banda theta
df = pd.DataFrame(theta_bands)
theta_df = df.transpose()
theta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
theta_df

rows = list()
for _,row in theta_df.iterrows():
    rows += [row]*40
aux_theta = pd.DataFrame(rows).reset_index(drop=True)

#Bnada alpha
df = pd.DataFrame(alpha_bands)
alpha_df = df.transpose()
alpha_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
alpha_df

rows = list()
for _,row in alpha_df.iterrows():
    rows += [row]*40
aux_alpha = pd.DataFrame(rows).reset_index(drop=True)

# Banda beta
df = pd.DataFrame(beta_bands)
beta_df = df.transpose()
beta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
beta_df

rows = list()
for _,row in beta_df.iterrows():
    rows += [row]*40
aux_beta = pd.DataFrame(rows).reset_index(drop=True)

# Banda gamma
df = pd.DataFrame(gamma_bands)
gamma_df = df.transpose()
gamma_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
gamma_df

rows = list()
for _,row in gamma_df.iterrows():
    rows += [row]*40
aux_gamma = pd.DataFrame(rows).reset_index(drop=True)

df_labels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])
rows = list()
for _,row in dfLabels.iterrows():
    rows += [row]*64
aux_labels = pd.DataFrame(rows).reset_index(drop=True)

data_delta = aux_delta.merge(aux_labels, left_index=True, right_index=True)
data_theta = aux_theta.merge(aux_labels, left_index=True, right_index=True)
data_alpha = aux_alpha.merge(aux_labels, left_index=True, right_index=True)
data_beta = aux_beta.merge(aux_labels, left_index=True, right_index=True)
data_gamma = aux_gamma.merge(aux_labels, left_index=True, right_index=True)

frames = [aux_delta, aux_theta, aux_alpha, aux_beta, aux_gamma]
aux_all_bands = pd.concat(frames, axis=1)
all_bands_df = aux_all_bands.merge(aux_labels, left_index=True, right_index=True)

data = all_bands_df.drop(['dominance', 'liking'], axis=1)

conditions = [
    ((data['valence'] <4.5) & (data['arousal']<4.5)),
    ((data['valence'] <4.5) & (data['arousal'] > 4.5)),
    ((data['valence'] >4.5) & (data['arousal']<4.5)),
    ((data['valence'] >4.5) & (data['arousal'] > 4.5))]
choices = ['sadness', 'pleasure', 'anger', 'joy']

data['valence'] = np.select(conditions, choices, default='black')
data = data.rename(columns={'valence' : 'label'})
data = data.drop(columns='arousal')

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['label']]

x = copia.drop(columns=['label'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment6_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)
    
rf_acc = accuracy_score(yTest, predict)
svm_acc = accuracy_score(yTest, svm_predict)
knn_acc = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_sadness = precision_score(yTest, predict, average="micro", labels=["sadness"])
rf_prec_pleasure = precision_score(yTest, predict, average="micro", labels=["pleasure"])
rf_prec_anger = precision_score(yTest, predict, average="micro", labels=["anger"])
rf_prec_joy = precision_score(yTest, predict, average="micro", labels=["joy"])


svm_prec_sadness = precision_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_prec_pleasure = precision_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_prec_anger = precision_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_prec_joy = precision_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_prec_sadness = precision_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_prec_pleasure = precision_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_prec_anger = precision_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_prec_joy = precision_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import recall_score

rf_rec_sadness = recall_score(yTest, predict, average="micro", labels=["sadness"])
rf_rec_pleasure = recall_score(yTest, predict, average="micro", labels=["pleasure"])
rf_rec_anger = recall_score(yTest, predict, average="micro", labels=["anger"])
rf_rec_joy = recall_score(yTest, predict, average="micro", labels=["joy"])


svm_rec_sadness = recall_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_rec_pleasure = recall_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_rec_anger = recall_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_rec_joy = recall_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_rec_sadness = recall_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_rec_pleasure = recall_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_rec_anger = recall_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_rec_joy = recall_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import f1_score

rf_f1_sadness = f1_score(yTest, predict, average="micro", labels=["sadness"])
rf_f1_pleasure = f1_score(yTest, predict, average="micro", labels=["pleasure"])
rf_f1_anger = f1_score(yTest, predict, average="micro", labels=["anger"])
rf_f1_joy = f1_score(yTest, predict, average="micro", labels=["joy"])


svm_f1_sadness = f1_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_f1_pleasure = f1_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_f1_anger = f1_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_f1_joy = f1_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_f1_sadness = f1_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_f1_pleasure = f1_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_f1_anger = f1_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_f1_joy = f1_score(yTest, knn_predict, average="micro", labels=["joy"])

data_CSV = [subject_id, 6, "STFT", "All bands", "-", "Random Forest", rf_acc, rf_prec_sadness, rf_prec_pleasure, rf_prec_anger, rf_prec_joy, rf_rec_sadness, rf_rec_pleasure, rf_rec_anger, rf_rec_joy, rf_f1_sadness, rf_f1_pleasure, rf_f1_anger, rf_f1_joy], [subject_id, 6, "STFT", "All bands", "-", "SVM", svm_acc, svm_prec_sadness, svm_prec_pleasure, svm_prec_anger, svm_prec_joy, svm_rec_sadness, svm_rec_pleasure, svm_rec_anger, svm_rec_joy, svm_f1_sadness, svm_f1_pleasure, svm_f1_anger, svm_f1_joy], [subject_id, 6, "STFT", "All bands", "-", "kNN", knn_acc, knn_prec_sadness, knn_prec_pleasure, knn_prec_anger, knn_prec_joy, knn_rec_sadness, knn_rec_pleasure, knn_rec_anger, knn_rec_joy, knn_f1_sadness, knn_f1_pleasure, knn_f1_anger, knn_f1_joy]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment6_1.csv", mode="a", header=False, index=False)

<ipython-input-38-05678e9fb950>:120: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['valence'] = np.select(conditions, choices, default='black')
[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  45 tasks      | elapsed:    2.4s
[Parallel(n_jobs=5)]: Done  91 out of 100 | elapsed:    2.5s remaining:    0.2s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    2.5s finished
[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  80 task

KeyError: 0

In [ ]:
f = "data_preprocessed_python\\s32.dat"
subject_id = 32
dictRaw = pd.read_pickle(f)

labels = dictRaw.get('labels')

dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

data = dictRaw.get('data')

df_videos = {}
    #Recorrido de cada una de las columnas de la tabla 3d, y guardado en una una lista de dataframes para cada uno de los videos.
for i in range(40):
    df_videos[i] = pd.DataFrame(data[i])
    df_videos[i] = df_videos[i].transpose()
    df_videos[i].columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature']
    df_videos[i].drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature'],axis = 'columns', inplace=True)

for video in range(0, 40):
    delta_bands = []
    theta_bands = []
    beta_bands = []
    alpha_bands = []
    gamma_bands = []

    for channel in ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']:
        Zxx = do_stft(video, channel)
        eeg_band_fft = make_bands(Zxx)
     #   print(video)
      #  print("\n")
       # print(channel)
        freq = channel_freq(eeg_band_fft)

        delta_bands.append(freq[0])
        theta_bands.append(freq[1])
        alpha_bands.append(freq[2])
        beta_bands.append(freq[3])
        gamma_bands.append(freq[4])

# Banda delta
df = pd.DataFrame(delta_bands)
delta_df = df.transpose()
delta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
delta_df

rows = list()
for _,row in delta_df.iterrows():
    rows += [row]*40
aux_delta = pd.DataFrame(rows).reset_index(drop=True)

# Banda theta
df = pd.DataFrame(theta_bands)
theta_df = df.transpose()
theta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
theta_df

rows = list()
for _,row in theta_df.iterrows():
    rows += [row]*40
aux_theta = pd.DataFrame(rows).reset_index(drop=True)

#Bnada alpha
df = pd.DataFrame(alpha_bands)
alpha_df = df.transpose()
alpha_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
alpha_df

rows = list()
for _,row in alpha_df.iterrows():
    rows += [row]*40
aux_alpha = pd.DataFrame(rows).reset_index(drop=True)

# Banda beta
df = pd.DataFrame(beta_bands)
beta_df = df.transpose()
beta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
beta_df

rows = list()
for _,row in beta_df.iterrows():
    rows += [row]*40
aux_beta = pd.DataFrame(rows).reset_index(drop=True)

# Banda gamma
df = pd.DataFrame(gamma_bands)
gamma_df = df.transpose()
gamma_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
gamma_df

rows = list()
for _,row in gamma_df.iterrows():
    rows += [row]*40
aux_gamma = pd.DataFrame(rows).reset_index(drop=True)

df_labels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])
rows = list()
for _,row in dfLabels.iterrows():
    rows += [row]*64
aux_labels = pd.DataFrame(rows).reset_index(drop=True)

data_delta = aux_delta.merge(aux_labels, left_index=True, right_index=True)
data_theta = aux_theta.merge(aux_labels, left_index=True, right_index=True)
data_alpha = aux_alpha.merge(aux_labels, left_index=True, right_index=True)
data_beta = aux_beta.merge(aux_labels, left_index=True, right_index=True)
data_gamma = aux_gamma.merge(aux_labels, left_index=True, right_index=True)

frames = [aux_delta, aux_theta, aux_alpha, aux_beta, aux_gamma]
aux_all_bands = pd.concat(frames, axis=1)
all_bands_df = aux_all_bands.merge(aux_labels, left_index=True, right_index=True)

data = all_bands_df.drop(['dominance', 'liking'], axis=1)

conditions = [
    ((data['valence'] <4.5) & (data['arousal']<4.5)),
    ((data['valence'] <4.5) & (data['arousal'] > 4.5)),
    ((data['valence'] >4.5) & (data['arousal']<4.5)),
    ((data['valence'] >4.5) & (data['arousal'] > 4.5))]
choices = ['sadness', 'pleasure', 'anger', 'joy']

data['valence'] = np.select(conditions, choices, default='black')
data = data.rename(columns={'valence' : 'label'})
data = data.drop(columns='arousal')

from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['label']]

x = copia.drop(columns=['label'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
#print(rnd_svm_search.best_params_)
svm_predict = model.predict(xTest)

from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment6_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)
    
rf_acc = accuracy_score(yTest, predict)
svm_acc = accuracy_score(yTest, svm_predict)
knn_acc = accuracy_score(yTest, knn_predict)

from sklearn.metrics import precision_score

rf_prec_sadness = precision_score(yTest, predict, average="micro", labels=["sadness"])
rf_prec_pleasure = precision_score(yTest, predict, average="micro", labels=["pleasure"])
rf_prec_anger = precision_score(yTest, predict, average="micro", labels=["anger"])
rf_prec_joy = precision_score(yTest, predict, average="micro", labels=["joy"])


svm_prec_sadness = precision_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_prec_pleasure = precision_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_prec_anger = precision_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_prec_joy = precision_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_prec_sadness = precision_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_prec_pleasure = precision_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_prec_anger = precision_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_prec_joy = precision_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import recall_score

rf_rec_sadness = recall_score(yTest, predict, average="micro", labels=["sadness"])
rf_rec_pleasure = recall_score(yTest, predict, average="micro", labels=["pleasure"])
rf_rec_anger = recall_score(yTest, predict, average="micro", labels=["anger"])
rf_rec_joy = recall_score(yTest, predict, average="micro", labels=["joy"])


svm_rec_sadness = recall_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_rec_pleasure = recall_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_rec_anger = recall_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_rec_joy = recall_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_rec_sadness = recall_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_rec_pleasure = recall_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_rec_anger = recall_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_rec_joy = recall_score(yTest, knn_predict, average="micro", labels=["joy"])

from sklearn.metrics import f1_score

rf_f1_sadness = f1_score(yTest, predict, average="micro", labels=["sadness"])
rf_f1_pleasure = f1_score(yTest, predict, average="micro", labels=["pleasure"])
rf_f1_anger = f1_score(yTest, predict, average="micro", labels=["anger"])
rf_f1_joy = f1_score(yTest, predict, average="micro", labels=["joy"])


svm_f1_sadness = f1_score(yTest, svm_predict, average="micro", labels=["sadness"])
svm_f1_pleasure = f1_score(yTest, svm_predict, average="micro", labels=["pleasure"])
svm_f1_anger = f1_score(yTest, svm_predict, average="micro", labels=["anger"])
svm_f1_joy = f1_score(yTest, svm_predict, average="micro", labels=["joy"])

knn_f1_sadness = f1_score(yTest, knn_predict, average="micro", labels=["sadness"])
knn_f1_pleasure = f1_score(yTest, knn_predict, average="micro", labels=["pleasure"])
knn_f1_anger = f1_score(yTest, knn_predict, average="micro", labels=["anger"])
knn_f1_joy = f1_score(yTest, knn_predict, average="micro", labels=["joy"])

data_CSV = [subject_id, 6, "STFT", "All bands", "-", "Random Forest", rf_acc, rf_prec_sadness, rf_prec_pleasure, rf_prec_anger, rf_prec_joy, rf_rec_sadness, rf_rec_pleasure, rf_rec_anger, rf_rec_joy, rf_f1_sadness, rf_f1_pleasure, rf_f1_anger, rf_f1_joy], [subject_id, 6, "STFT", "All bands", "-", "SVM", svm_acc, svm_prec_sadness, svm_prec_pleasure, svm_prec_anger, svm_prec_joy, svm_rec_sadness, svm_rec_pleasure, svm_rec_anger, svm_rec_joy, svm_f1_sadness, svm_f1_pleasure, svm_f1_anger, svm_f1_joy], [subject_id, 6, "STFT", "All bands", "-", "kNN", knn_acc, knn_prec_sadness, knn_prec_pleasure, knn_prec_anger, knn_prec_joy, knn_rec_sadness, knn_rec_pleasure, knn_rec_anger, knn_rec_joy, knn_f1_sadness, knn_f1_pleasure, knn_f1_anger, knn_f1_joy]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment6_1.csv", mode="a", header=False, index=False)

<ipython-input-39-c814bf7c53f8>:120: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['valence'] = np.select(conditions, choices, default='black')
[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  80 tasks      | elapsed:    0.2s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  80 tasks      | elapsed:    0.1s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:  